# **Process Diagnosis**

## _ETL Workflow Notebook 6_

## Content:
1. Obtaining Statistical Process Control (SPC) charts;
2. Evaluating the Process Capability (in relation to specifications).

Marco Cesar Prado Soares, Data Scientist Specialist - Bayer Crop Science LATAM
- marcosoares.feq@gmail.com
- marco.soares@bayer.com

In [ ]:
# To install a library (e.g. tensorflow), unmark and run:
# ! pip install tensorflow
# to update a library (e.g. tensorflow), unmark and run:
# ! pip install tensorflow --upgrade
# to update pip, unmark and run:
# ! pip install pip --upgrade
# to show if a library is installed and visualize its information, unmark and run
# (e.g. tensorflow):
# ! pip show tensorflow
# To run a Python file (e.g idsw_etl.py) saved in the notebook's workspace directory,
# unmark and run:
# import idsw_etl
# or:
# import idsw_etl as etl

## **Load Python Libraries in Global Context**

In [94]:
import pandas as pd
import numpy as np

# **Function for mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [95]:
def mount_storage_system (source = 'aws', path_to_store_imported_s3_bucket = '', s3_bucket_name = None, s3_obj_prefix = None):
    
    # source = 'google' for mounting the google drive;
    # source = 'aws' for mounting an AWS S3 bucket.
    
    # THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'
    
    # path_to_store_imported_s3_bucket: path of the Python environment to which the
    # S3 bucket contents will be imported. If it is None, or if 
    # path_to_store_imported_s3_bucket = '/', bucket will be imported to the root path. 
    # Alternatively, input the path as a string (in quotes). e.g. 
    # path_to_store_imported_s3_bucket = 'copied_s3_bucket'
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # s3_obj_prefix = None. Keep it None or as an empty string (s3_obj_key_prefix = '')
    # to import the whole bucket content, instead of a single object from it.
    # Alternatively, set it as a string containing the subfolder from the bucket to import:
    # Suppose that your bucket (admin-created) has four objects with the following object 
    # keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
    # s3-dg.pdf. The s3-dg.pdf key does not have a prefix, so its object appears directly 
    # at the root level of the bucket. If you open the Development/ folder, you see 
    # the Projects.xlsx object in it.
    # Check Amazon documentation:
    # https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-keys.html
    
    # In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
    # where 'bucket' is the bucket's name, key_prefix = 'my_path/.../', without the
    # 'file.csv' (file name with extension) last part.
    
    # So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
    # a given folder (directory) of the bucket.
    # DO NOT PUT A SLASH before (to the right of) the prefix;
    # DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

    # Alternatively, provide the full path of a given file if you want to import only it:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
    # where my_file is the file's name, and ext is its extension.


    # Attention: after running this function for fetching AWS Simple Storage System (S3), 
    # your 'AWS Access key ID' and your 'Secret access key' will be requested.
    # The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
    # other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
    # and the prefix. All of these are sensitive information from the organization.
    # Therefore, after importing the information, always remember of cleaning the output of this cell
    # and of removing such information from the strings.
    # Remember that these data may contain privilege for accessing the information, so it should not
    # be used for non-authorized people.

    # Also, remember of deleting the imported files from the workspace after finishing the analysis.
    # The costs for storing the files in S3 is quite inferior than those for storing directly in the
    # workspace. Also, files stored in S3 may be accessed for other users than those with access to
    # the notebook's workspace.
    
    
    if (source == 'google'):
        
        from google.colab import drive
        # Google Colab library must be imported only in case it is
        # going to be used, for avoiding AWS compatibility issues.
        
        print("Associate the Python environment to your Google Drive account, and authorize the access in the opened window.")
        
        drive.mount('/content/drive')
        
        print("Now your Python environment is connected to your Google Drive: the root directory of your environment is now the root of your Google Drive.")
        print("In Google Colab, navigate to the folder icon (\'Files\') of the left navigation menu to find a specific folder or file in your Google Drive.")
        print("Click on the folder or file name and select the elipsis (...) icon on the right of the name to reveal the option \'Copy path\', which will give you the path to use as input for loading objects and files on your Python environment.")
        print("Caution: save your files into different directories of the Google Drive. If files are all saved in a same folder or directory, like the root path, they may not be accessible from your Python environment.")
        print("If you still cannot see the file after moving it to a different folder, reload the environment.")
    
    elif (source == 'aws'):
        
        import os
        import boto3
        # boto3 is AWS S3 Python SDK
        # sagemaker and boto3 libraries must be imported only in case 
        # they are going to be used, for avoiding 
        # Google Colab compatibility issues.
        from getpass import getpass

        # Check if path_to_store_imported_s3_bucket is None. If it is, make it the root directory:
        if ((path_to_store_imported_s3_bucket is None)|(str(path_to_store_imported_s3_bucket) == "/")):
            
            # For the S3 buckets, the path should not start with slash. Assign the empty
            # string instead:
            path_to_store_imported_s3_bucket = ""
            print("Bucket\'s content will be copied to the notebook\'s root directory.")
        
        elif (str(path_to_store_imported_s3_bucket) == ""):
            # Guarantee that the path is the empty string.
            # Avoid accessing the else condition, what would raise an error
            # since the empty string has no character of index 0
            path_to_store_imported_s3_bucket = str(path_to_store_imported_s3_bucket)
            print("Bucket\'s content will be copied to the notebook\'s root directory.")
        
        else:
            # Use the str attribute to guarantee that the path was read as a string:
            path_to_store_imported_s3_bucket = str(path_to_store_imported_s3_bucket)
            
            if(path_to_store_imported_s3_bucket[0] == "/"):
                # the first character is the slash. Let's remove it

                # In AWS, neither the prefix nor the path to which the file will be imported
                # (file from S3 to workspace) or from which the file will be exported to S3
                # (the path in the notebook's workspace) may start with slash, or the operation
                # will not be concluded. Then, we have to remove this character if it is present.

                # The slash is character 0. Then, we want all characters from character 1 (the
                # second) to character len(str(path_to_store_imported_s3_bucket)) - 1, the index
                # of the last character. So, we can slice the string from position 1 to position
                # the slicing syntax is: string[1:] - all string characters from character 1
                # string[:10] - all string characters from character 10-1 = 9 (including 9); or
                # string[1:10] - characters from 1 to 9
                # So, slice the whole string, starting from character 1:
                path_to_store_imported_s3_bucket = path_to_store_imported_s3_bucket[1:]
                # attention: even though strings may be seem as list of characters, that can be
                # sliced, we cannot neither simply assign a character to a given position nor delete
                # a character from a position.

        # Ask the user to provide the credentials:
        ACCESS_KEY = input("Enter your AWS Access Key ID here (in the right). It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
        print("\n") # line break
        SECRET_KEY = getpass("Enter your password (Secret key) here (in the right). It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
        
        # The use of 'getpass' instead of 'input' hide the password behind dots.
        # So, the password is not visible by other users and cannot be copied.
        
        print("\n")
        print("WARNING: The bucket\'s name, the prefix, the AWS access key ID, and the AWS Secret access key are all sensitive information, which may grant access to protected information from the organization.\n")
        print("After copying data from S3 to your workspace, remember of removing these information from the notebook, specially if it is going to be shared. Also, remember of removing the files from the workspace.\n")
        print("The cost for storing files in Simple Storage Service is quite inferior than the one for storing directly in SageMaker workspace. Also, files stored in S3 may be accessed for other users than those with access the notebook\'s workspace.\n")

        # Check if the user actually provided the mandatory inputs, instead
        # of putting None or empty string:
        if ((ACCESS_KEY is None) | (ACCESS_KEY == '')):
            print("AWS Access Key ID is missing. It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
            return "error"
        elif ((SECRET_KEY is None) | (SECRET_KEY == '')):
            print("AWS Secret Access Key is missing. It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
            return "error"
        elif ((s3_bucket_name is None) | (s3_bucket_name == '')):
            print ("Please, enter a valid S3 Bucket\'s name. Do not add sub-directories or folders (prefixes), only the name of the bucket itself.")
            return "error"
        
        else:
            # Use the str attribute to guarantee that all AWS parameters were properly read as strings, and not as
            # other variables (like integers or floats):
            ACCESS_KEY = str(ACCESS_KEY)
            SECRET_KEY = str(SECRET_KEY)
            s3_bucket_name = str(s3_bucket_name)
        
        if(s3_bucket_name[0] == "/"):
                # the first character is the slash. Let's remove it

                # In AWS, neither the prefix nor the path to which the file will be imported
                # (file from S3 to workspace) or from which the file will be exported to S3
                # (the path in the notebook's workspace) may start with slash, or the operation
                # will not be concluded. Then, we have to remove this character if it is present.

                # So, slice the whole string, starting from character 1 (as did for 
                # path_to_store_imported_s3_bucket):
                s3_bucket_name = s3_bucket_name[1:]

        # Remove any possible trailing (white and tab spaces) spaces
        # That may be present in the string. Use the Python string
        # rstrip method, which is the equivalent to the Trim function:
        # When no arguments are provided, the whitespaces and tabulations
        # are the removed characters
        # https://www.w3schools.com/python/ref_string_rstrip.asp?msclkid=ee2d05c3c56811ecb1d2189d9f803f65
        s3_bucket_name = s3_bucket_name.rstrip()
        ACCESS_KEY = ACCESS_KEY.rstrip()
        SECRET_KEY = SECRET_KEY.rstrip()
        # Since the user manually inputs the parameters ACCESS and SECRET_KEY,
        # it is easy to input whitespaces without noticing that.

        # Now process the non-obbligatory parameter.
        # Check if a prefix was passed as input parameter. If so, we must select only the names that start with
        # The prefix.
        # Example: in the bucket 'my_bucket' we have a directory 'dir1'.
        # In the main (root) directory, we have a file 'file1.json' like: '/file1.json'
        # If we pass the prefix 'dir1', we want only the files that start as '/dir1/'
        # such as: 'dir1/file2.json', excluding the file in the main (root) directory and excluding the files in other
        # directories. Also, we want to eliminate the file names with no extensions, like 'dir1/' or 'dir1/dir2',
        # since these object names represent folders or directories, not files.	

        if (s3_obj_prefix is None):
            print ("No prefix, specific object, or subdirectory provided.") 
            print (f"Then, retrieving all content from the bucket \'{s3_bucket_name}\'.\n")
        elif ((s3_obj_prefix == "/") | (s3_obj_prefix == '')):
            # The root directory in the bucket must not be specified starting with the slash
            # If the root "/" or the empty string '' is provided, make
            # it equivalent to None (no directory)
            s3_obj_prefix = None
            print ("No prefix, specific object, or subdirectory provided.") 
            print (f"Then, retrieving all content from the bucket \'{s3_bucket_name}\'.\n")
    
        else:
            # Since there is a prefix, use the str attribute to guarantee that the path was read as a string:
            s3_obj_prefix = str(s3_obj_prefix)
            
            if(s3_obj_prefix[0] == "/"):
                # the first character is the slash. Let's remove it

                # In AWS, neither the prefix nor the path to which the file will be imported
                # (file from S3 to workspace) or from which the file will be exported to S3
                # (the path in the notebook's workspace) may start with slash, or the operation
                # will not be concluded. Then, we have to remove this character if it is present.

                # So, slice the whole string, starting from character 1 (as did for 
                # path_to_store_imported_s3_bucket):
                s3_obj_prefix = s3_obj_prefix[1:]

            # Remove any possible trailing (white and tab spaces) spaces
            # That may be present in the string. Use the Python string
            # rstrip method, which is the equivalent to the Trim function:
            s3_obj_prefix = s3_obj_prefix.rstrip()
            
            # Store the total characters in the prefix string after removing the initial slash
            # and trailing spaces:
            prefix_len = len(s3_obj_prefix)
            
            print("AWS Access Credentials, and bucket\'s prefix, object or subdirectory provided.\n")	

            
        print ("Starting connection with the S3 bucket.\n")
        
        try:
            # Start S3 client as the object 's3_client'
            s3_client = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = SECRET_KEY)
        
            print(f"Credentials accepted by AWS. S3 client successfully started.\n")
            # An object 'data_table.xlsx' in the main (root) directory of the s3_bucket is stored in Python environment as:
            # s3.ObjectSummary(bucket_name='bucket_name', key='data_table.xlsx')
            # The name of each object is stored as the attribute 'key' of the object.
        
        except:
            
            print("Failed to connect to AWS Simple Storage Service (S3). Review if your credentials are correct.")
            print("The variable \'access_key\' must be set as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("The variable \'secret_key\' must be set as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
        
        try:
            # Connect to the bucket specified as 'bucket_name'.
            # The bucket is started as the object 's3_bucket':
            s3_bucket = s3_client.Bucket(s3_bucket_name)
            print(f"Connection with bucket \'{s3_bucket_name}\' stablished.\n")
            
        except:
            
            print("Failed to connect with the bucket, which usually happens when declaring a wrong bucket\'s name.") 
            print("Check the spelling of your bucket_name string and remember that it must be all in lower-case.\n")
                

        # Then, let's obtain a list of all objects in the bucket (list bucket_objects):
        
        bucket_objects_list = []

        # Loop through all objects of the bucket:
        for stored_obj in s3_bucket.objects.all():
            
            # Loop through all elements 'stored_obj' from s3_bucket.objects.all()
            # Which stores the ObjectSummary for all objects in the bucket s3_bucket:
            # Let's store only the key attribute and use the str function
            # to guarantee that all values were stored as strings.
            bucket_objects_list.append(str(stored_obj.key))
        
        # Now start a support list to store only the elements from
        # bucket_objects_list that are not folders or directories
        # (objects with extensions).
        # If a prefix was provided, only files with that prefix should
        # be added:
        support_list = []
        
        for stored_obj in bucket_objects_list:
            
            # Loop through all elements 'stored_obj' from the list
            # bucket_objects_list

            # Check the file extension.
            file_extension = os.path.splitext(stored_obj)[1][1:]
            
            # The os.path.splitext method splits the string into its FIRST dot (".") to
            # separate the file extension from the full path. Example:
            # "C:/dir1/dir2/data_table.csv" is split into:
            # "C:/dir1/dir2/data_table" (root part) and '.csv' (extension part)
            # https://www.geeksforgeeks.org/python-os-path-splitext-method/?msclkid=2d56198fc5d311ec820530cfa4c6d574

            # os.path.splitext(stored_obj) is a tuple of strings: the first is the complete file
            # root with no extension; the second is the extension starting with a point: '.txt'
            # When we set os.path.splitext(stored_obj)[1], we are selecting the second element of
            # the tuple. By selecting os.path.splitext(stored_obj)[1][1:], we are taking this string
            # from the second character (index 1), eliminating the dot: 'txt'


            # Check if the file extension is not an empty string '' (i.e., that it is different from != the empty
            # string:
            if (file_extension != ''):
                    
                    # The extension is different from the empty string, so it is not neither a folder nor a directory
                    # The object is actually a file and may be copied if it satisfies the prefix condition. If there
                    # is no prefix to check, we may simply copy the object to the list.

                    # If there is a prefix, the first characters of the stored_obj must be the prefix:
                    if not (s3_obj_prefix is None):
                        
                        # Check the characters from the position 0 (1st character) to the position
                        # prefix_len - 1. Since a prefix was declared, we want only the objects that this first portion
                        # corresponds to the prefix. string[i:j] slices the string from index i to index j-1
                        # Then, the 1st portion of the string to check is: string[0:(prefix_len)]

                        # Slice the string stored_obj from position 0 (1st character) to position prefix_len - 1,
                        # The position that the prefix should end.
                        obj_name_first_part = (stored_obj)[0:(prefix_len)]
                        
                        # If this first part is the prefix, then append the object to 
                        # support list:
                        if (obj_name_first_part == (s3_obj_prefix)):

                                support_list.append(stored_obj)

                    else:
                        # There is no prefix, so we can simply append the object to the list:
                        support_list.append(stored_obj)

            
        # Make the objects list the support list itself:
        bucket_objects_list = support_list
            
        # Now, bucket_objects_list contains the names of all objects from the bucket that must be copied.

        print("Finished mapping objects to fetch. Now, all these objects from S3 bucket will be copied to the notebook\'s workspace, in the specified directory.\n")
        print(f"A total of {len(bucket_objects_list)} files were found in the specified bucket\'s prefix (\'{s3_obj_prefix}\').")
        print(f"The first file found is \'{bucket_objects_list[0]}\'; whereas the last file found is \'{bucket_objects_list[len(bucket_objects_list) - 1]}\'.")
            
        # Now, let's try copying the files:
            
        try:
            
            # Loop through all objects in the list bucket_objects and copy them to the workspace:
            for copied_object in bucket_objects_list:

                # Select the object in the bucket previously started as 's3_bucket':
                selected_object = s3_bucket.Object(copied_object)
            
                # Now, copy this object to the workspace:
                # Set the new file_path. Notice that by now, copied_object may be a string like:
                # 'dir1/.../dirN/file_name.ext', where dirN is the n-th directory and ext is the file extension.
                # We want only the file_name to joing with the path to store the imported bucket. So, we can use the
                # str.split method specifying the separator sep = '/' to break the string into a list of substrings.
                # The last element from this list will be 'file_name.ext'
                # https://www.w3schools.com/python/ref_string_split.asp?msclkid=135399b6c63111ecada75d7d91add056

                # 1. Break the copied_object full path into the list object_path_list, using the .split method:
                object_path_list = copied_object.split(sep = "/")

                # 2. Get the last element from this list. Since it has length len(object_path_list) and indexing starts from
                # zero, the index of the last element is (len(object_path_list) - 1):
                fetched_object = object_path_list[(len(object_path_list) - 1)]

                # 3. Finally, join the string fetched_object with the new path (path on the notebook's workspace) to finish
                # The new object's file_path:

                file_path = os.path.join(path_to_store_imported_s3_bucket, fetched_object)

                # Download the selected object to the workspace in the specified file_path
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" copies a xlsx file named 'my_table' to the notebook's main (root)
                # directory
                selected_object.download_file(Filename = file_path)

                print(f"The file \'{fetched_object}\' was successfully copied to notebook\'s workspace.\n")

                
            print("Finished copying the files from the bucket to the notebook\'s workspace. It may take a couple of minutes untill they be shown in SageMaker environment.\n") 
            print("Do not forget to delete these copies after finishing the analysis. They will remain stored in the bucket.\n")


        except:

            # Run this code for any other exception that may happen (no exception error
            # specified, so any exception runs the following code).
            # Check: https://pythonbasics.org/try-except/?msclkid=4f6b4540c5d011ecb1fe8a4566f632a6
            # for seeing how to handle successive exceptions

            print("Attention! The function raised an exception error, which is probably due to the AWS Simple Storage Service (S3) permissions.")
            print("Before running again this function, check this quick guide for configuring the permission roles in AWS.\n")
            print("It is necessary to create an user with full access permissions to interact with S3 from SageMaker. To configure the User, go to the upper ribbon of AWS, click on Services, and select IAM – Identity and Access Management.")
            print("1. In IAM\'s lateral panel, search for \'Users\' in the group of Access Management.")
            print("2. Click on the \'Add users\' button.")
            print("3. Set an user name in the text box \'User name\'.")
            print("Attention: users and S3 buckets cannot be written in upper case. Also, selecting a name already used by an Amazon user or bucket will raise an error message.\n")
            print("4. In the field \'Select type of Access to AWS\'-\'Select type of AWS credentials\' select the option \'Access key - Programmatic access\'. After that, click on the button \'Next: Permissions\'.")
            print("5. In the field \'Set Permissions\', keep the \'Add user to a group\' button marked.")
            print("6. In the field \'Add user to a group\', click on \'Create group\' (alternatively, you can be added to a group already configured or copy the permissions of another user.")
            print("7. In the text box \'Group\'s name\', set a name for the new group of permissions.")
            print("8. In the search bar below (\'Filter politics\'), search for a politics that fill your needs, and check the option button on the left of this politic. The politics \'AmazonS3FullAccess\' grants full access to the S3 content.")
            print("9. Finally, click on \'Create a group\'.")
            print("10. After the group is created, it will appear with a check box marked, over the previous groups. Keep it marked and click on the button \'Next: Tags\'.")
            print("11. Create and note down the Access key ID and Secret access key. You can also download a comma separated values (CSV) file containing the credentials for future use.")
            print("ATTENTION: These parameters are required for accessing the bucket\'s content from any application, including AWS SageMaker.")
            print("12. Click on \'Next: Review\' and review the user credentials information and permissions.")
            print("13. Click on \'Create user\' and click on the download button to download the CSV file containing the user credentials information.")
            print("The headers of the CSV file (the stored fields) is: \'User name, Password, Access key ID, Secret access key, Console login link\'.")
            print("You need both the values indicated as \'Access key ID\' and as \'Secret access key\' to fetch the S3 bucket.")
            print("\n") # line break
            print("After acquiring the necessary user privileges, use the boto3 library to fetch the bucket from the Python code. boto3 is AWS S3 Python SDK.")
            print("For fetching a specific bucket\'s file use the following code:\n")
            print("1. Set a variable \'access_key\' as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("2. Set a variable \'secret_key\' as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
            print("3. Set a variable \'bucket_name\' as a string containing only the name of the bucket. Do not add subdirectories, folders (prefixes), or file names.")
            print("Example: if your bucket is named \'my_bucket\' and its main directory contains folders like \'folder1\', \'folder2\', etc, do not declare bucket_name = \'my_bucket/folder1\', even if you only want files from folder1.")
            print("ALWAYS declare only the bucket\'s name: bucket_name = \'my_bucket\'.")
            print("4. Set a variable \'file_path\' containing the path from the bucket\'s subdirectories to the file you want to fetch. Include the file name and its extension.")
            print("If the file is stored in the bucket\'s root (main) directory: file_path = \"my_file.ext\".")
            print("If the path of the file in the bucket is: \'dir1/…/dirN/my_file.ext\', where dirN is the N-th subdirectory, and dir1 is a folder or directory of the main (root) bucket\'s directory: file_path = \"dir1/…/dirN/my_file.ext\".")
            print("Also, we say that \'dir1/…/dirN/\' is the file\'s prefix. Notice that the name of the bucket is never declared here as the path for fetching its content from the Python code.")
            print("5. Set a variable named \'new_path\' to store the path of the file copied to the notebook’s workspace. This path must contain the file name and its extension.")
            print("Example: if you want to copy \'my_file.ext\' to the root directory of the notebook’s workspace, set: new_path = \"/my_file.ext\".")
            print("6. Finally, declare the following code, which refers to the defined variables:\n")

            # Let's use triple quotes to declare a formated string
            example_code = """
                import boto3
                # Start S3 client as the object 's3_client'
                s3_client = boto3.resource('s3', aws_access_key_id = access_key, aws_secret_access_key = secret_key)
                # Connect to the bucket specified as 'bucket_name'.
                # The bucket is started as the object 's3_bucket':
                s3_bucket = s3_client.Bucket(bucket_name)
                # Select the object in the bucket previously started as 's3_bucket':
                selected_object = s3_bucket.Object(file_path)
                # Download the selected object to the workspace in the specified file_path
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" copies a xlsx file named 'my_table' to the notebook's main (root)
                # directory
                selected_object.download_file(Filename = new_path)
                """

            print(example_code)

            print("An object \'my_file.ext\' in the main (root) directory of the s3_bucket is stored in Python environment as:")
            print("""s3.ObjectSummary(bucket_name='bucket_name', key='my_file.ext'""") 
            # triple quotes to keep the internal quotes without using too much backslashes "\" (the ignore next character)
            print("Then, the name of each object is stored as the attribute \'key\' of the object. To view all objects, we can loop through their \'key\' attributes:\n")
            example_code = """
                # Loop through all objects of the bucket:
                for stored_obj in s3_bucket.objects.all():		
                    # Loop through all elements 'stored_obj' from s3_bucket.objects.all()
                    # Which stores the ObjectSummary for all objects in the bucket s3_bucket:
                    # Print the object’s names:
                    print(stored_obj.key)
                    """

            print(example_code)

                
    else:
        
        print("Select a valid source: \'google\' for mounting Google Drive; or \'aws\' for accessing AWS S3 Bucket.")

# **Function for loading the dataframe**

In [112]:
def load_pandas_dataframe (file_directory_path, file_name_with_extension, load_txt_file_with_json_format = False, how_missing_values_are_registered = None, has_header = True, decimal_separator = '.', txt_csv_col_sep = "comma", load_all_sheets_at_once = False, sheet_to_load = None, json_record_path = None, json_field_separator = "_", json_metadata_prefix_list = None):
    
    # Pandas documentation:
    # pd.read_csv: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
    # pd.read_excel: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
    # pd.json_normalize: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
    # Python JSON documentation:
    # https://docs.python.org/3/library/json.html
    
    import os
    import json
    import numpy as np
    import pandas as pd
    from pandas import json_normalize
    
    ## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, xlsm, xlsb, odf, ods and odt), 
    ## JSON, txt, or CSV (comma separated values) files.
    
    # file_directory_path - (string, in quotes): input the path of the directory (e.g. folder path) 
    # where the file is stored. e.g. file_directory_path = "/" or file_directory_path = "/folder"
    
    # FILE_NAME_WITH_EXTENSION - (string, in quotes): input the name of the file with the 
    # extension. e.g. FILE_NAME_WITH_EXTENSION = "file.xlsx", or, 
    # FILE_NAME_WITH_EXTENSION = "file.csv", "file.txt", or "file.json"
    # Again, the extensions may be: xls, xlsx, xlsm, xlsb, odf, ods, odt, json, txt or csv.
    
    # load_txt_file_with_json_format = False. Set load_txt_file_with_json_format = True 
    # if you want to read a file with txt extension containing a text formatted as JSON 
    # (but not saved as JSON).
    # WARNING: if load_txt_file_with_json_format = True, all the JSON file parameters of the 
    # function (below) must be set. If not, an error message will be raised.
    
    # HOW_MISSING_VALUES_ARE_REGISTERED = None: keep it None if missing values are registered as None,
    # empty or np.nan. Pandas automatically converts None to NumPy np.nan objects (floats).
    # This parameter manipulates the argument na_values (default: None) from Pandas functions.
    # By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, 
    #‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, 
    # ‘n/a’, ‘nan’, ‘null’.

    # If a different denomination is used, indicate it as a string. e.g.
    # HOW_MISSING_VALUES_ARE_REGISTERED = '.' will convert all strings '.' to missing values;
    # HOW_MISSING_VALUES_ARE_REGISTERED = 0 will convert zeros to missing values.

    # If dict passed, specific per-column NA values. For example, if zero is the missing value
    # only in column 'numeric_col', you can specify the following dictionary:
    # how_missing_values_are_registered = {'numeric-col': 0}
    
    
    # has_header = True if the the imported table has headers (row with columns names).
    # Alternatively, has_header = False if the dataframe does not have header.
    
    # DECIMAL_SEPARATOR = '.' - String. Keep it '.' or None to use the period ('.') as
    # the decimal separator. Alternatively, specify here the separator.
    # e.g. DECIMAL_SEPARATOR = ',' will set the comma as the separator.
    # It manipulates the argument 'decimal' from Pandas functions.
    
    # txt_csv_col_sep = "comma" - This parameter has effect only when the file is a 'txt'
    # or 'csv'. It informs how the different columns are separated.
    # Alternatively, txt_csv_col_sep = "comma", or txt_csv_col_sep = "," 
    # for columns separated by comma;
    # txt_csv_col_sep = "whitespace", or txt_csv_col_sep = " " 
    # for columns separated by simple spaces.
    # You can also set a specific separator as string. For example:
    # txt_csv_col_sep = '\s+'; or txt_csv_col_sep = '\t' (in this last example, the tabulation
    # is used as separator for the columns - '\t' represents the tab character).
    
    
    ## Parameters for loading Excel files:
    
    # load_all_sheets_at_once = False - This parameter has effect only when for Excel files.
    # If load_all_sheets_at_once = True, the function will return a list of dictionaries, each
    # dictionary containing 2 key-value pairs: the first key will be 'sheet', and its
    # value will be the name (or number) of the table (sheet). The second key will be 'df',
    # and its value will be the pandas dataframe object obtained from that sheet.
    # This argument has preference over sheet_to_load. If it is True, all sheets will be loaded.
    
    # sheet_to_load - This parameter has effect only when for Excel files.
    # keep sheet_to_load = None not to specify a sheet of the file, so that the first sheet
    # will be loaded.
    # sheet_to_load may be an integer or an string (inside quotes). sheet_to_load = 0
    # loads the first sheet (sheet with index 0); sheet_to_load = 1 loads the second sheet
    # of the file (index 1); sheet_to_load = "Sheet1" loads a sheet named as "Sheet1".
    # Declare a number to load the sheet with that index, starting from 0; or declare a
    # name to load the sheet with that name.
    
    
    ## Parameters for loading JSON files:
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_prefix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_prefix_list = ['name', 'last']
    
    
    # Create the complete file path:
    file_path = os.path.join(file_directory_path, file_name_with_extension)
    # Extract the file extension
    file_extension = os.path.splitext(file_path)[1][1:]
    # os.path.splitext(file_path) is a tuple of strings: the first is the complete file
    # root with no extension; the second is the extension starting with a point: '.txt'
    # When we set os.path.splitext(file_path)[1], we are selecting the second element of
    # the tuple. By selecting os.path.splitext(file_path)[1][1:], we are taking this string
    # from the second character (index 1), eliminating the dot: 'txt'
    
    # Check if the decimal separator is None. If it is, set it as '.' (period):
    if (decimal_separator is None):
        decimal_separator = '.'
    
    if ((file_extension == 'txt') | (file_extension == 'csv')): 
        # The operator & is equivalent to 'And' (intersection).
        # The operator | is equivalent to 'Or' (union).
        # pandas.read_csv method must be used.
        if (load_txt_file_with_json_format == True):
            
            print("Reading a txt file containing JSON parsed data. A reading error will be raised if you did not set the JSON parameters.\n")
            
            with open(file_path, 'r') as opened_file:
                # 'r' stands for read mode; 'w' stands for write mode
                # read the whole file as a string named 'file_full_text'
                file_full_text = opened_file.read()
                # if we used the readlines() method, we would be reading the
                # file by line, not the whole text at once.
                # https://stackoverflow.com/questions/8369219/how-to-read-a-text-file-into-a-string-variable-and-strip-newlines?msclkid=a772c37bbfe811ec9a314e3629df4e1e
                # https://www.tutorialkart.com/python/python-read-file-as-string/#:~:text=example.py%20%E2%80%93%20Python%20Program.%20%23open%20text%20file%20in,and%20prints%20it%20to%20the%20standard%20output.%20Output.?msclkid=a7723a1abfe811ecb68bba01a2b85bd8
                
            #Now, file_full_text is a string containing the full content of the txt file.
            json_file = json.loads(file_full_text)
            # json.load() : This method is used to parse JSON from URL or file.
            # json.loads(): This method is used to parse string with JSON content.
            # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
            # like a dataframe.
            # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
            dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_prefix_list)
        
        else:
            # Not a JSON txt
        
            if (has_header == True):

                if ((txt_csv_col_sep == "comma") | (txt_csv_col_sep == ",")):

                    dataset = pd.read_csv(file_path, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    # verbose = True for showing number of NA values placed in non-numeric columns.
                    #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                    # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                    # parsing speed by 5-10x.

                elif ((txt_csv_col_sep == "whitespace") | (txt_csv_col_sep == " ")):

                    dataset = pd.read_csv(file_path, delim_whitespace = True, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    
                    
                else:
                    
                    try:
                        
                        # Try using the character specified as the argument txt_csv_col_sep:
                        dataset = pd.read_csv(file_path, sep = txt_csv_col_sep, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    
                    except:
                        # An error was raised, the separator is not valid
                        print(f"Enter a valid column separator for the {file_extension} file, like: \'comma\' or \'whitespace\'.")


            else:
                # has_header == False

                if ((txt_csv_col_sep == "comma") | (txt_csv_col_sep == ",")):

                    dataset = pd.read_csv(file_path, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)

                    
                elif ((txt_csv_col_sep == "whitespace") | (txt_csv_col_sep == " ")):

                    dataset = pd.read_csv(file_path, delim_whitespace = True, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    
                    
                else:
                    
                    try:
                        
                        # Try using the character specified as the argument txt_csv_col_sep:
                        dataset = pd.read_csv(file_path, sep = txt_csv_col_sep, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    
                    except:
                        # An error was raised, the separator is not valid
                        print(f"Enter a valid column separator for the {file_extension} file, like: \'comma\' or \'whitespace\'.")

    elif (file_extension == 'json'):
        
        with open(file_path, 'r') as opened_file:
            
            json_file = json.load(opened_file)
            # The structure json_file = json.load(open(file_path)) relies on the GC to close the file. That's not a 
            # good idea: If someone doesn't use CPython the garbage collector might not be using refcounting (which 
            # collects unreferenced objects immediately) but e.g. collect garbage only after some time.
            # Since file handles are closed when the associated object is garbage collected or closed 
            # explicitly (.close() or .__exit__() from a context manager) the file will remain open until 
            # the GC kicks in.
            # Using 'with' ensures the file is closed as soon as the block is left - even if an exception 
            # happens inside that block, so it should always be preferred for any real application.
            # source: https://stackoverflow.com/questions/39447362/equivalent-ways-to-json-load-a-file-in-python
            
        # json.load() : This method is used to parse JSON from URL or file.
        # json.loads(): This method is used to parse string with JSON content.
        # Then, json.load for a .json file
        # and json.loads for text file containing json
        # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.   
        dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_prefix_list)
    
    else:
        # If it is not neither a csv nor a txt file, let's assume it is one of different
        # possible Excel files.
        print("Excel file inferred. If an error message is shown, check if a valid file extension was used: \'xlsx\', \'xls\', etc.\n")
        # For Excel type files, Pandas automatically detects the decimal separator and requires only the parameter parse_dates.
        # Firstly, the argument infer_datetime_format was present on read_excel function, but was removed.
        # From version 1.4 (beta, in 10 May 2022), it will be possible to pass the parameter 'decimal' to
        # read_excel function for detecting decimal cases in strings. For numeric variables, it is not needed, though
        
        if (load_all_sheets_at_once == True):
            
            # Corresponds to setting sheet_name = None
            
            if (has_header == True):
                
                xlsx_doc = pd.read_excel(file_path, sheet_name = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True)
                # verbose = True for showing number of NA values placed in non-numeric columns.
                #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                # parsing speed by 5-10x.
                
            else:
                #No header
                xlsx_doc = pd.read_excel(file_path, sheet_name = None, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True)
            
            # xlsx_doc is a dictionary containing the sheet names as keys, and dataframes as items.
            # Let's convert it to the desired format.
            # Dictionary dict, dict.keys() is the array of keys; dict.values() is an array of the values;
            # and dict.items() is an array of tuples with format ('key', value)
            
            # Create a list of returned datasets:
            list_of_datasets = []
            
            # Let's iterate through the array of tuples. The first element returned is the key, and the
            # second is the value
            for sheet_name, dataframe in (xlsx_doc.items()):
                # sheet_name = key; dataframe = value
                # Define the dictionary with the standard format:
                df_dict = {'sheet': sheet_name,
                            'df': dataframe}
                
                # Add the dictionary to the list:
                list_of_datasets.append(df_dict)
            
            print("\n")
            print(f"A total of {len(list_of_datasets)} dataframes were retrieved from the Excel file.\n")
            print(f"The dataframes correspond to the following Excel sheets: {list(xlsx_doc.keys())}\n")
            print("Returning a list of dictionaries. Each dictionary contains the key \'sheet\', with the original sheet name; and the key \'df\', with the Pandas dataframe object obtained.\n")
            print(f"Check the 10 first rows of the dataframe obtained from the first sheet, named {list_of_datasets[0]['sheet']}:\n")
            
            try:
                # only works in Jupyter Notebook:
                from IPython.display import display
                display((list_of_datasets[0]['df']).head(10))
            
            except: # regular mode
                print((list_of_datasets[0]['df']).head(10))
            
            return list_of_datasets
            
        elif (sheet_to_load is not None):        
        #Case where the user specifies which sheet of the Excel file should be loaded.
            
            if (has_header == True):
                
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True)
                # verbose = True for showing number of NA values placed in non-numeric columns.
                #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                # parsing speed by 5-10x.
                
            else:
                #No header
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True)
                
        
        else:
            #No sheet specified
            if (has_header == True):
                
                dataset = pd.read_excel(file_path, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True)
                
            else:
                #No header
                dataset = pd.read_excel(file_path, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True)
                
    print(f"Dataset extracted from {file_path}. Check the 10 first rows of this dataframe:\n")
    
    try:
        # only works in Jupyter Notebook:
        from IPython.display import display
        display(dataset.head(10))
            
    except: # regular mode
        print(dataset.head(10))
    
    return dataset

# **Function for converting JSON object to dataframe**
- Objects may be:
    - String with JSON formatted text;
    - List with nested dictionaries (JSON formatted);
    - Each dictionary may contain nested dictionaries, or nested lists of dictionaries (nested JSON).

In [113]:
def json_obj_to_pandas_dataframe (json_obj_to_convert, json_obj_type = 'list', json_record_path = None, json_field_separator = "_", json_metadata_prefix_list = None):
    
    import json
    import pandas as pd
    from pandas import json_normalize
    
    # JSON object in terms of Python structure: list of dictionaries, where each value of a
    # dictionary may be a dictionary or a list of dictionaries (nested structures).
    # example of highly nested structure saved as a list 'json_formatted_list'. Note that the same
    # structure could be declared and stored into a string variable. For instance, if you have a txt
    # file containing JSON, you could read the txt and save its content as a string.
    # json_formatted_list = [{'field1': val1, 'field2': {'dict_val': dict_val}, 'field3': [{
    # 'nest1': nest_val1}, {'nest2': nestval2}]}, {'field1': val1, 'field2': {'dict_val': dict_val}, 
    # 'field3': [{'nest1': nest_val1}, {'nest2': nestval2}]}]    

    # json_obj_type = 'list', in case the object was saved as a list of dictionaries (JSON format)
    # json_obj_type = 'string', in case it was saved as a string (text) containing JSON.

    # json_obj_to_convert: object containing JSON, or string with JSON content to parse.
    # Objects may be: string with JSON formatted text;
    # list with nested dictionaries (JSON formatted);
    # dictionaries, possibly with nested dictionaries (JSON formatted).
    
    # https://docs.python.org/3/library/json.html
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html#pandas.json_normalize
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_prefix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_prefix_list = ['name', 'last']

    
    if (json_obj_type == 'string'):
        # Use the json.loads method to convert the string to json
        json_file = json.loads(json_obj_to_convert)
        # json.load() : This method is used to parse JSON from URL or file.
        # json.loads(): This method is used to parse string with JSON content.
        # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
        # like a dataframe.
        # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
    
    elif (json_obj_type == 'list'):
        
        # make the json_file the object itself:
        json_file = json_obj_to_convert
    
    else:
        print ("Enter a valid JSON object type: \'list\', in case the JSON object is a list of dictionaries in JSON format; or \'string\', if the JSON is stored as a text (string variable).")
        return "error"
    
    dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_prefix_list)
    
    print(f"JSON object converted to a flat dataframe object. Check the 10 first rows of this dataframe:\n")
    
    try:
        # only works in Jupyter Notebook:
        from IPython.display import display
        display(dataset.head(10))
            
    except: # regular mode
        print(dataset.head(10))
    
    return dataset

# **Obtaining Statistical Process Control (SPC) charts**
- Class for creating and using the chart assistant object;
- Class for creating and using the chart object;
- Function for data processing and plot obtention.

In [114]:
class spc_chart_assistant:
            
    # Initialize instance attributes.
    # define the Class constructor, i.e., how are its objects:
    def __init__(self, assistant_startup = True, keep_assistant_on = True):
                
        import os
        
        # If the user passes the argument, use them. Otherwise, use the standard values.
        # Set the class objects' attributes.
        # Suppose the object is named assistant. We can access the attribute as:
        # assistant.assistant_startup, for instance.
        # So, we can save the variables as objects' attributes.
        self.assistant_startup = assistant_startup
        self.keep_assistant_on = keep_assistant_on
        # Base Github directory containing the assistant images to be downloaded:
        self.base_git_dir = "https://github.com/marcosoares-92/img_examples_guides/raw/main"
        # Create a new folder to store the images in local environment, 
        # if the folder do not exists:
        self.new_dir = "tmp"
        
        os.makedirs(self.new_dir, exist_ok = True)
        # exist_ok = True creates the directory only if it does not exist.
        
        self.last_img_number = 18 # number of the last image on the assistant
        self.numbers_to_end_assistant = (3, 4, 7, 9, 10, 13, 15, 16, 19, 20, 21, 22)
        # tuple: cannot be modified
        # 3: 'g', 4: 't', 7: 'i_mr', 9: 'std_error', 10: '3s', 13: 'x_bar_s'
        # 15: 'std_error' (grouped), 16: '3s' (grouped), 19: 'p', 20: 'np',
        # 21: 'c', 22: 'u'
        self.screen_number = 0 # start as zero
        self.file_to_fetch = ''
        self.img_url = ''
        self.img_local_path = ''
        # to check the class attributes, use the __dict__ method. Examples:
        ## object.__dict__ will show all attributes from object
                
    # Define the class methods.
    # All methods must take an object from the class (self) as one of the parameters
    
    def download_assistant_imgs (self):
                
        import os
        import shutil # component of the standard library to move or copy files.
        from html2image import Html2Image
                
        # Start the html object
        html_img = Html2Image()
                
        for screen_number in range(0, (self.last_img_number + 1)):
                
            # ranges from 0 to (last_img_number + 1) - 1 = last_img_number
            # convert the screen number to string to create the file name:
            
            # Update the attributes:
            self.file_to_fetch = "cc_s" + str(screen_number) + ".png"
            self.img_url = os.path.join(self.base_git_dir, self.file_to_fetch)
            
            # Download the image:
            # pypi.org/project/html2image/
            img = html_img.screenshot(url = self.img_url, save_as = self.file_to_fetch, size = (500, 500))
            # If size is omitted, the image is downloaded in the low-resolution default.
            # save_as must be a file name, a path is not accepted.
            # Make the output from the method equals to a variable eliminates its verbosity
                    
            # Create the new path for the image (local environment):
            self.img_local_path = os.path.join(self.new_dir, self.file_to_fetch)
            # Move the image files to the new paths:
            # use shutil.move(source, destination) method to move the files:
            # pynative.com/python-move-files
            # docs.python.org/3/library/shutil.html
            shutil.move(self.file_to_fetch, self.img_local_path)
            # Notice that file_to_fetch attribute still stores a file name like 'cc_s0.png'
        
        # Now, all images for the assistant were downloaded and stored in the temporary
        # folder. So, let's start the two boolean variables to initiate it and run it:
        self.assistant_startup = True 
        # attribute to start the assistant in the first screen
        self.keep_assistant_on = True
        # attribute to maintain the assistant working
        
        return self

    def delete_assistant_imgs (self):
                
        import os
        # Now, that the user closed the assistant, we can remove the downloaded files 
        # (delete them) from the notebook's workspace.
                
        # The os.remove function deletes a file or directory specified.
        for screen_number in range(0, (self.last_img_number + 1)):
                    
            self.file_to_fetch = "cc_s" + str(screen_number) + ".png"
            self.img_local_path = os.path.join(self.new_dir, self.file_to_fetch)
            os.remove(self.img_local_path)
                
        # Now that the files were removed, check if the tmp folder is empty:
        size = os.path.getsize(self.new_dir)
        # os.path.getsize returns the total size in Bytes from a folder or a file.
                
        # Get the list of sub-folders, files or subdirectories (the content) from the folder:
        list_of_contents = os.listdir(self.new_dir)
        # doc.python.org/3/library/os.html
        # It returns a list of strings representing the paths of each file or directory 
        # in the analyzed folder.
                
        # If the size is 0 and the length of the list_of_contents is also zero (i.e., there is no
        # previous sub-directory created), then remove the directory:
        if ((size == 0) & (len(list_of_contents) == 0)):
            
            os.rmdir(self.new_dir)

    def print_screen_legend (self):
        
        if (self.screen_number == 0):
            
            print("The control chart is a line graph showing a measure (y-axis) over time (x-axis).")
            
            print("In contrast to the run chart, the central line of the control chart represents the (weighted) mean, rather than the median.")
            print("Additionally, two lines representing the upper and lower control limits are shown.\n")
            print("The control limits represent the boundaries of the so-called common cause variation, which is inherent to the process.")
            print("Walther A. Shewhart, who invented the control chart, described two types of variation: chance-cause variation and assignable-cause variation.")
            print("These were later renamed to common-cause and special-cause variation.\n")
            
            print("Common-cause variation:")
            print("Is present in any process.")
            print("It is caused by phenomena that are always present within the system.")
            print("It makes the process predictable (within limits).")
            print("Common-cause variation is also called random variation or noise.\n")
                    
            print("Special-cause variation:")
            print("Is present in some processes.")
            print("It is caused by phenomena that are not normally present in the system.")
            print("It makes the process unpredictable.")
            print("Special-cause variation is also called non-random variation or signal.\n")
                    
            print("It is important to notice that neither common, nor special-cause variation is in itself 'good' or 'bad'.")
            print("A stable process may function at an unsatisfactory level; and an unstable process may be moving in the right direction.")
            print("On the other hand, the end goal of improvement is always to achieve a stable process functioning at a satisfactory level.\n")
                    
            print("Control chart limits:")
            print("The control limits, also called sigma limits, are usually placed at ±3 standard deviations from the central line.")
            print("So, the standard deviation is estimated as the common variation of the process of interest.")
            print("This variation depends on the theoretical distribution of data.")
            print("It is a beginner's mistake to simply calculate the standard deviation of all the data points.")
            print("This procedure would include both the common and special-cause variation in the calculus.")
            print("Since the calculations of control limits depend on the type of data (distribution), many types of control charts have been developed for specific purposes.")
        
        elif (self.screen_number == 1):
                    
            print("CHARTS FOR RARE EVENTS\n")
            print("ATTENTION: Due not previously group data in this case. Since events are rare, they are likely to be eliminated during aggregation.\n")
                    
            print("G-chart for units produced between (rare) defectives or defects;")
            print("or total events between successive rare occurrences:\n")
            print("When defects or defectives are rare and the subgroups are small, C, U, and P-charts become useless.")
            print("That is because most subgroups will have no defects.")
                    
            print("Example: if 8% of discharged patients have a hospitals-acquired pressure ulcer, and the average weekly number of discharges in a small department is 10, we would, on average, expect to have less than one pressure ulcer per week.")
            print("Instead, we could plot the number of discharges between each discharge of a patient with one or more pressure ulcers.\n")
            print("The number of units between defectives is modelled by the geometric distribution.")
            print("So, the G-control chart plots counting of occurrence by number; time unit; or timestamp.\n")
                    
            print("In the example of discharged patients: the indicator is the number of discharges between each of these rare cases.")
            print("Note that the first patient with pressure ulcer is missing from the chart.")
            print("It is due to the fact that we do not know how many discharges there had been before the first patient with detected pressure ulcer.\n")
            print("The central line of the G-chart is the theoretical median of the distribution")
            print("median = mean × 0.693")
                    
            print("Since the geometric distribution is highly skewed, the median is a better representation of the process center.")
            print("Also, notice that the G-chart rarely has a lower control limit.\n")
                    
            print("T-chart for time between successive rare events:\n")
            print("Like the G-chart, the T-chart is a rare event chart.")
            print("Instead of displaying the number of cases between events (defectives), this chart represents the time between successive rare events.\n")
            print("Since time is a continuous variable, the T-chart belongs with the other charts for measure numeric data.")
            print("Then, T-chart plots the timedelta (e.g. number of days between occurrences) by the measurement, time unit, or timestamp.")
                
        elif (self.screen_number == 2):
            
            print("A quality characteristic that is measured on a numerical scale is called a variable.")
            print("Examples: length or width, temperature, and volume.\n")
            
            print("The Shewhart control charts are widely used to monitor the mean and variability of variables.")
            print("On the other hand, many quality characteristics can be expressed in terms of a numerical measurement.")
                    
            print("For example: the diameter of a bearing could be measured with a micrometer and expressed in millimeters.\n")
            print("A single measurable quality characteristic, such as a dimension, weight, or volume, is a variable.")
            print("Control charts for variables are used extensively, and are one of the primary tools used in the analize and control steps of DMAIC.")
            
            print("Many quality characteristics cannot be conveniently represented numerically, though.")
            print("In such cases, we usually classify each item inspected as either conforming or nonconforming to the specifications on that quality characteristic.")
            print("The terminology defective or nondefective is often used to identify these two classifications of product.")
            print("More recently, this terminology conforming and nonconforming has become popular.")        
            print("Quality characteristics of this type are called attributes.\n")
            
            print("Control Charts for Nonconformities (defects):")
            print("A nonconforming item is a unit of product that does not satisfy one or more of the specifications of that product.")
            print("Each specific point at which a specification is not satisfied results in a defect or nonconformity.")
            print("Consequently, a nonconforming item will contain at least one nonconformity.")
            print("However, depending on their nature and severity, it is quite possible for a unit to contain several nonconformities and not be classified as nonconforming.")
                    
            print("Example: suppose we are manufacturing personal computers. Each unit could have one or more very minor flaws in the cabinet finish,")
            print("but since these flaws do not seriously affect the unit's functional operation, it could be classified as conforming.")
            print("However, if there are too many of these flaws, the personal computer should be classified as nonconforming,")
            print("because the flaws would be very noticeable to the customer and might affect the sale of the unit.\n")
                    
            print("There are many practical situations in which we prefer to work directly with the number of defects or nonconformities,")
            print("rather than the fraction nonconforming.")
            print("These include:")
            print("1. Number of defective welds in 100 m of oil pipeline.")
            print("2. Number of broken rivets in an aircraft wing.")
            print("3. Number of functional defects in an electronic logic device.")
            print("4. Number of errors on a document, etc.\n")
                    
            print("It is possible to develop control charts for either the total number of nonconformities in a unit,")
            print("or for the average number of nonconformities per unit.\n")
            print("These control charts usually assume that the occurrence of nonconformities in samples of constant size is well modeled by the Poisson distribution.\n")
                    
            print("Essentially, this requires that the number of opportunities or potential locations for nonconformities be infinitely large;")
            print("and that the probability of occurrence of a nonconformity at any location be small and constant.")
            print("Furthermore, the inspection unit must be the same for each sample.")
            print("That is, each inspection unit must always represent an identical area of opportunity for the occurrence of nonconformities.")
                    
            print("In addition, we can count nonconformities of several different types on one unit, as long as the above conditions are satisfied for each class of nonconformity.\n")
            print("In most practical situations, these conditions will not be perfectly satisfied.")
            print("The number of opportunities for the occurrence of nonconformities may be finite,")
            print("or the probability of occurrence of nonconformities may not be constant.\n")
                    
            print("As long as these departures from the assumptions are not severe,")
            print("the Poisson model will usually work reasonably well.")
            print("There are cases, however, in which the Poisson model is completely inappropriate.")
            print("So, always check carefully the distributions.")
                    
            print("If you are not sure, use the estimatives based on more general assumptions, i.e.,")
            print("The estimative of the natural variation as 3 times the standard deviation;")
            print("or as 3 times the standard error.\n")
                    
            print("Individual samples x Grouped data")
            print("Often, we collect a batch of samples corresponding to the same conditions, and use aggregation measurements such as mean, sum, or standard deviation to represent them.")
            print("In this case, we are grouping our data, and not working with individual measurements.")
            print("In turns, we can collect individual samples: there are no repetitions, only individual measurements corresponding to different conditions.\n")
            print("Usually, time series data is collected individually: each measurement corresponds to an instant, so it is not possible to collect multiple samples corresponding to the same conditions for further grouping.")
            print("Example: instant assessment of pH, temperature, pressure, etc.\n")
            print("Naturally, we can define a time window like a day, and group values on that window.")
            print("The dynamic of the phenomena should not create significant differences between samples collected for a same window, though.")
        
        elif (self.screen_number == 5):
            
            print("CHARTS FOR NUMERICAL VARIABLES\n")
            print("When dealing with a quality characteristic that is a variable, it is usually necessary to monitor both the mean value of the quality characteristic and its variability.")
            print("The control of the process average or mean quality level is usually done with the control chart for means, or the X-bar control chart.")
            print("The process variability can be monitored with either a control chart for the standard deviation, called the s control chart, or with a control chart for the range, called an R control chart.\n")
            
            print("I and MR charts for individual measurements:")
            print("ATTENTION: The I-MR chart can only be used for data that follows the normal distribution.")
            print("That is because the calculus of the control limits are based on the strong hypothesis of normality.")
            print("If you have individual samples that do not follow the normal curve (like skewed data, or data with high kurtosis);")
            print("or data with an unknown distribution, select number 8 for using less restrictive hypotheses for the estimative of the natural variation.\n")
                    
            print("Example: in healthcare, most quality data are count data.")
            print("However, from time to time, there are measurement data present.")
            print("These data are often in the form of physiological parameters or waiting times.")
            print("e.g. a chart of birth weights from 24 babies.")
            print("If the birth weights follow the normal, you can use the individuals chart.\n")
                    
            print("Actually, there are many situations in which the sample size used for process monitoring is n = 1; that is, the sample consists of an individual unit.")
            print("Some other examples of these situations are as follows:")
            print("1. Automated inspection and measurement technology is used, and every unit manufactured is analyzed.")
            print("So, there is no basis for rational subgrouping.")
            print("2. Data comes available relatively slowly, and it is inconvenient to allow sample sizes of n > 1 to accumulate before analysis.") 
            print("The long interval between observations will cause problems with rational subgrouping.")
            print("This occurs frequently in both manufacturing and non-manufacturing situations.")
            print("3. Repeat measurements on the process differ only because of laboratory or analysis error, as in many chemical processes.")
            print("4. Multiple measurements are taken on the same unit of product, such as measuring oxide thickness at several different locations on a wafer in semiconductor manufacturing.")
            print("5. In process plants, such as papermaking, measurements on some parameter (such as coating thickness across the roll) will differ very little and produce a standard deviation that is much too small if the objective is to control coating thickness along the roll.")
                    
            print("In such situations, the control chart for individual units is useful.")
            print("In many applications of the individuals control chart, we use the moving range two successive observations as the basis of estimating the process variability.\n")
            print("I-charts are often accompanied by moving range (MR) charts, which show the absolute difference between neighbouring data points.")
            print("The purpose of the MR chart is to identify sudden changes in the (estimated) within-subgroup variation.")
            print("If any data point in the MR is above the upper control limit, one should interpret the I-chart very cautiously.\n")
        
        elif(self.screen_number == 6):
                    
            print("One important difference: numeric variables are representative of continuous data, usually in the form of real numbers (float values).")
            print("It means that its possible values cannot be counted: there is an infinite number of possible real values.")
            
            print("Categoric variables, in turn, are discrete.")        
            print("It means they can be counted, since there is a finite number of possibilities.")
            print("Such variables are usually present as strings (texts), or as ordinal (integer) numbers.")
                    
            print("If there are only two categories, we have a binary classification.")
            print("Each category can be reduced to a binary system: or the category is present, or it is not.")
            print("This is the idea for the One-Hot Encoding.")
            print("Usually, values in a binary classification are 1 or 0, so that a probability can be easily associated through the sigmoid function.\n")
                    
            print("Some examples of quality characteristics that are based on the analysis of attributes:")
            print("1. Proportion of warped automobile engine connecting rods in a day's production.")
            print("2. Number of nonfunctional semiconductor chips on a wafer.")
            print("3. Number of errors or mistakes made in completing a loan application.")
            print("4. Number of medical errors made in a hospital.\n")
        
        elif((self.screen_number == 8) | (self.screen_number == 14)):
            
            print("If you have a distribution that is not normal, like distributions with high skewness or high kurtosis,")
            print("use less restrictive methodologies to estimate the natural variation.\n")
                    
            print("You may estimate the natural variation as 3 times the standard error; or as 3 times the standard deviation.")
            print("The interval will be symmetric around the mean value.\n")
            print("Recommended: standard error, which normalizes by the total of values.")
                
        elif(self.screen_number == 11):
            
            print("CHARTS FOR NUMERICAL VARIABLES\n")
            print("When dealing with a quality characteristic that is a variable, it is usually necessary to monitor both the mean value of the quality characteristic and its variability.")
            print("The control of the process average or mean quality level is usually done with the control chart for means, or the X-bar control chart.")
            print("The process variability can be monitored with either a control chart for the standard deviation, called the s control chart, or with a control chart for the range, called an R control chart.\n")
                    
            print("X-bar and S charts for average measurements:")
            print("If there is more than one measurement of a numeric variable in each subgroup,")
            print("the Xbar and S charts will display the average and the within-subgroup standard deviation, respectively.")
            print("e.g. a chart of average birth weights per month, for babies born over last year.")
        
        elif(self.screen_number == 12):
            
            print("CHARTS FOR CATEGORICAL VARIABLES\n")
            print("There are 4 widely used attributes control charts: P, nP, U, and C.\n")
                    
            print("To illustrate them, consider a dataset containing the weekly number of hospital acquired pressure ulcers at a hospital")
            print("The hospital has 300 patients, with an average length of stay of four days.") 
            print("Each of the dataframe's 24 rows contains information for one week on: the number of discharges,")
            print("patient days; pressure ulcers; and number of discharged patients with one or more pressure ulcers.")
            print("On average, 8% of discharged patients have 1.5 hospital acquired pressure ulcers.\n")
                    
            print("Some of the charts for categorical variables are based on the definition of the fraction nonconforming.")
            print("The fraction nonconforming is defined as the ratio between:")
            print("the number of nonconforming items in a population; by the total number of items in that population.")
            print("The items may have several quality characteristics that are examined simultaneously by the inspector.")
            print("If the item does not conform to the standards of one or more of these characteristics, it is classified as nonconforming.\n")
                    
            print("ATTENTION: Although it is customary to work with fraction nonconforming,")
            print("we could also analyze the fraction conforming just as easily, resulting in a control chart of process yield.")
            print("Many manufacturing organizations operate a yield-management system at each stage of their manufacturing process,")
            print("with the first-pass yield tracked on a control chart.\n")
            
            print("Traditionally, the term 'defect' has been used to name whatever it is being analyzed through counting with control charts.\n")
            print("There is a subtle, but important, distinction between:")
            print("counting defects, e.g. number of pressure ulcers;")
            print("and counting defectives, e.g. number of patients with one or more pressure ulcers.\n")
                    
            print("Defects are expected to reflect the Poisson distribution,")
            print("while defectives reflect the binomial distribution.\n")
        
        elif(self.screen_number == 17):
                    
            print("P-charts for proportion of defective units:")
            print("The first of these relates to the fraction of nonconforming or defective product produced by a manufacturing process, and is called the control chart for fraction nonconforming, or P-chart.")
            
            print("The P chart is probably the most common control chart in healthcare.")
            print("It is used to plot the proportion (or percent) of defective units.")
            print("e.g. the proportion of patients with one or more pressure ulcers.")
                    
            print("As mentioned, defectives are modelled by the binomial distribution.")
            print("In theory, the P chart is less sensitive to special cause variation than the U chart.")
            print("That is because it discards information by dichotomising inspection units (patients) in defectives and non-defectives ignoring the fact that a unit may have more than one defect (pressure ulcers).")
            print("On the other hand, the P chart often communicates better.")
                    
            print("For most people, not to mention the press, the percent of harmed patients is easier to grasp than the the rate of pressure ulcers expressed in counts per 1000 patient days.\n")
            print("The sample fraction nonconforming is defined as the ratio of the number of nonconforming units in the sample D to the sample size n:")
            print("p = D/n")
            print("From the binomial distribution, the mean should be estimated as p, and the variance s² as p(1-p)/n.")
                    
            print("nP-Charts for number nonconforming:")
            print("It is also possible to base a control chart on the number nonconforming,")
            print("rather than on the fraction nonconforming.")
            print("This is often called as number nonconforming (nP) control chart.\n")
            
        elif(self.screen_number == 18):
            
            print("C-charts for count of defects:")
            print("In some situations, it is more convenient to deal with the number of defects or nonconformities observed,")
            print("rather than the fraction nonconforming.\n")
            
            print("So, another type of control chart, called the control chart for nonconformities, or the C chart,")
            print("is designed to deal with this case.\n")
                    
            print("In the hospital example:")
            print("The correct control chart for the number of pressure ulcers is the C-chart,")
            print("which is based on the Poisson distribution.\n")
                    
            print("As mentioned, DEFECTIVES are modelled by the BINOMIAL distribution, whereas DEFECTS are are modelled by POISSON distribution.\n")
            
            print("U-charts for rate of defects:")
            print("The control chart for nonconformities per unit, or the U-chart, is useful in situations")
            print("where the average number of nonconformities per unit is a more convenient basis for process control.\n")
                
            print("The U-chart is different from the C-chart in that it accounts for variation in the area of opportunity.")
            print("Examples:")
            print("1. Number of patients over time.")
            print("2. Number of patients between units one wishes to compare.")
            print("3. Number of patient days over time.")
            print("4. Number of patient days between units one wishes to compare.\n")
                 
            print("If there are many more patients in the hospital in the winter than in the summer,")
            print("the C-chart may falsely detect special cause variation in the raw number of pressure ulcers.\n")
            
            print("The U-chart plots the rate of defects.")
            print("A rate differs from a proportion in that the numerator and the denominator need not be of the same kind,")
            print("and that the numerator may exceed the denominator.\n")
                
            print("For example: the rate of pressure ulcers may be expressed as the number of pressure ulcers per 1000 patient days.\n")
            print("The larger the numerator, the narrower the control limits.\n")
            print("So, the main difference between U and C-charts is that U is based on the average number of nonconformities per inspection unit.\n")
            
            print("If we find x total nonconformities in a sample of n inspection units,")
            print("then the average number of nonconformities per inspection unit will be:")
            print("u = x/n")
            print("\n")
               
    def open_chart_assistant_screen (self):
                
        import os
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        from html2image import Html2Image
        from tensorflow.keras.preprocessing.image import img_to_array, load_img
        # img_to_array: convert the image into its numpy array representation
                
        if (self.assistant_startup): #run if it is True:
            
            self.screen_number = 0 # first screen
        
        if (self.screen_number not in self.numbers_to_end_assistant):
            
            self.print_screen_legend()
            # Use its own method
            
            # Update attributes:
            self.file_to_fetch = "cc_s" + str(self.screen_number) + ".png"
            # Obtain the path of the image (local environment):
            self.img_local_path = os.path.join(self.new_dir, self.file_to_fetch)
                    
            # Load the image and save it on variables:
            assistant_screen = load_img(self.img_local_path)
                    
            # show image with plt.imshow function:
            fig = plt.figure(figsize = (12, 8))
            plt.imshow(assistant_screen)
            # If the image is black and white, you can color it with a cmap as fig.set_cmap('hot')
            
            #set axis off:
            plt.axis('off')
            plt.show()
            print("\n")
            
            # Run again the assistant for next screen (keep assistant on):
            self.keep_assistant_on = True
            # In the next round, the assistant should not be restarted:
            self.assistant_startup = False
            
            screen_number = input("Enter the number you wish here (in the right), according to the shown in the image above: ")
            #convert the screen number to string:
            screen_number = str(screen_number)        
            # Strip spaces and format characters (trim):
            screen_number = screen_number.strip()        
            # We do not call the str attribute for string variables (only for iterables)
            # Convert to integer
            screen_number = int(screen_number)
            # Update the attribute:
            self.screen_number = screen_number
        
        else:
            
            # user selected a value that ends the assistant:
            self.keep_assistant_on = False
            self.assistant_startup = False
        
        # Return the booleans to the main function:
        return self
        
    def chart_selection (self):
                
        # Only if the screen is in the tuple numbers_to_end_assistant:
        if (self.screen_number in self.numbers_to_end_assistant):
                    
            # Variables are created only when requested:
            rare_events_tuple = (3, 4) # g, t
            continuous_dist_not_defined_tuple = (9, 10) # std_error, 3std
            grouped_dist_not_defined_tuple = (15, 16) # std_error, 3std
            grouped_tuple = (13, 19, 20, 21, 22) # x, p, np, c, u
            charts_map_dict = {3:'g', 4:'t', 7:'i_mr', 9:'std_error', 10:'3s_as_natural_variation',
                                13:'xbar_s', 15:'std_error', 16:'3s_as_natural_variation',
                                19:'p', 20:'np', 21:'c', 22:'u'}
                    
            chart_to_use = charts_map_dict[self.screen_number]
                    
            # Variable with subgroups, which will be updated if needed:
            column_with_labels_or_subgroups = None
                    
            # Variable for skewed distribution, which will be updated if needed:
            consider_skewed_dist_when_estimating_with_std = False
                    
            column_with_variable_to_be_analyzed = str(input("Enter here (in the right) the name or number of the column (its header) that will be analyzed with the control chart.\nDo not type it in quotes.\nKeep the exact same format of the dataset, with spaces, characters, upper and lower cases, etc (or an error will be raised): "))
            # Try to convert it to integer, if it is a number:
            try:
                # Clean the string:
                column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed.strip()
                column_with_variable_to_be_analyzed = int(column_with_variable_to_be_analyzed)
                    
            except: # simply pass
                pass
                    
            print("\n")
            
            yes_no = str(input("Do your data have a column containing timestamps or time indication (event order)?\nType yes or no, here (in the right).\nDo not type it in quotes: "))
            yes_no = yes_no.strip()        
            # convert to full lower case, independently of the user:
            yes_no = yes_no.lower()
                    
            if (yes_no == 'yes'):
                    
                print("\n")
                timestamp_tag_column = str(input("Enter here (in the right) the name or number of the column containing timestamps or time indication (event order).\nDo not type it in quotes.\nKeep the exact same format of the dataset, with spaces, characters, upper and lower cases, etc (or an error will be raised): "))
                
                # Try to convert it to integer, if it is a number:
                try:
                    timestamp_tag_column = timestamp_tag_column.strip()
                    timestamp_tag_column = int(timestamp_tag_column)
                        
                except: # simply pass
                    pass
                    
            else:
                timestamp_tag_column = None
                    
            yes_no = str(input("Do your data have a column containing event frame indication; indication for separating time windows for comparison analysis;\nstages; events to be analyzed separately; or any other indication for slicing the time axis for comparison of different means, variations, etc?\nType yes or no, here (in the right).\nDo not type it in quotes: "))
            yes_no = yes_no.strip()
            yes_no = yes_no.lower()
                    
            if (yes_no == 'yes'):
                        
                print("\n")
                column_with_event_frame_indication = str(input("Enter here (in the right) the name or number of the column containing the event frame indication.\nDo not type it in quotes.\nKeep the exact same format of the dataset, with spaces, characters, upper and lower cases, etc (or an error will be raised): "))
                        
                # Try to convert it to integer, if it is a number:
                try:
                    column_with_event_frame_indication = column_with_event_frame_indication.strip()
                    column_with_event_frame_indication = int(column_with_event_frame_indication)
                        
                except: # simply pass
                    pass
            
            else:
                column_with_event_frame_indication = None
                    
            if (self.screen_number in rare_events_tuple):
                        
                print("\n")
                print(f"How are the rare events represented in the column {column_with_variable_to_be_analyzed}?")
                print(f"Before obtaining the chart, you must have modified the {column_with_variable_to_be_analyzed} to labe these data.")
                print("The function cannot work with boolean filters. So, if a value corresponds to a rare event occurrence, modify its value to properly labelling it.")
                print("You can set a special string or a special numeric value for indicating that a particular row corresponds to a rare event.")
                print("That is because rare events occurrences must be compared against all other 'regular' events.")
                print(f"For instance, {column_with_variable_to_be_analyzed} may show a value like 'rare_event', or 'ulcer' (in our example) if it is a rare occurrence.")
                print("Also, you could input a value extremely high, like 1000000000, or extremely low, like -10000000 for marking the rare events in the column.")
                print("The chart will be obtained after finding these rare events marks on the column.\n")
                        
                rare_event_indication = str(input(f"How are the rare events represented in the column {column_with_variable_to_be_analyzed}?\nEnter here (in the right) the text or number representing a rare event.\nDo not type it in quotes.\nKeep the exact same format of the dataset, with spaces, characters, upper and lower cases, etc (or the rare events will not be localized in the dataset): "))
                        
                # Try to convert it to float, if it is a number:
                try:
                    column_with_event_frame_indication = column_with_event_frame_indication.strip()
                    column_with_event_frame_indication = float(column_with_event_frame_indication)
                
                except: # simply pass
                    pass
                        
                rare_event_timedelta_unit = str(input(f"What is the usual order of magnitude for the intervals (timedeltas) between rare events?\nEnter here (in the right).\nYou may type: year, month, day, hour, minute, or second.\nDo not type it in quotes: "))
                rare_event_timedelta_unit = rare_event_timedelta_unit.strip()
                rare_event_timedelta_unit = rare_event_timedelta_unit.lower()
                
                while (rare_event_timedelta_unit not in ['year', 'month', 'day', 'hour', 'minute', 'second']):
                    
                    rare_event_timedelta_unit = str(input("Please, enter a valid timedelta unit: year, month, day, hour, minute, or second.\nDo not type it in quotes: "))
                    rare_event_timedelta_unit = rare_event_timedelta_unit.strip()
                    rare_event_timedelta_unit = rare_event_timedelta_unit.lower()
                    
            else:
                
                rare_event_timedelta_unit = None
                rare_event_indication = None
                        
                if ((self.screen_number in grouped_dist_not_defined_tuple) | (self.screen_number in grouped_tuple)):
                            
                    print("\n")
                    column_with_labels_or_subgroups = str(input("Enter here (in the right) the name or number of the column containing the subgroups or samples for aggregating the measurements in terms of mean, standard deviation, etc.\nIt may be a column with indications like 'A', 'B', or 'C'; 'subgroup1',..., 'sample1',..., or an integer like 1, 2, 3,...\nThis column will allow grouping of rows in terms of the correspondent samples.\nDo not type it in quotes.\nKeep the exact same format of the dataset, with spaces, characters, upper and lower cases, etc (or an error will be raised): "))
                            
                    # Try to convert it to integer, if it is a number:
                    try:
                        column_with_labels_or_subgroups = column_with_labels_or_subgroups.strip()
                        column_with_labels_or_subgroups = int(column_with_labels_or_subgroups)
                    
                    except: # simply pass
                        pass
                
                if ((self.screen_number in grouped_dist_not_defined_tuple) | (self.screen_number in continuous_dist_not_defined_tuple)):
                            
                    print("\n")
                    print("Is data skewed or with high kurtosis? If it is, the median will be used as the central line estimative.")
                    print("median = mean × 0.693\n")
                            
                    yes_no = str(input("Do you want to assume a skewed (or with considerable kurtosis) distribution?\nType yes or no, here (in the right).\nDo not type it in quotes: "))
                    yes_no = yes_no.strip()
                    yes_no = yes_no.lower()
                            
                    if (yes_no == 'yes'):
                        
                        # update the boolean variable
                        consider_skewed_dist_when_estimating_with_std = True
                
                
            print("Finished mapping the variables for obtaining the control chart plots.")
            print("If an error is raised; or if the chart is not complete, check if the columns' names inputs are strictly correct.\n")
            
            return chart_to_use, column_with_labels_or_subgroups, consider_skewed_dist_when_estimating_with_std, column_with_variable_to_be_analyzed, timestamp_tag_column, column_with_event_frame_indication, rare_event_timedelta_unit, rare_event_indication

In [115]:
class spc_plot:
            
    # Initialize instance attributes.
    # define the Class constructor, i.e., how are its objects:
    def __init__ (self, dictionary, column_with_variable_to_be_analyzed, timestamp_tag_column, chart_to_use, column_with_labels_or_subgroups = None, consider_skewed_dist_when_estimating_with_std = False, rare_event_indication = None, rare_event_timedelta_unit = 'day'):
                
        import numpy as np
        import pandas as pd
        
        # If the user passes the argument, use them. Otherwise, use the standard values.
        # Set the class objects' attributes.
        # Suppose the object is named plot. We can access the attribute as:
        # plot.dictionary, for instance.
        # So, we can save the variables as objects' attributes.
        self.dictionary = dictionary
        self.df = self.dictionary['df']
        # Start the attribute number of labels with value 2 (correspondent to moving range)
        self.number_of_labels = 2
        # List the possible numeric data types for a Pandas dataframe column:
        self.numeric_dtypes = [np.int16, np.int32, np.int64, np.float16, np.float32, np.float64]
        # Start a dictionary of constants
        self.dict_of_constants = {}
        self.column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed
        # Indicate which is the timestamp column:
        self.timestamp_tag_column = timestamp_tag_column
        # Set the chart 
        self.chart_to_use = chart_to_use
        
        # Other arguments of the constructor (attributes):
        # These ones have default values to use if omitted when creating the object
        self.column_with_labels_or_subgroups = column_with_labels_or_subgroups
        self.consider_skewed_dist_when_estimating_with_std = consider_skewed_dist_when_estimating_with_std
        self.rare_event_indication = rare_event_indication 
        self.rare_event_timedelta_unit = rare_event_timedelta_unit
        # to check the class attributes, use the __dict__ method. Examples:
        ## object.__dict__ will show all attributes from object
                
    # Define the class methods.
    # All methods must take an object from the class (self) as one of the parameters
   
    # Define a dictionary of constants.
    # Each key in the dictionary corresponds to a number of samples in a subgroup.
    # number_of_labels - This variable represents the total of labels or subgroups n. 
    # If there are multiple labels, this variable will be updated later.
    
    def get_constants (self):
        
        if (self.number_of_labels < 2):
            
            self.number_of_labels = 2
            
        if (self.number_of_labels <= 25):
            
            dict_of_constants = {
                
                2: {'A':2.121, 'A2':1.880, 'A3':2.659, 'c4':0.7979, '1/c4':1.2533, 'B3':0, 'B4':3.267, 'B5':0, 'B6':2.606, 'd2':1.128, '1/d2':0.8865, 'd3':0.853, 'D1':0, 'D2':3.686, 'D3':0, 'D4':3.267},
                3: {'A':1.732, 'A2':1.023, 'A3':1.954, 'c4':0.8862, '1/c4':1.1284, 'B3':0, 'B4':2.568, 'B5':0, 'B6':2.276, 'd2':1.693, '1/d2':0.5907, 'd3':0.888, 'D1':0, 'D2':4.358, 'D3':0, 'D4':2.574},
                4: {'A':1.500, 'A2':0.729, 'A3':1.628, 'c4':0.9213, '1/c4':1.0854, 'B3':0, 'B4':2.266, 'B5':0, 'B6':2.088, 'd2':2.059, '1/d2':0.4857, 'd3':0.880, 'D1':0, 'D2':4.698, 'D3':0, 'D4':2.282},
                5: {'A':1.342, 'A2':0.577, 'A3':1.427, 'c4':0.9400, '1/c4':1.0638, 'B3':0, 'B4':2.089, 'B5':0, 'B6':1.964, 'd2':2.326, '1/d2':0.4299, 'd3':0.864, 'D1':0, 'D2':4.918, 'D3':0, 'D4':2.114},
                6: {'A':1.225, 'A2':0.483, 'A3':1.287, 'c4':0.9515, '1/c4':1.0510, 'B3':0.030, 'B4':1.970, 'B5':0.029, 'B6':1.874, 'd2':2.534, '1/d2':0.3946, 'd3':0.848, 'D1':0, 'D2':5.078, 'D3':0, 'D4':2.004},
                7: {'A':1.134, 'A2':0.419, 'A3':1.182, 'c4':0.9594, '1/c4':1.0423, 'B3':0.118, 'B4':1.882, 'B5':0.113, 'B6':1.806, 'd2':2.704, '1/d2':0.3698, 'd3':0.833, 'D1':0.204, 'D2':5.204, 'D3':0.076, 'D4':1.924},
                8: {'A':1.061, 'A2':0.373, 'A3':1.099, 'c4':0.9650, '1/c4':1.0363, 'B3':0.185, 'B4':1.815, 'B5':0.179, 'B6':1.751, 'd2':2.847, '1/d2':0.3512, 'd3':0.820, 'D1':0.388, 'D2':5.306, 'D3':0.136, 'D4':1.864},
                9: {'A':1.000, 'A2':0.337, 'A3':1.032, 'c4':0.9693, '1/c4':1.0317, 'B3':0.239, 'B4':1.761, 'B5':0.232, 'B6':1.707, 'd2':2.970, '1/d2':0.3367, 'd3':0.808, 'D1':0.547, 'D2':5.393, 'D3':0.184, 'D4':1.816},
                10: {'A':0.949, 'A2':0.308, 'A3':0.975, 'c4':0.9727, '1/c4':1.0281, 'B3':0.284, 'B4':1.716, 'B5':0.276, 'B6':1.669, 'd2':3.078, '1/d2':0.3249, 'd3':0.797, 'D1':0.687, 'D2':5.469, 'D3':0.223, 'D4':1.777},
                11: {'A':0.905, 'A2':0.285, 'A3':0.927, 'c4':0.9754, '1/c4':1.0252, 'B3':0.321, 'B4':1.679, 'B5':0.313, 'B6':1.637, 'd2':3.173, '1/d2':0.3152, 'd3':0.787, 'D1':0.811, 'D2':5.535, 'D3':0.256, 'D4':1.744},
                12: {'A':0.866, 'A2':0.266, 'A3':0.886, 'c4':0.9776, '1/c4':1.0229, 'B3':0.354, 'B4':1.646, 'B5':0.346, 'B6':1.610, 'd2':3.258, '1/d2':0.3069, 'd3':0.778, 'D1':0.922, 'D2':5.594, 'D3':0.283, 'D4':1.717},
                13: {'A':0.832, 'A2':0.249, 'A3':0.850, 'c4':0.9794, '1/c4':1.0210, 'B3':0.382, 'B4':1.618, 'B5':0.374, 'B6':1.585, 'd2':3.336, '1/d2':0.2998, 'd3':0.770, 'D1':1.025, 'D2':5.647, 'D3':0.307, 'D4':1.693},
                14: {'A':0.802, 'A2':0.235, 'A3':0.817, 'c4':0.9810, '1/c4':1.0194, 'B3':0.406, 'B4':1.594, 'B5':0.399, 'B6':1.563, 'd2':3.407, '1/d2':0.2935, 'd3':0.763, 'D1':1.118, 'D2':5.696, 'D3':0.328, 'D4':1.672},
                15: {'A':0.775, 'A2':0.223, 'A3':0.789, 'c4':0.9823, '1/c4':1.0180, 'B3':0.428, 'B4':1.572, 'B5':0.421, 'B6':1.544, 'd2':3.472, '1/d2':0.2880, 'd3':0.756, 'D1':1.203, 'D2':5.741, 'D3':0.347, 'D4':1.653},
                16: {'A':0.750, 'A2':0.212, 'A3':0.763, 'c4':0.9835, '1/c4':1.0168, 'B3':0.448, 'B4':1.552, 'B5':0.440, 'B6':1.526, 'd2':3.532, '1/d2':0.2831, 'd3':0.750, 'D1':1.282, 'D2':5.782, 'D3':0.363, 'D4':1.637},
                17: {'A':0.728, 'A2':0.203, 'A3':0.739, 'c4':0.9845, '1/c4':1.0157, 'B3':0.466, 'B4':1.534, 'B5':0.458, 'B6':1.511, 'd2':3.588, '1/d2':0.2787, 'd3':0.744, 'D1':1.356, 'D2':5.820, 'D3':0.378, 'D4':1.622},
                18: {'A':0.707, 'A2':0.194, 'A3':0.718, 'c4':0.9854, '1/c4':1.0148, 'B3':0.482, 'B4':1.518, 'B5':0.475, 'B6':1.496, 'd2':3.640, '1/d2':0.2747, 'd3':0.739, 'D1':1.424, 'D2':5.856, 'D3':0.391, 'D4':1.608},
                19: {'A':0.688, 'A2':0.187, 'A3':0.698, 'c4':0.9862, '1/c4':1.0140, 'B3':0.497, 'B4':1.503, 'B5':0.490, 'B6':1.483, 'd2':3.689, '1/d2':0.2711, 'd3':0.734, 'D1':1.487, 'D2':5.891, 'D3':0.403, 'D4':1.597},
                20: {'A':0.671, 'A2':0.180, 'A3':0.680, 'c4':0.9869, '1/c4':1.0133, 'B3':0.510, 'B4':1.490, 'B5':0.504, 'B6':1.470, 'd2':3.735, '1/d2':0.2677, 'd3':0.729, 'D1':1.549, 'D2':5.921, 'D3':0.415, 'D4':1.585},
                21: {'A':0.655, 'A2':0.173, 'A3':0.663, 'c4':0.9876, '1/c4':1.0126, 'B3':0.523, 'B4':1.477, 'B5':0.516, 'B6':1.459, 'd2':3.778, '1/d2':0.2647, 'd3':0.724, 'D1':1.605, 'D2':5.951, 'D3':0.425, 'D4':1.575},
                22: {'A':0.640, 'A2':0.167, 'A3':0.647, 'c4':0.9882, '1/c4':1.0119, 'B3':0.534, 'B4':1.466, 'B5':0.528, 'B6':1.448, 'd2':3.819, '1/d2':0.2618, 'd3':0.720, 'D1':1.659, 'D2':5.979, 'D3':0.434, 'D4':1.566},
                23: {'A':0.626, 'A2':0.162, 'A3':0.633, 'c4':0.9887, '1/c4':1.0114, 'B3':0.545, 'B4':1.455, 'B5':0.539, 'B6':1.438, 'd2':3.858, '1/d2':0.2592, 'd3':0.716, 'D1':1.710, 'D2':6.006, 'D3':0.443, 'D4':1.557},
                24: {'A':0.612, 'A2':0.157, 'A3':0.619, 'c4':0.9892, '1/c4':1.0109, 'B3':0.555, 'B4':1.445, 'B5':0.549, 'B6':1.429, 'd2':3.895, '1/d2':0.2567, 'd3':0.712, 'D1':1.759, 'D2':6.031, 'D3':0.451, 'D4':1.548},
                25: {'A':0.600, 'A2':0.153, 'A3':0.606, 'c4':0.9896, '1/c4':1.0105, 'B3':0.565, 'B4':1.435, 'B5':0.559, 'B6':1.420, 'd2':3.931, '1/d2':0.2544, 'd3':0.708, 'D1':1.806, 'D2':6.056, 'D3':0.459, 'D4':1.541},
            }
            
            # Access the key:
            dict_of_constants = dict_of_constants[self.number_of_labels]
            
        else: #>= 26
            
            dict_of_constants = {'A':(3/(self.number_of_labels**(0.5))), 'A2':0.153, 
                                 'A3':3/((4*(self.number_of_labels-1)/(4*self.number_of_labels-3))*(self.number_of_labels**(0.5))), 
                                 'c4':(4*(self.number_of_labels-1)/(4*self.number_of_labels-3)), 
                                 '1/c4':1/((4*(self.number_of_labels-1)/(4*self.number_of_labels-3))), 
                                 'B3':(1-3/(((4*(self.number_of_labels-1)/(4*self.number_of_labels-3)))*((2*(self.number_of_labels-1))**(0.5)))), 
                                 'B4':(1+3/(((4*(self.number_of_labels-1)/(4*self.number_of_labels-3)))*((2*(self.number_of_labels-1))**(0.5)))),
                                 'B5':(((4*(self.number_of_labels-1)/(4*self.number_of_labels-3)))-3/((2*(self.number_of_labels-1))**(0.5))), 
                                 'B6':(((4*(self.number_of_labels-1)/(4*self.number_of_labels-3)))+3/((2*(self.number_of_labels-1))**(0.5))), 
                                 'd2':3.931, '1/d2':0.2544, 'd3':0.708, 'D1':1.806, 'D2':6.056, 'D3':0.459, 'D4':1.541}
        
        # Update the attribute
        self.dict_of_constants = dict_of_constants
        
        return self
    
    def chart_i_mr (self):
        
        import numpy as np
        import pandas as pd
        
        # access the dataframe:
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        
        # CONTROL LIMIT EQUATIONS:
        # X-bar = (sum of measurements)/(number of measurements)
        # R = Absolute value of [(largest in subgroup) - (lowest in subgroup)]
        # Individual chart: subgroup = 1
        # R = Absolute value of [(data) - (next data)]
        # R-bar = (sum of ranges R)/(number of R values calculated)
        # Lower control limit (LCL) = X-bar - (2.66)R-bar
        # Upper control limit (UCL) = X-bar + (2.66)R-bar
        
        # loop through each row from df, starting from the second (row 1):    
        # calculate mR as the difference (Xmax - Xmin) of the difference between
        # df[column_with_variable_to_be_analyzed] on row i and the row
        # i-1. Since we do not know, in principle, which one is the maximum, we can use
        # the max and min functions from Python:
        # https://www.w3schools.com/python/ref_func_max.asp
        # https://www.w3schools.com/python/ref_func_min.asp
        # Also, the moving range here must be calculated as an absolute value
        # https://www.w3schools.com/python/ref_func_abs.asp
        
        moving_range = [abs(max((df[column_with_variable_to_be_analyzed][i]), (df[column_with_variable_to_be_analyzed][(i-1)])) - min((df[column_with_variable_to_be_analyzed][i]), (df[column_with_variable_to_be_analyzed][(i-1)]))) for i in range (1, len(df))]
        x_bar_list = [(df[column_with_variable_to_be_analyzed][i] + df[column_with_variable_to_be_analyzed][(i-1)])/2 for i in range (1, len(df))]
        
        # These lists were created from index 1. We must add a initial element to
        # make their sizes equal to the original dataset length
        # Start the list to store the moving ranges, containing only the number 0
        # for the moving range (by simple concatenation):
        moving_range = [0] + moving_range
        
        # Start the list that stores the mean values of the 2-elements subgroups
        # with the first element itself (index 0):
        x_bar_list = [df[column_with_variable_to_be_analyzed][0]] + x_bar_list
        
        # Save the moving ranges as a new column from df (it may be interesting to check it):
        df['moving_range'] = moving_range
        
        # Save x_bar_list as the column to be analyzed:
        df[column_with_variable_to_be_analyzed] = x_bar_list
        
        # Get the mean values from x_bar:
        x_bar_bar = df[column_with_variable_to_be_analyzed].mean()
        
        # Calculate the mean value of the column moving_range, and save it as r_bar.
        r_bar = df['moving_range'].mean()
        
        # Get the control chart constant A2 from the dictionary, considering n = 2 the
        # number of elements of each subgroup:
        # Apply the get_constants method to update the dict_of_constants attribute:
        self = self.get_constants()
        
        control_chart_constant = self.dict_of_constants['1/d2']
        control_chart_constant = control_chart_constant * 3
        
        # calculate the upper control limit as x_bar + (3/d2)r_bar:
        upper_cl = x_bar_bar + (control_chart_constant) * (r_bar)
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as x_bar - (3/d2)r_bar:
        lower_cl = x_bar_bar - (control_chart_constant) * (r_bar)
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = x_bar_bar
        
        # Update the dataframe in the dictionary and return it:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self

    def chart_3s (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        
        if(self.consider_skewed_dist_when_estimating_with_std):
            
            # Skewed data. Use the median:
            center = df[column_with_variable_to_be_analyzed].median()
            
        else:
            
            center = dictionary['center']
            
        # calculate the upper control limit as the mean + 3s
        upper_cl = center + 3 * (dictionary['std'])
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as the mean - 3s:
        lower_cl = center - 3 * (dictionary['std'])
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = center
        
        # Update the dataframe in the dictionary:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self
    
    def chart_std_error (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        
        n_samples = df[column_with_variable_to_be_analyzed].count()
        
        s = dictionary['std']
        std_error = s/(n_samples**(0.5))
        
        if(self.consider_skewed_dist_when_estimating_with_std):
            
            # Skewed data. Use the median:
            center = df[column_with_variable_to_be_analyzed].median()
            
        else:
            
            center = dictionary['center']
            
        # calculate the upper control limit as the mean + 3 std_error
        upper_cl = center + 3 * (std_error)
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as the mean - 3 std_error:
        lower_cl = center - 3 * (std_error)
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = center
        
        # Update the dataframe in the dictionary:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self
        
    # CONTROL CHARTS FOR SUBGROUPS 
    
    def create_grouped_df (self):
        
        import numpy as np
        import pandas as pd
        from scipy import stats
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        column_with_labels_or_subgroups = self.column_with_labels_or_subgroups
        numeric_dtypes = self.numeric_dtypes
           
        # We need to group each dataframe in terms of the subgroups stored in the variable
        # column_with_labels_or_subgroups.
        # The catehgorical or datetime columns must be aggregated in terms of mode.
        # The numeric variables must be aggregated both in terms of mean and in terms of count
        # (subgroup size)
        
        # 1. Start a list for categorical columns and other for numeric columns:
        categorical_cols = []
        numeric_cols = []
        
        # Variables to map if there are categorical or numeric variables:
        is_categorical = 0
        is_numeric = 0
        
        # 2. Loop through each column from the list of columns of the dataframe:
        for column in list(df.columns):
            
            # check the type of column:
            column_data_type = df[column].dtype
            
            if (column_data_type not in numeric_dtypes):
                
                # If the Pandas series was defined as an object, it means it is categorical
                # (string, date, etc). Also, this if captures the variables converted to datetime64
                # Append the column to the list of categorical columns:
                categorical_cols.append(column)
                
            else:
                # append the column to the list of numeric columns:
                numeric_cols.append(column)
                
        # 3. Check if column_with_labels_or_subgroups is in both lists. 
        # If it is missing, append it. We need that this column in all subsets for grouping.
        if (column_with_labels_or_subgroups not in categorical_cols):
            
            categorical_cols.append(column_with_labels_or_subgroups)
        
        if (column_with_labels_or_subgroups not in numeric_cols):
            
            numeric_cols.append(column_with_labels_or_subgroups)
        
        if (len(categorical_cols) > 1):    
            # There is at least one column plus column_with_labels_or_subgroups:
            is_categorical = 1
            
        if (len(numeric_cols) > 1):
            # There is at least one column plus column_with_labels_or_subgroups:
            is_numeric = 1
            
        # 4. Create copies of df, subsetting by type of column
        # 5. Group the dataframes by column_with_labels_or_subgroups, 
        # according to the aggregate function:
        if (is_categorical == 1):
            
            df_agg_mode = df.copy(deep = True)
            df_agg_mode = df_agg_mode[categorical_cols]
            df_agg_mode = df_agg_mode.groupby(by = column_with_labels_or_subgroups, as_index = False, sort = True).agg(stats.mode)
            
            # 6. df_agg_mode processing:
            # Loop through each column from this dataframe:
            for col_mode in list(df_agg_mode.columns):
                
                # take the mode for all columns, except the column_with_labels_or_subgroups,
                # used for grouping the dataframe. This column already has the correct value
                if (col_mode != column_with_labels_or_subgroups):
            
                    # start a list of modes:
                    list_of_modes = []

                    # Now, loop through each row from the dataset:
                    for i in range(0, len(df_agg_mode)):
                        # i = 0 to i = len(df_agg_mode) - 1

                        mode_array = df_agg_mode[col_mode][i]    

                        try:
                            # try accessing the mode
                            # mode array is like:
                            # ModeResult(mode=array([calculated_mode]), count=array([counting_of_occurrences]))
                            # To retrieve only the mode, we must access the element [0][0] from this array:
                            mode = mode_array[0][0]

                        except:
                            mode = np.nan

                        # Append it to the list of modes:
                        list_of_modes.append(mode)

                    # Finally, make the column the list of modes itself:
                    df_agg_mode[col_mode] = list_of_modes
                
                    # try to convert to datetime64 (case it is not anymore):
                    try:
                        df_agg_mode[col_mode] = df_agg_mode[col_mode].astype(np.datetime64)    

                    except:
                        # simply ignore this step in case it is not possible to parse
                        # because it is a string:
                        pass
                
        if (is_numeric == 1):
            
            df_agg_mean = df.copy(deep = True)
            df_agg_sum = df.copy(deep = True)
            df_agg_std = df.copy(deep = True)
            df_agg_count = df.copy(deep = True)
            
            df_agg_mean = df_agg_mean[numeric_cols]
            df_agg_sum = df_agg_sum[numeric_cols]
            df_agg_std = df_agg_std[numeric_cols]
            df_agg_count = df_agg_count[numeric_cols]
            
            df_agg_mean = df_agg_mean.groupby(by = column_with_labels_or_subgroups, as_index = False, sort = True).mean()
            df_agg_sum = df_agg_sum.groupby(by = column_with_labels_or_subgroups, as_index = False, sort = True).sum()
            df_agg_std = df_agg_std.groupby(by = column_with_labels_or_subgroups, as_index = False, sort = True).std()
            df_agg_count = df_agg_count.groupby(by = column_with_labels_or_subgroups, as_index = False, sort = True).count()
            # argument as_index = False: prevents the grouper variable to be set as index of the new dataframe.
            # (default: as_index = True).
            
            # 7. df_agg_count processing:
            # Here, all original columns contain only the counting of elements in each
            # label. So, let's select only the columns 'key_for_merging' and column_with_variable_to_be_analyzed:
            df_agg_count = df_agg_count[[column_with_variable_to_be_analyzed]]
            
            # Rename the columns:
            df_agg_count.columns = ['count_of_elements_by_label']
            
            # Analogously, let's keep only the colums column_with_variable_to_be_analyzed and
            # 'key_for_merging' from the dataframes df_agg_sum and df_agg_std, and rename them:
            df_agg_sum = df_agg_sum[[column_with_variable_to_be_analyzed]]
            df_agg_std = df_agg_std[[column_with_variable_to_be_analyzed]]
            
            df_agg_sum.columns = ['sum_of_values_by_label']
            df_agg_std.columns = ['std_of_values_by_label']
            
        if ((is_categorical + is_numeric) == 2):
            # Both subsets are present and the column column_with_labels_or_subgroups
            # is duplicated.
            
            # Remove this column from df_agg_mean:
            df_agg_mean = df_agg_mean.drop(columns = column_with_labels_or_subgroups)
            
            # Concatenate all dataframes:
            df = pd.concat([df_agg_mode, df_agg_mean, df_agg_sum, df_agg_std, df_agg_count], axis = 1, join = "inner")
            
        elif (is_numeric == 1):
            
            # Only the numeric dataframes are present. So, concatenate them:
            df = pd.concat([df_agg_mean, df_agg_sum, df_agg_std, df_agg_count], axis = 1, join = "inner")
            
        elif (is_categorical == 1):
            
            # There is only the categorical dataframe:
            df = df_agg_mode
            
        df = df.reset_index(drop = True)
        
        # Notice that now we have a different mean value: we have a mean value
        # of the means calculated for each subgroup. So, we must update the
        # dictionary information:    
        dictionary['center'] = df[column_with_variable_to_be_analyzed].mean()
        dictionary['sum'] = df[column_with_variable_to_be_analyzed].sum()
        dictionary['std'] = df[column_with_variable_to_be_analyzed].std()
        dictionary['var'] = df[column_with_variable_to_be_analyzed].var()
        dictionary['count'] = len(df) # Total entries from the new dataframe
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        # Notice that the number of labels is now the total of entries of the dataframe
        # grouped by labels
        self.number_of_labels = dictionary['count']
        
        return self
            
    def chart_x_bar_s (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        number_of_labels = self.number_of_labels
        
        # CONTROL LIMIT EQUATIONS:
        # X-bar = mean =  (sum of measurements)/(subgroup size)
        # s = standard deviation in each subgroup
        # s-bar = mean (s) = (sum of all s values)/(number of subgroups)
        # x-bar-bar = mean (x-bar) = (sum of all x-bar)/(number of subgroups)
        # Lower control limit (LCL) = X-bar-bar - (A3)(s-bar)
        # Upper control limit (UCL) = X-bar-bar + (A3)(s-bar) 
        
        s = df['std_of_values_by_label']
        
        s_bar = (s.sum())/(number_of_labels)
        x_bar_bar = dictionary['center']
        
        # Retrieve A3
        self = self.get_constants()
        control_chart_constant = self.dict_of_constants['A3']
        
        # calculate the upper control limit as X-bar-bar + (A3)(s-bar):
        upper_cl = x_bar_bar + (control_chart_constant) * (s_bar)
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as X-bar-bar - (A3)(s-bar):
        lower_cl = x_bar_bar - (control_chart_constant) * (s_bar)
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = x_bar_bar
        
        # Update the dataframe in the dictionary:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self
    
    def chart_p (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        number_of_labels = self.number_of_labels
        
        print("\n")
        print("Attention: before obtaining this chart, substitute the values of the analyzed binary variable by 0 or 1 (integers), or an error will be raised.")
        print("This function do not perform the automatic ordinal or One-Hot Encoding of the variables.\n")
        
        # CONTROL LIMIT EQUATIONS:
        # p-chart: control chart for proportion of defectives.
        # p = mean =  (sum of measurements)/(subgroup size)
        # pbar = (sum of subgroup defective counts)/(sum of subgroups sizes)
        # n = subgroup size
        # Lower control limit (LCL) = pbar - 3.sqrt((pbar)*(1-pbar)/n)
        # Upper control limit (UCL) = pbar + 3.sqrt((pbar)*(1-pbar)/n)
        
        count_per_label = df['count_of_elements_by_label']
        p_bar = (df['sum_of_values_by_label'].sum())/(df['count_of_elements_by_label'].sum())
        
        # calculate the upper control limit as pbar + 3.sqrt((pbar)*(1-pbar)/n):
        upper_cl = p_bar + 3 * (((p_bar)*(1 - p_bar)/(count_per_label))**(0.5))
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as pbar - 3.sqrt((pbar)*(1-pbar)/n):
        lower_cl = p_bar - 3 * (((p_bar)*(1 - p_bar)/(count_per_label))**(0.5))
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = p_bar
        
        # Update the dataframe in the dictionary:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self

    def chart_np (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        number_of_labels = self.number_of_labels
        
        print("\n")
        print("Attention: before obtaining this chart, substitute the values of the analyzed binary variable by 0 or 1 (integers), or an error will be raised.")
        print("This function do not perform the automatic ordinal or One-Hot Encoding of the variables.\n")
        
        # CONTROL LIMIT EQUATIONS:
        # np-chart: control chart for count of defectives.
        # p = mean =  (sum of measurements)/(subgroup size)
        # np = sum = subgroup defective count
        # npbar = (sum of subgroup defective counts)/(number of subgroups)
        # n = subgroup size
        # pbar = npbar/n
        # Center line: npbar
        # Lower control limit (LCL) = np - 3.sqrt((np)*(1-p))
        # Upper control limit (UCL) = np + 3.sqrt((np)*(1-p))
        # available function: **(0.5) - 0.5 power
        
        # p = mean
        p = df[column_with_variable_to_be_analyzed]
        
        # Here, the column that we want to evaluate is not the mean, but the sum.
        # Since the graphics will be plotted using the column column_with_variable_to_be_analyzed
        # Let's make this column equals to the column of sums:
        
        df[column_with_variable_to_be_analyzed] = df['sum_of_values_by_label']
        np_series = df[column_with_variable_to_be_analyzed]
        
        npbar = (df['sum_of_values_by_label'].sum())/(number_of_labels) # center
        
        # calculate the upper control limit as np + 3.sqrt((np)*(1-p)):
        upper_cl = np_series + 3 * (((np_series)*(1 - p))**(0.5))
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as np - 3.sqrt((np)*(1-p)):
        lower_cl = np_series - 3 * (((np_series)*(1 - p))**(0.5))
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = npbar
        
        # Update the dataframe in the dictionary:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self
    
    def chart_c (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        number_of_labels = self.number_of_labels
        
        # CONTROL LIMIT EQUATIONS:
        # c-chart: control chart for counts of occurrences per unit.
        # c = sum = sum of subgroup occurrences
        # cbar = (sum of subgroup occurrences)/(number of subgroups)
        # n = subgroup size
        # Lower control limit (LCL) = cbar - 3.sqrt(cbar)
        # Upper control limit (UCL) = cbar + 3.sqrt(cbar)
        
        # Here, the column that we want to evaluate is not the mean, but the sum.
        # Since the graphics will be plotted using the column column_with_variable_to_be_analyzed
        # Let's make this column equals to the column of sums:
        
        df[column_with_variable_to_be_analyzed] = df['sum_of_values_by_label']
        
        c_bar = (df['sum_of_values_by_label'].sum())/(number_of_labels)
        
        # calculate the upper control limit as cbar + 3.sqrt(cbar):
        upper_cl = c_bar + 3 * ((c_bar)**(0.5))
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as cbar - 3.sqrt(cbar):
        lower_cl = c_bar - 3 * ((c_bar)**(0.5))
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = c_bar
        
        # Update the dataframe in the dictionary:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self
    
    def chart_u (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        number_of_labels = self.number_of_labels
        
        # CONTROL LIMIT EQUATIONS:
        # u-chart: control chart for average occurrence per unit.
        # u = mean =  (subgroup count of occurrences)/(subgroup size, in units)
        # ubar = mean value of u
        # n = subgroup size
        # Lower control limit (LCL) = ubar - 3.sqrt(ubar/n)
        # Upper control limit (UCL) = ubar + 3.sqrt(ubar/n)
        
        count_per_label = df['count_of_elements_by_label']
        
        u_bar = dictionary['center']
        
        # calculate the upper control limit as ubar + 3.sqrt(ubar/n):
        upper_cl = u_bar + 3 * ((u_bar/count_per_label)**(0.5))
        
        # add a column 'upper_cl' on the dataframe with this value:
        df['upper_cl'] = upper_cl
        
        # calculate the lower control limit as ubar - 3.sqrt(ubar/n):
        lower_cl = u_bar - 3 * ((u_bar/count_per_label)**(0.5))
        
        # add a column 'lower_cl' on the dataframe with this value:
        df['lower_cl'] = lower_cl
        
        # Add a column with the mean value of the considered interval:
        df['center'] = u_bar
        
        # Update the dataframe in the dictionary:
        dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self
    
    def rare_events_chart (self):
        
        import numpy as np
        import pandas as pd
        
        dictionary = self.dictionary
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        rare_event_indication = self.rare_event_indication
        rare_event_timedelta_unit = self.rare_event_timedelta_unit
        timestamp_tag_column = self.timestamp_tag_column
        numeric_dtypes = self.numeric_dtypes
        chart_to_use = self.chart_to_use
        
        # Filter df to the rare events:
        rare_events_df = df.copy(deep = True)
        rare_events_df = rare_events_df[rare_events_df[column_with_variable_to_be_analyzed] == rare_event_indication]
        
        # rare_events_df stores only the entries for rare events.
        # Let's get a list of the indices of these entries (we did not reset the index):
        rare_events_indices = list(rare_events_df.index)
        
        # Start lists for storing the count of events between the rares and the time between
        # the rare events. Start both lists from np.nan, since we do not have information of
        # any rare event before the first one registered (np.nan is float).
        count_between_rares = [np.nan]
        timedelta_between_rares = [np.nan]
        
        # Check if the times are datetimes or not:
        column_data_type = df[timestamp_tag_column].dtype
        
        if (column_data_type not in numeric_dtypes):            
            # It is a datetime. Let's loop between successive indices:
            if (len(rare_events_indices) > 1):
                
                for i in range(0, (len(rare_events_indices)-1)):
                    # get the timedelta:
                    index_i = rare_events_indices[i]
                    index_i_plus = rare_events_indices[i + 1]  
                    
                    t_i = pd.Timestamp((df[timestamp_tag_column])[index_i], unit = 'ns')
                    t_i_plus = pd.Timestamp((df[timestamp_tag_column])[index_i_plus], unit = 'ns')
                    
                    # to slice a dataframe from row i to row j (including j): df[i:(j+1)]
                    total_events_between_rares = len(df[(index_i + 1):(index_i_plus)])
                    
                    # We sliced the dataframe from index_i + 1 not to include the rare
                    # event, so we started from the next one. Also, the last element is
                    # of index index_i_plus - 1, the element before the next rare.
                    count_between_rares.append(total_events_between_rares)
                    
                    # Calculate the timedelta:
                    # Convert to an integer representing the total of nanoseconds:
                    timedelta = pd.Timedelta(t_i_plus - t_i).delta
                    
                    if (rare_event_timedelta_unit == 'year'):
                        #1. Convert the list to seconds (1 s = 10**9 ns, where 10**9 represents
                        #the potentiation operation in Python, i.e., 10^9. e.g. 10**2 = 100):
                        timedelta = timedelta / (10**9) #in seconds
                        #2. Convert it to minutes (1 min = 60 s):
                        timedelta = timedelta / 60.0 #in minutes
                        #3. Convert it to hours (1 h = 60 min):
                        timedelta = timedelta / 60.0 #in hours
                        #4. Convert it to days (1 day = 24 h):
                        timedelta = timedelta / 24.0 #in days
                        #5. Convert it to years. 1 year = 365 days + 6 h = 365 days + 6/24 h/(h/day)
                        # = (365 + 1/4) days = 365.25 days
                        timedelta = timedelta / (365.25) #in years
                        #The .0 after the numbers guarantees a float division.
                        
                    elif (rare_event_timedelta_unit == 'month'):
                        #1. Convert the list to seconds (1 s = 10**9 ns, where 10**9 represents
                        #the potentiation operation in Python, i.e., 10^9. e.g. 10**2 = 100):
                        timedelta = timedelta / (10**9) #in seconds
                        #2. Convert it to minutes (1 min = 60 s):
                        timedelta = timedelta / 60.0 #in minutes
                        #3. Convert it to hours (1 h = 60 min):
                        timedelta = timedelta / 60.0 #in hours
                        #4. Convert it to days (1 day = 24 h):
                        timedelta = timedelta / 24.0 #in days
                        #5. Convert it to months. Consider 1 month = 30 days
                        timedelta = timedelta / (30.0) #in months
                        #The .0 after the numbers guarantees a float division.
                        
                    elif (rare_event_timedelta_unit == 'day'):
                        #1. Convert the list to seconds (1 s = 10**9 ns, where 10**9 represents
                        #the potentiation operation in Python, i.e., 10^9. e.g. 10**2 = 100):
                        timedelta = timedelta / (10**9) #in seconds
                        #2. Convert it to minutes (1 min = 60 s):
                        timedelta = timedelta / 60.0 #in minutes
                        #3. Convert it to hours (1 h = 60 min):
                        timedelta = timedelta / 60.0 #in hours
                        #4. Convert it to days (1 day = 24 h):
                        timedelta = timedelta / 24.0 #in days
                        
                    elif (rare_event_timedelta_unit == 'hour'):
                        #1. Convert the list to seconds (1 s = 10**9 ns, where 10**9 represents
                        #the potentiation operation in Python, i.e., 10^9. e.g. 10**2 = 100):
                        timedelta = timedelta / (10**9) #in seconds
                        #2. Convert it to minutes (1 min = 60 s):
                        timedelta = timedelta / 60.0 #in minutes
                        #3. Convert it to hours (1 h = 60 min):
                        timedelta = timedelta / 60.0 #in hours
                        
                    elif (rare_event_timedelta_unit == 'minute'):
                        #1. Convert the list to seconds (1 s = 10**9 ns, where 10**9 represents
                        #the potentiation operation in Python, i.e., 10^9. e.g. 10**2 = 100):
                        timedelta = timedelta / (10**9) #in seconds
                        #2. Convert it to minutes (1 min = 60 s):
                        timedelta = timedelta / 60.0 #in minutes
                        
                    elif (rare_event_timedelta_unit == 'second'):
                        #1. Convert the list to seconds (1 s = 10**9 ns, where 10**9 represents
                        #the potentiation operation in Python, i.e., 10^9. e.g. 10**2 = 100):
                        timedelta = timedelta / (10**9) #in seconds
                        
                    else:
                        timedelta = timedelta # nanoseconds.
                    
                    # Append the timedelta to the list:
                    timedelta_between_rares.append(timedelta)
            
            else:
                # There is a single rare event.
                print("There is a single rare event. Impossible to calculate timedeltas and counting between rare events.\n")
                return self
        
        else: 
            # The column is not a timestamp. Simply subtract the values to calculate the
            # timedeltas. Let's loop between successive indices:
            if (len(rare_events_indices) > 1):
                
                for i in range(0, (len(rare_events_indices)-1)):
                    
                    # get the timedelta:
                    index_i = rare_events_indices[i]
                    index_i_plus = rare_events_indices[i + 1]
                    
                    t_i = (df[timestamp_tag_column])[index_i]
                    t_i_plus = (df[timestamp_tag_column])[index_i_plus]
                    
                    timedelta = (t_i_plus - t_i)
                    
                    # to slice a dataframe from row i to row j (including j): df[i:(j+1)] 
                    total_events_between_rares= len(df[(index_i + 1):(index_i_plus)])
                    
                    count_between_rares.append(total_events_between_rares)
                    timedelta_between_rares.append(timedelta)
            else:
                # There is a single rare event.
                print("There is a single rare event. Impossible to calculate timedeltas and counting between rare events.\n")
                return self
            
        # Notice that the lists still have same number of elements of the dataframe of rares.
        
        # Now, lists have the same total elements of the rare_events_df, and can be
        # added as columns:        
        # firstly, reset the index:
        rare_events_df = rare_events_df.reset_index(drop = True)
        
        # Add the columns:
        rare_events_df['count_between_rares'] = count_between_rares
        rare_events_df['timedelta_between_rares'] = timedelta_between_rares
        
        # Now, make the rares dataframe the df itself:
        df = rare_events_df
        
        if (chart_to_use == 'g'):
            
            # Here, the column that we want to evaluate is not the mean, but the 'count_between_rares'.
            # Since the graphics will be plotted using the column column_with_variable_to_be_analyzed
            # Let's make this column equals to the column 'count_between_rares':
            df[column_with_variable_to_be_analyzed] = df['count_between_rares']
            
            g_bar = df['count_between_rares'].median()
            n_samples = len(df['count_between_rares'])
            
            try:
                p = (1/(g_bar + 1))*((n_samples - 1)/n_samples)
            
                # np.log = natural logarithm
                # https://numpy.org/doc/stable/reference/generated/numpy.log.html
                center = ((np.log(0.5))/(np.log(1 - p))) - 1

                # calculate the upper control limit as log(0.00135)/log(1-p)-1:
                upper_cl = ((np.log(0.00135))/(np.log(1 - p))) - 1

                # calculate the lower control limit as Max(0, log(1-0.00135)/log(1-p)-1):
                lower_cl = max(0, ((np.log(1 - 0.00135))/(np.log(1 - p)) - 1))
            
            except:
                # division by zero
                # Here, we are prone to it due to the obtention of the rare dataframe.
                p = np.nan
                center = np.nan
                upper_cl = np.nan
                lower_cl = np.nan
                
            # add a column 'lower_cl' on the dataframe with this value:
            df['lower_cl'] = lower_cl
            
            # add a column 'upper_cl' on the dataframe with this value:
            df['upper_cl'] = upper_cl
            
            # Add a column with the mean value of the considered interval:
            df['center'] = center
            
            # Update the dataframe in the dictionary:
            dictionary['df'] = df
            
        elif (chart_to_use == 't'):
            
            # Here, the column that we want to evaluate is not the mean, but the 'timedelta_between_rares'.
            # Since the graphics will be plotted using the column column_with_variable_to_be_analyzed
            # Let's make this column equals to the column 'timedelta_between_rares':
            df[column_with_variable_to_be_analyzed] = df['timedelta_between_rares']
            
            # Create the transformed series:
            # y = df['timedelta_between_rares']
            # y_transf = y**(1/3.6)
            y_transf = (df['timedelta_between_rares'])**(1/(3.6))
            # Now, let's create an I-MR chart for y_transf        
            moving_range = [abs(max((y_transf[i]), (y_transf[(i-1)])) - min((y_transf[i]), (y_transf[(i-1)]))) for i in range (1, len(y_transf))]
            
            # The first y_transf is np.nan. We cannot use it for calculating the averages.
            # That is because the average with np.nan is np.nan. So, let's skip the first entry,
            # by starting from index i = 2.
            y_bar = [(y_transf[i] + y_transf[(i-1)])/2 for i in range (2, len(y_transf))]
            # These lists were created from index 1. We must add a initial element to
            # make their sizes equal to the original dataset length
            
            # Start the list to store the moving ranges, containing only the number 0
            # for the moving range (by simple concatenation):
            moving_range = [0] + moving_range
            # Since we do not have the first element (which is np.nan), with index 0, let's
            # take the average corresponding to index 1, the first valid entry, as the y_transf[1]
            # itself:
            y_bar = [y_transf[1]] + y_bar
            # Notice that y_bar list did not start from index 0, but from index 1, so it has one
            # element less than moving_range list. With this strategy, we eliminated the null element
            # from the calculation of the mean.
            
            # The presence of other missing values in these lists will turn all results NaN.
            # So, let's convert the list to pandas series, and apply the mean method, which
            # automatically ignores the missing values from calculations (the function
            # np.average for np.arrays would return NaN)
            moving_range = pd.Series(moving_range)
            y_bar = pd.Series(y_bar)
            
            # Now we can get the mean values from y_bar list and moving_range:
            y_bar_bar = y_bar.mean()
            r_bar = moving_range.mean()
            # Get the control chart constant A2 from the dictionary, considering n = 2 the
            # number of elements of each subgroup:
            
            # Update the number of labels attribute for the moving range case
            self.number_of_labels = 2
            self = self.get_constants()
            
            control_chart_constant = self.dict_of_constants['1/d2']
            control_chart_constant = control_chart_constant * 3
            
            # calculate the upper control limit as y_bar_bar + (3/d2)r_bar:
            upper_cl_transf = y_bar_bar + (control_chart_constant) * (r_bar)
        
            # calculate the lower control limit as y_bar_bar - (3/d2)r_bar:
            lower_cl_transf = y_bar_bar - (control_chart_constant) * (r_bar)
          
            # Notice that these values are for the transformed variables:
            # y_transf = (df['timedelta_between_rares'])**(1/(3.6))
            
            # To reconvert to the correct time scale, we reverse this transform as:
            # (y_transf)**(3.6)
            
            # add a column 'upper_cl' on the dataframe with upper_cl_transf
            # converted to the original scale:
            df['upper_cl'] = (upper_cl_transf)**(3.6)
            
            # add a column 'lower_cl' on the dataframe with lower_cl_transf
            # converted to the original scale:
            df['lower_cl'] = (lower_cl_transf)**(3.6)
            
            # Finally, add the central line by reconverting y_bar_bar to the
            # original scale:
            df['center'] = (y_bar_bar)**(3.6)
            
            # Notice that this procedure naturally corrects the deviations caused by
            # the skewness of the distribution. Actually, log and exponential transforms
            # tend to reduce the skewness and to normalize the data.
            # Update the dataframe in the dictionary:
            dictionary['df'] = df
        
        # Update the attributes:
        self.dictionary = dictionary
        self.df = df
        
        return self

In [116]:
def statistical_process_control_chart (df, column_with_variable_to_be_analyzed, timestamp_tag_column = None, column_with_labels_or_subgroups = None, column_with_event_frame_indication = None, specification_limits = {'lower_spec_lim': None, 'upper_spec_lim': None}, reference_value = None, use_spc_chart_assistant = False, chart_to_use = 'std_error', consider_skewed_dist_when_estimating_with_std = False, rare_event_indication = None, rare_event_timedelta_unit = 'day', x_axis_rotation = 70, y_axis_rotation = 0, grid = True, horizontal_axis_title = None, vertical_axis_title = None, plot_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 330):
     
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from scipy import stats
    
    # matplotlib.colors documentation:
    # https://matplotlib.org/3.5.0/api/colors_api.html?msclkid=94286fa9d12f11ec94660321f39bf47f
    
    # Matplotlib list of colors:
    # https://matplotlib.org/stable/gallery/color/named_colors.html?msclkid=0bb86abbd12e11ecbeb0a2439e5b0d23
    # Matplotlib colors tutorial:
    # https://matplotlib.org/stable/tutorials/colors/colors.html
    # Matplotlib example of Python code using matplotlib.colors:
    # https://matplotlib.org/stable/_downloads/0843ee646a32fc214e9f09328c0cd008/colors.py
    # Same example as Jupyter Notebook:
    # https://matplotlib.org/stable/_downloads/2a7b13c059456984288f5b84b4b73f45/colors.ipynb
    
    # df: dataframe to be analyzed.
    
    # TIMESTAMP_TAG_COLUMN: name (header) of the column containing the timestamps or the numeric scale
    # used to represent time (column with floats or integers). The column name may be a string or a number.
    # e.g. TIMESTAMP_TAG_COLUMN = 'date' will use the values from column 'date'
    # Keep TIMESTAMP_TAG_COLUMN = None if the dataframe do not contain timestamps, so the index will be
    # used.

    # COLUMN_WITH_VARIABLE_TO_BE_ANALYZED: name (header) of the column containing the variable
    # which stability will be analyzed by the control chart. The column name may be a string or a number.
    # Example: COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'analyzed_column' will analyze a column named
    # "analyzed_column", whereas COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'col1' will evaluate column 'col1'.

    # COLUMN_WITH_LABELS_OR_SUBGROUPS: name (header) of the column containing the variable
    # indicating the subgroups or label indication which will be used for grouping the samples. 
    # Example: Suppose you want to analyze the means for 4 different subgroups: 1, 2, 3, 4. For each subgroup,
    # 4 or 5 samples of data (the values in COLUMN_WITH_VARIABLE_TO_BE_ANALYZED) are collected, and you
    # want the average and standard deviation within the subgroups. Them, you create a column named
    # 'label' and store the values: 1 for samples correspondent to subgroup 1; 2 for samples from
    # subgroup 2,... etc. In this case, COLUMN_WITH_LABELS_OR_SUBGROUPS = 'label'
    # Notice that the samples do not need to be collected in order. The function will automatically separate
    # the entries according to the subgroups. So, the entries in the dataset may be in an arbitrary order
    # like: 1, 1, 2, 1, 4, 3, etc.
    # The values in the COLUMN_WITH_LABELS_OR_SUBGROUPS may be strings (text) or numeric values (like
    # integers), but different values will be interpreted as different subgroups.
    # As an example of text, you could have a column named 'col1' with group identifications as: 
    # 'A', 'B', 'C', 'D', and COLUMN_WITH_LABELS_OR_SUBGROUPS = 'col1'.
    # Notice the difference between COLUMN_WITH_LABELS_OR_SUBGROUPS and COLUMN_WITH_VARIABLE_TO_BE_ANALYZED:
    # COLUMN_WITH_VARIABLE_TO_BE_ANALYZED accepts only numeric values, so the binary variables must be
    # converted to integers 0 and 1 before the analysis. The COLUMN_WITH_LABELS_OR_SUBGROUPS, in turns,
    # accept both numeric and text (string) values.
    
    # COLUMN_WITH_EVENT_FRAME_INDICATION: name (header) of the column containing the variable
    # indicating the stages, time windows, or event frames. The central line and the limits of natural
    # variation will be independently calculated for each event frame. The indication of an event frame
    # may be textual (string) or numeric. 
    # For example: suppose you have a column named 'event_frames'. For half of the entries, event_frame = 
    # 'A'; and for the second half, event_frame = 'B'. If COLUMN_WITH_EVENT_FRAME_INDICATION = 'event_frame',
    # the dataframe will be divided into two subsets: 'A', and 'B'. For each subset, the central lines
    # and the limits of natural variation will be calculated independently. So, you can check if there is
    # modification of the average value and of the dispersion when the time window is modified. It could
    # reflect, for example, the use of different operational parameters on each event frame.
    # Other possibilities of event indications: 0, 1, 2, 3, ... (sequence of integers); 'A', 'B', 'C', etc;
    # 'stage1', 'stage2', ..., 'treatment1', 'treatment2',....; 'frame0', 'frame1', 'frame2', etc.
    # ATTENTION: Do not identify different frames with the same value. For example, if
    # COLUMN_WITH_EVENT_FRAME_INDICATION has missing values for the first values, then a sequence of rows
    # is identified as number 0; followed by a sequence of missing values. In this case, the two windows
    # with missing values would be merged as a single window, and the mean and variation would be
    # obtained for this merged subset. Then, always specify different windows with different values.
    # Other example: COLUMN_WITH_EVENT_FRAME_INDICATION = 'col1' will search for event frame indications
    # in column 'col1'.
    
    # specification_limits = {'lower_spec_lim': None, 'upper_spec_lim': None}
    # If there are specification limits, input them in this dictionary. Do not modify the keys,
    # simply substitute None by the lower and/or the upper specification.
    # e.g. Suppose you have a tank that cannot have more than 10 L. So:
    # specification_limits = {'lower_spec_lim': None, 'upper_spec_lim': 10}, there is only
    # an upper specification equals to 10 (do not add units);
    # Suppose a temperature cannot be lower than 10 ºC, but there is no upper specification. So,
    # specification_limits = {'lower_spec_lim': 10, 'upper_spec_lim': None}. Finally, suppose
    # a liquid which pH must be between 6.8 and 7.2:
    # specification_limits = {'lower_spec_lim': 6.8, 'upper_spec_lim': 7.2}
    
    # REFERENCE_VALUE: keep it as None or add a float value.
    # This reference value will be shown as a red constant line to be compared
    # with the plots. e.g. REFERENCE_VALUE = 1.0 will plot a line passing through
    # VARIABLE_TO_ANALYZE = 1.0
    
    # USE_SPC_CHART_ASSISTANT = False. Set USE_SPC_CHART_ASSISTANT = True to open the 
    # visual flow chart assistant that will help you select the appropriate parameters; 
    # as well as passing the data in the correct format. If the assistant is open, many of the 
    # arguments of the function will be filled when using it.

    # chart_to_use = '3s_as_natural_variation', 'std_error', 'i_mr', 'xbar_s', 'np', 'p', 
    # 'u', 'c', 'g', 't'
    # The type of chart that will be obtained, as well as the methodology used for estimating the
    # natural variation of the process. Notice that it may be strongly dependent on the statistical
    # distribution. So, if you are not sure about the distribution, or simply want to apply a more
    # general (less restrictive) methodology, set:

    # CHART_TO_USE = '3s_as_natural_variation' to estimate the natural variation as 3 times the
    # standard deviation (s); or
    # CHART_TO_USE = 'std_error' for estimating it as 3 times the standard error, where 
    # standard error = s/(n**0.5) = s/sqrt(n), n = number of samples (that may be the number of 
    # individual data samples collected, or the number of subgroups or labels); sqrt is the square root.
    # https://en.wikipedia.org/wiki/Standard_error
    # CHART_TO_USE = '3s_as_natural_variation' and CHART_TO_USE = 'std_error' are the only ones available
    # for both individual and grouped data.
    ## ATTENTION: Do not group the variables before using the function. It will perform the automatic
    # grouping in accordance to the values specified as COLUMN_WITH_LABELS_OR_SUBGROUPS.
    # Other values allowed for CHART_TO_USE:
    # CHART_TO_USE = 'i_mr', for individual values of a numeric (continuous) variable which follows the
    # NORMAL distribution.
    # CHART_TO_USE = 'xbar_s', for grouped values (by mean) of a numeric variable, where the mean values
    # of labels or subgroups follow a NORMAL distribution.
    # CHART_TO_USE = 'np', for grouped binary variables (allowed values: 0 or 1). This is the
    # control chart for proportion of defectives. - Original data must follow the BINOMIAL distribution.
    # CHART_TO_USE = 'p', for grouped binary variables (allowed values: 0 or 1). This is the
    # control chart for count of defectives. - Original data must follow the BINOMIAL distribution.
    # Attention: an error will be raised if CHART_TO_USE = 'np' or 'p' and the variable was not converted
    # to a numeric binary, with values 0 or 1. This function do not perform the automatic ordinal or
    # One-Hot encoding of the categorical features.
    # CHART_TO_USE = 'u', for counts of occurrences per unit. - Original data must follow the POISSON
    # distribution (special case of the gamma distribution).
    # CHART_TO_USE = 'c', for average occurrence per unit. - Original data must follow the POISSON
    # distribution (special case of the gamma distribution).
    # CHARTS FOR ANALYZING RARE EVENTS
    # CHART_TO_USE = 'g', for analyzing count of events between successive rare events occurrences 
    # (data follow the GEOMETRIC distribution).
    # CHART_TO_USE = 't', for analyzing time interval between successive rare events occurrences.
    
    # consider_skewed_dist_when_estimating_with_std.
    # Whether the distribution of data to be analyzed present high skewness or kurtosis.
    # If CONSIDER_SKEWED_DISTRIBUTION_WHEN_ESTIMATING_STD = False, the central lines will be estimated
    # as the mean values of the analyzed variable. 
    # If CONSIDER_SKEWED_DISTRIBUTION_WHEN_ESTIMATING_STD = True, the central lines will be estimated 
    # as the median of the analyzed variable, which is a better alternative for skewed data such as the 
    # ones that follow geometric or lognormal distributions (median = mean × 0.693).
    # Notice that this argument has effect only when CHART_TO_USE = '3s_as_natural_variation' or 
    # CHART_TO_USE = 'std_error'.

    # RARE_EVENT_INDICATION = None. String (in quotes), float or integer. If you want to analyze a
    # rare event through 'g' or 't' control charts, this parameter is obbligatory. Also, notice that:
    # COLUMN_WITH_VARIABLE_TO_BE_ANALYZED must be the column which contains an indication of the rare
    # event occurrence, and the RARE_EVENT_INDICATION is the value of the column COLUMN_WITH_VARIABLE_TO_BE_ANALYZED
    # when a rare event takes place.
    # For instance, suppose RARE_EVENT_INDICATION = 'shutdown'. It means that column COLUMN_WITH_VARIABLE_TO_BE_ANALYZED
    # has the value 'shutdown' when the rare event occurs, i.e., for timestamps when the system
    # stopped. Other possibilities are RARE_EVENT_INDICATION = 0, or RARE_EVENT_INDICATION = -1,
    # indicating that when COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 0 (or -1), we know that
    # a rare event occurred. The most important thing here is that the value given to the rare event
    # should be assigned ONLY to the rare events.

    # You do not need to assign values for the other timestamps when no rare event took place. But it is
    # important to keep all timestamps in the dataframe. That is because the rare events charts will
    # compare the rare event occurrence against all other events and timestamps.
    # If you are not analyzing rare events with 'g' or 't' charts, keep RARE_EVENT_INDICATION = None.
    
    # RARE_EVENT_TIMEDELTA_UNIT: 'day', 'second', 'nanosecond', 'minute', 'hour',
    # 'month', 'year' - This is the unit of time that will be used to plot the time interval
    # (timedelta) between each successive rare events. If None or invalid value used, timedelta
    # will be given in days.
    # Notice that this parameter is referrent only to the rare events analysis with 'g' or 't' charts.
    # Also, it is valid only when the timetag column effectively stores a timestamp. If the timestamp
    # column stores a float or an integer (numeric) value, then the final dataframe and plot will be
    # obtained in the same numeric scale of the original data, not in the unit indicated as
    # RARE_EVENT_TIMEDELTA_UNIT.
    
    # List the possible numeric data types for a Pandas dataframe column:
    numeric_dtypes = [np.int16, np.int32, np.int64, np.float16, np.float32, np.float64]

    ## CONTROL CHARTS CALCULATION
    
    # References: 
    # Douglas C. Montgomery. Introduction to Statistical Quality Control. 6th Edition. 
    # John Wiley and Sons, 2009.
    # Jacob Anhoej. Control Charts with qicharts for R. 2021-04-20. In: https://cran.r-project.org/web/packages/qicharts/vignettes/controlcharts.html

    # CENTER LINE = m (mean)
    # s = std deviation
    # General equation of the control limits:
    # UCL = upper control limit = m + L*s
    # LCL = lower control limit = m - L*s
    # where L is a measurement of distance from central line.
    
    # for a subgroup of data collected for a continuous variable X:
    # x_bar_1 = mean value for subgroup 1,..., x_bar_n = mean for subgroup n.
    # x_bar_bar = (x_bar_1 + ... + x_bar_n)/n
    
    # On the other hand, the range R between two collected values is defined as: R = x_max - x_min
    # If R1, ... Rm are the ranges for m subgroups (which may be of size 2),
    # we have R_bar = (R1 + R2 + ... + Rm)/m
    
    # Analogously, if s_i is the standard deviation for a subgroup i and there are m subgroups:
    # s_bar = (s_1 +... +s_m)/m
    
    ## INDIVIDUAL MEASUREMENTS (Montgomery, pg.259, section 6.4)
    
    # For individual measurements, we consider subgroups formed by two consecutive measurements, so
    # m = 2, and R = abs(x_i - x_(i-1)), where abs function calculates the absolute value of the
    # difference between two successive subgroups.
    # UCL =  x_bar_bar + 3*(1/d2)*R_bar
    # LCL =  x_bar_bar - 3*(1/d2)*R_bar
    # Center line = x_bar_bar
        
    # CONTROL CHARTS FOR SUBGROUPS 
            
    # X-bar-S (continuous variables) - (Montgomery, pg.253, section 6.3.1):
    # For subgroups with m elements:
    # UCL = x_bar_bar + A3*s_bar
    # Center line = x_bar_bar
    # LCL = x_bar_bar - A3*s_bar
    
    # CONTROL CHARTS FOR CATEGORICAL VARIABLES
    
    # P-chart - (Montgomery, pg.291, section 7.2.1):
    # The sample fraction nonconforming is defined as the ratio of the number of nonconforming units 
    # in the sample D to the sample size n:
    # p = D/n. For a subgroup i containing n elements: p_i = D_i/n, i = 1,... m subgroups
    # From the binomial distribution, the mean should be estimated as p, and the variance s² 
    # as p(1-p)/n. The average of p is:
    # p_bar = (p_1 + ... + p_m)/m
    
    # UCL = p_bar + 3((p_bar(1-p_bar)/n)**(0.5))
    # Center line = p_bar
    # LCL = p_bar - 3((p_bar(1-p_bar)/n)**(0.5)), whereas n is the size of a given subgroup
    # Therefore, the width of control limits here vary with the size of the subgroups (it is
    # a function of the size of each subgroup)
    
    # np-chart - (Montgomery, pg.300, section 7.2.1):
    # UCL = np + 3((np(1-p))**(0.5))
    # Center line = np_bar
    # LCL = np - 3((np(1-p))**(0.5))
    # where np is the total sum of nonconforming, considering all subgroups, and p is the
    # propoertion for each individual subgroup, as previously defined.
    # Then, again the control limits depend on the subgroup (label) size.
    
    # C-chart - (Montgomery, pg.309, section 7.3.1):
    # In Poisson distribution, mean and variance are both equal to c.
    # UCL = c_bar + 3(c_bar**(0.5))
    # Center line = c_bar
    # LCL = c_bar + 3(c_bar**(0.5))
    # where c_bar is the mean c among all subgroups (labels)
    
    # U-chart - (Montgomery, pg.314, section 7.3.1):
    # If we find x total nonconformities in a sample of n inspection units, then the average 
    # number of nonconformities per inspection unit is: u = x/n
    # UCL = u_bar + 3((u_bar/n)**(0.5))
    # Center line = u_bar
    # LCL = u_bar + 3((u_bar/n)**(0.5))
    # where u_bar is the mean u among all subgroups (labels), but n is the individual subgroup size,
    # making the chart limit not-constant depending on the subgroups' sizes

    # RARE EVENTS
    # ATTENTION: Due not group data in this case. Since events are rare, they are likely to be 
    # eliminated during aggregation.
    # Usually, similarly to what is observed for the log-normal, data that follow the geometric
    # distribution is highly skewed, so the mean is not a good estimator for the central line.
    # Usually, it is better to apply the median = 0.693 x mean.
    
    # G-Chart: counts of total occurences between successive rare events occurences.
    # https://www.spcforexcel.com/knowledge/attribute-control-charts/g-control-chart
    # (Montgomery, pg.317, section 7.3.1)
    
    # e.g. total of patients between patient with ulcer.
    # The probability model that they use for the geometric distribution is p(x) = p(1-p)**(x-a)
    # where a is the known minimum possible number of events.
    # The number of units between defectives is modelled by the geometric distribution.
    # So, the G-control chart plots counting of occurrence by number, time unit, or timestamp.
    # Geometric distribution is highly skewed, thus the median is a better representation of the 
    # process centre to be used with the runs analysis.

    # Let y = total count of events between successive occurences of the rare event.
    
    # g_bar = median(y) = 0.693 x mean(y)
    # One can estimate the control limits as g_bar + 3(((g_bar)*(g_bar+1))**(0.5)) and
    # g_bar - 3(((g_bar)*(g_bar+1))**(0.5)), and central line = g_bar.
    # A better approach takes into account the probabilities associated to the geometric (g)
    # distribution.
    
    # In the probability approach, we start by calculating the value of p, which is an event
    # probability:
    # p = (1/(g_bar + 1))*((N-1)/N), where N is the total of individual values of counting between
    # successive rare events. So, if we have 3 rare events, A, B, and C, we have two values of 
    # counting between rare events, from A to B, and from B to C. In this case, since we have two
    # values that follow the G distribution, N = 2.
    # Let alpha_UCL a constant dependent on the number of sigma limits for the control limits. For
    # the usual 3 sigma limits, the value of alpha_UCL = 0.00135. With this constant, we can
    # calculate the control limits and central line
    
    # UCL = ((ln(0.00135))/(ln(1 - p))) - 1
    # Center line = ((ln(0.5))/(ln(1 - p))) - 1
    # LCL = max(0, (((ln(1 - 0.00135))/(ln(1 - p))) - 1))
    # where max represent the maximum among the two values in parentheses; and ln is the natural
    # logarithm (base e), sometimes referred as log (inverse of the exponential): ln(exp(x)) = x
    # = log(exp(x)) = x
    
    # t-chart: timedelta between rare events.
    # (Montgomery, pg.324, section 7.3.5):
    # https://www.qimacros.com/control-chart-formulas/t-chart-formula/
    
    # But instead of displaying the number of cases between events (defectives) it displays 
    # the time between events.
    
    # Nelson (1994) has suggested solving this problem by transforming the exponential random
    # variable to a Weibull random variable such that the resulting Weibull distribution is well
    # approximated by the normal distribution. 
    # If y represents the original exponential random variable (timedelta between successive rare
    # event occurence), the appropriate transformation is:
    # x = y**(1/3.6)= y**(0.2777)
    # where x is treated as a normal distributed variable, and the control chart for x is the I-MR.
    # So: 1. transform y into x;
    # 2. Calculate the control limits for x as if it was a chart for individuals;
    # reconvert the values to the original scale doing: y = x**(3.6)
    
    # We calculate for the transformed values x the moving ranges R and the mean of the 
    # mean values calculated for the 2-elements consecutive subgroups x_bar_bar, in the exact same way
    # we would do for any I-MR chart
    # UCL_transf =  x_bar_bar + 3*(1/d2)*R_bar
    # LCL_transf =  x_bar_bar - 3*(1/d2)*R_bar
    # Center line_transf = x_bar_bar
    
    # These values are in the transformed scale. So, we must reconvert them to the original scale for
    # obtaining the control limits and central line:
    # UCL = (UCL_transf)**(3.6)
    # LCL = (LCL_transf)**(3.6)
    # Center line = (Center line_transf)**(3.6)
    
    # Notice that this procedure naturally corrects the deviations caused by the skewness of the 
    # distribution. Actually, log and exponential transforms tend to reduce the skewness and to 
    # normalize the data.
    
    
    if (use_spc_chart_assistant == True):
        
        # Run if it is True. Requires TensorFlow to load. Load the extra library only
        # if necessary:
        # To show the Python class attributes, use the __dict__ method:
        # http://www.learningaboutelectronics.com/Articles/How-to-display-all-attributes-of-a-class-or-instance-of-a-class-in-Python.php#:~:text=So%20the%20__dict__%20method%20is%20a%20very%20useful,other%20data%20type%20such%20as%20a%20class%20itself.

        # instantiate the object
        assistant = spc_chart_assistant()
        # Download the images:
        assistant = assistant.download_assistant_imgs()

        # Run the assistant:
        while (assistant.keep_assistant_on == True):

            # Run the wrapped function until the user tells you to stop:
            # Notice that both variables are True for starting the first loop:
            assistant = assistant.open_chart_assistant_screen()

        # Delete the images
        assistant.delete_assistant_imgs()
        # Select the chart and the parameters:
        chart_to_use, column_with_labels_or_subgroups, consider_skewed_dist_when_estimating_with_std, column_with_variable_to_be_analyzed, timestamp_tag_column, column_with_event_frame_indication, rare_event_timedelta_unit, rare_event_indication = assistant.chart_selection()

    # Back to the main code, independently on the use of the assistant:    
    # set a local copy of the dataframe:
    DATASET = df.copy(deep = True)

    # Start a list unique_labels containing only element 0:
    unique_labels = [0]
    # If there is a column of labels or subgroups, this list will be updated. So we can control
    # the chart selection using the length of this list: if it is higher than 1, we have subgroups,
    # not individual values.

    if (timestamp_tag_column is None):

        # use the index itself:
        timestamp_tag_column = 'index'
        DATASET[timestamp_tag_column] = DATASET.index

    elif (DATASET[timestamp_tag_column].dtype not in numeric_dtypes):

        # The timestamp_tag_column was read as an object, indicating that it is probably a timestamp.
        # Try to convert it to datetime64:
        try:
            DATASET[timestamp_tag_column] = (DATASET[timestamp_tag_column]).astype(np.datetime64)
            print(f"Variable {timestamp_tag_column} successfully converted to datetime64[ns].\n")

        except:
            # Simply ignore it
            pass
        
        
    # Check if there are subgroups or if values are individuals. Also, check if there is no selected
    # chart:
    if ((column_with_labels_or_subgroups is None) & (chart_to_use is None)):
            
        print("There are only individual observations and no valid chart selected, so using standard error as natural variation (control limits).\n")
        # values are individual, so set it as so:
        chart_to_use = 'std_error'
        
    elif ((chart_to_use is None) | (chart_to_use not in ['std_error', '3s_as_natural_variation', 'i_mr', 'xbar_s', 'np', 'p', 'u', 'c', 'g', 't'])):
            
        print("No valid chart selected, so using standard error as natural variation (control limits).\n")
        # values are individual, so set it as so:
        chart_to_use = 'std_error'
        
    elif (chart_to_use in ['g', 't']):
            
        if (rare_event_timedelta_unit is None):
            print("No valid timedelta unit provided, so selecting \'day\' for analysis of rare events.\n")
            rare_event_timedelta_unit = 'day'
        elif (rare_event_timedelta_unit not in ['day', 'second', 'nanosecond', 'milisecond', 'hour', 'week', 'month', 'year']):
            print("No valid timedelta unit provided, so selecting \'day\' for analysis of rare events.\n")
            rare_event_timedelta_unit = 'day'
            
        if (rare_event_indication is None):
            print("No rare event indication provided, so changing the plot to \'std_error\'.\n")
            chart_to_use = 'std_error'
                
    # Now, a valid chart_to_use was selected and is saved as chart_to_use.
    # We can sort the dataframe according to the columns present:
          
    if ((column_with_labels_or_subgroups is not None) & (column_with_event_frame_indication is not None)):
        # There are time windows to consider and labels.
        # Update the list of unique labels:
        unique_labels = list(DATASET[column_with_labels_or_subgroups].unique())
        
        # sort DATASET by timestamp_tag_column, column_with_labels_or_subgroups, 
        # column_with_event_frame_indication, and column_with_variable_to_be_analyzed,
        # all in Ascending order:
        DATASET = DATASET.sort_values(by = [timestamp_tag_column, column_with_labels_or_subgroups, column_with_event_frame_indication, column_with_variable_to_be_analyzed], ascending = [True, True, True, True])
        
    elif (column_with_event_frame_indication is not None):
        # We already tested the simultaneous presence of both. So, to reach this condition, 
        # there is no column_with_labels_or_subgroups, but there is column_with_event_frame_indication
            
        # sort DATASET by timestamp_tag_column, column_with_event_frame_indication, 
        # and column_with_variable_to_be_analyzed, all in Ascending order:
        DATASET = DATASET.sort_values(by = [timestamp_tag_column, column_with_event_frame_indication, column_with_variable_to_be_analyzed], ascending = [True, True, True])
        
    elif (column_with_labels_or_subgroups is not None):
        # We already tested the simultaneous presence of both. So, to reach this condition, 
        # there is no column_with_event_frame_indication, but there is column_with_labels_or_subgroups
        # Update the list of unique labels:
        unique_labels = list(DATASET[column_with_labels_or_subgroups].unique())
        
        # sort DATASET by timestamp_tag_column, column_with_labels_or_subgroups, 
        # and column_with_variable_to_be_analyzed, all in Ascending order:
        DATASET = DATASET.sort_values(by = [timestamp_tag_column, column_with_labels_or_subgroups, column_with_variable_to_be_analyzed], ascending = [True, True, True])
        
    else:
        # There is neither column_with_labels_or_subgroups, nor column_with_event_frame_indication
        # sort DATASET by timestamp_tag_column, and column_with_variable_to_be_analyzed, 
        # both in Ascending order:
        DATASET = DATASET.sort_values(by = [timestamp_tag_column, column_with_variable_to_be_analyzed], ascending = [True, True])
        
    # Finally, reset indices:
    DATASET = DATASET.reset_index(drop = True)
    
    # Start a list of dictionaries to store the dataframes and subdataframes that will be analyzed:
    list_of_dictionaries_with_dfs = []
    # By now, dictionaries will contain a key 'event_frame' storing an integer identifier for the
    # event frame, starting from 0 (the index of the event in the list of unique event frames), or
    # zero, for the cases where there is a single event; a key 'df' storing the dataframe object, 
    # a key 'mean', storing the mean value of column_with_variable_to_be_analyzed; keys 'std' and
    # 'var', storing the standard deviation and the variance for column_with_variable_to_be_analyzed;
    # and column 'count', storing the counting of rows. 
    # After obtaining the control limits, they will also get a key 'list_of_points_out_of_cl'. 
    # This key will store a list of dictionaries (nested JSON), where each dictionary 
    # will have two keys, 'x', and 'y', with the coordinates of the point outside control 
    # limits as the values.
    
    if ((column_with_event_frame_indication is not None) & (chart_to_use not in ['g', 't'])):
        
        # Get a series of unique values of the event frame indications, and save it as a list 
        # using the list attribute:
        unique_event_indication = list(DATASET[column_with_event_frame_indication].unique())
        
        print(f"{len(unique_event_indication)} different values of event frame indication detected: {unique_event_indication}.\n")
        
        if (len(unique_event_indication) > 0):
            
            # There are more than one event frame to consider. Let's loop through the list of
            # event frame indication, where each element is referred as 'event_frame':
            for event_frame in unique_event_indication:
                
                # Define a boolean filter to select only the rows correspondent to event_frame:
                boolean_filter = (DATASET[column_with_event_frame_indication] == event_frame)
                
                # Create a copy of the dataset, with entries selected by that filter:
                ds_copy = (DATASET[boolean_filter]).copy(deep = True)
                # Sort again by timestamp_tag_column and column_with_variable_to_be_analyzed, 
                # to guarantee the results are in order:
                ds_copy = ds_copy.sort_values(by = [timestamp_tag_column, column_with_variable_to_be_analyzed], ascending = [True, True])
                # Restart the index of the copy:
                ds_copy = ds_copy.reset_index(drop = True)
                
                # Store ds_copy in a dictionary of values. Put the index of event_frame
                # in the list unique_event_indication as the value in key 'event_frame', to numerically
                # identify the dictionary according to the event frame.
                # Also, store the 'mean', 'sum,' 'std', 'var', and 'count' aggregates for the column
                # column_with_variable_to_be_analyzed from ds_copy:
                
                dict_of_values = {
                                    'event_frame': unique_event_indication.index(event_frame),
                                    'df': ds_copy, 
                                    'center': ds_copy[column_with_variable_to_be_analyzed].mean(),
                                    'sum': ds_copy[column_with_variable_to_be_analyzed].sum(),
                                    'std': ds_copy[column_with_variable_to_be_analyzed].std(),
                                    'var': ds_copy[column_with_variable_to_be_analyzed].var(),
                                    'count': ds_copy[column_with_variable_to_be_analyzed].count()
                                 }
                
                # append dict_of_values to the list:
                list_of_dictionaries_with_dfs.append(dict_of_values)
                
                # Now, the loop will pick the next event frame.
        
        else:
            
            # There is actually a single time window. The only dataframe 
            # stored in the dictionary is DATASET itself, which is stored with 
            # the aggregate statistics calculated for the whole dataframe. Since
            # there is a single dataframe, the single value in 'event_frame' is 0.
            dict_of_values = {
                                'event_frame': 0,
                                'df': DATASET, 
                                'center': DATASET[column_with_variable_to_be_analyzed].mean(),
                                'sum': DATASET[column_with_variable_to_be_analyzed].sum(),
                                'std': DATASET[column_with_variable_to_be_analyzed].std(),
                                'var': DATASET[column_with_variable_to_be_analyzed].var(),
                                'count': DATASET[column_with_variable_to_be_analyzed].count()
                            }
            
            # append dict_of_values to the list:
            list_of_dictionaries_with_dfs.append(dict_of_values)
    
    else:
        # The other case where there is a single time window or 'g' or 't' charts. 
        # So, the only dataframe stored in the dictionary is DATASET itself, which is stored with 
        # the aggregate statistics calculated for the whole dataframe. Since
        # there is a single dataframe, the single value in 'event_frame' is 0.
        dict_of_values = {
                            'event_frame': 0,
                            'df': DATASET, 
                            'center': DATASET[column_with_variable_to_be_analyzed].mean(),
                            'sum': DATASET[column_with_variable_to_be_analyzed].sum(),
                            'std': DATASET[column_with_variable_to_be_analyzed].std(),
                            'var': DATASET[column_with_variable_to_be_analyzed].var(),
                            'count': DATASET[column_with_variable_to_be_analyzed].count()
                        }
            
        # append dict_of_values to the list:
        list_of_dictionaries_with_dfs.append(dict_of_values)
    
    # Now, data is sorted, timestamps were converted to datetime objects, and values collected
    # for different timestamps were separated into dictionaries (elements) from the list
    # list_of_dictionaries_with_dfs. Each dictionary contains a key 'df' used to access the
    # dataframe, as well as keys storing the aggregate statistics: 'mean', 'std', 'var', and
    # 'count'.
    
    # Now, we can process the different control limits calculations.
    
    # Start a support list:
    support_list = []
    
    ## INDIVIDUAL VALUES
    # Use the unique_labels list to guarantee that there have to be more than 1 subgroup
    # to not treat data as individual values. If there is a single subgroup, unique_labels
    # will have a single element.
    if ((column_with_labels_or_subgroups is None) | (len(unique_labels) <= 1)):
        
        if (chart_to_use == 'i_mr'):
            
            print("WARNING: the I-MR control limits are based on the strong hypothesis that data follows a normal distribution. If it is not the case, do not use this chart.")
            print("If you are not confident about the statistical distribution, select chart_to_use = \'3s_as_natural_variation\' to use 3 times the standard deviation as estimator for the natural variation (the control limits); or chart_to_use = \'std_error\' to use 3 times the standard error as control limits.\n")
            
            for dictionary in list_of_dictionaries_with_dfs:
                
                # Create an instance (object) from spc_plot class:
                plot = spc_plot (dictionary = dictionary, column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed, timestamp_tag_column = timestamp_tag_column, chart_to_use = chart_to_use, column_with_labels_or_subgroups = column_with_labels_or_subgroups, consider_skewed_dist_when_estimating_with_std = consider_skewed_dist_when_estimating_with_std, rare_event_indication = rare_event_indication, rare_event_timedelta_unit = rare_event_timedelta_unit)
                # Apply the method
                plot = plot.chart_i_mr()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
            
            # Now that we finished looping through dictionaries, make list_of_dictionaries_with_dfs
            # the support_list itself:
            list_of_dictionaries_with_dfs = support_list
        
        elif (chart_to_use in ['g', 't']):
            
            print(f"Analyzing occurence of rare event {rare_event_indication} through chart {chart_to_use}.\n")
            
            for dictionary in list_of_dictionaries_with_dfs:
                
                # Create an instance (object) from spc_plot class:
                plot = spc_plot (dictionary = dictionary, column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed, timestamp_tag_column = timestamp_tag_column, chart_to_use = chart_to_use, column_with_labels_or_subgroups = column_with_labels_or_subgroups, consider_skewed_dist_when_estimating_with_std = consider_skewed_dist_when_estimating_with_std, rare_event_indication = rare_event_indication, rare_event_timedelta_unit = rare_event_timedelta_unit)
                # Apply the method
                plot = plot.rare_events_chart()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
            
            # Now that we finished looping through dictionaries, make list_of_dictionaries_with_dfs
            # the support_list itself:
            list_of_dictionaries_with_dfs = support_list
            
        elif (chart_to_use == '3s_as_natural_variation'):
            
            print("Using 3s (3 times the standard deviation) as estimator of the natural variation (control limits).\n")
            
            for dictionary in list_of_dictionaries_with_dfs:
                
                # Create an instance (object) from spc_plot class:
                plot = spc_plot (dictionary = dictionary, column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed, timestamp_tag_column = timestamp_tag_column, chart_to_use = chart_to_use, column_with_labels_or_subgroups = column_with_labels_or_subgroups, consider_skewed_dist_when_estimating_with_std = consider_skewed_dist_when_estimating_with_std, rare_event_indication = rare_event_indication, rare_event_timedelta_unit = rare_event_timedelta_unit)
                # Apply the method
                plot = plot.chart_3s()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
            
            # Now that we finished looping through dictionaries, make list_of_dictionaries_with_dfs
            # the support_list itself:
            list_of_dictionaries_with_dfs = support_list
        
        else:
            
            print("Using 3 times the standard error as estimator of the natural variation (control limits).\n")
            
            for dictionary in list_of_dictionaries_with_dfs:
                
                # Create an instance (object) from spc_plot class:
                plot = spc_plot (dictionary = dictionary, column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed, timestamp_tag_column = timestamp_tag_column, chart_to_use = chart_to_use, column_with_labels_or_subgroups = column_with_labels_or_subgroups, consider_skewed_dist_when_estimating_with_std = consider_skewed_dist_when_estimating_with_std, rare_event_indication = rare_event_indication, rare_event_timedelta_unit = rare_event_timedelta_unit)
                # Apply the method
                plot = plot.chart_std_error()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
            
            # Now that we finished looping through dictionaries, make list_of_dictionaries_with_dfs
            # the support_list itself:
            list_of_dictionaries_with_dfs = support_list
    
    ## DATA IN SUBGROUPS
    else:
        
        # Loop through each dataframe:
        for dictionary in list_of_dictionaries_with_dfs:
            
            # Create an instance (object) from spc_plot class:
            plot = spc_plot (dictionary = dictionary, column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed, timestamp_tag_column = timestamp_tag_column, chart_to_use = chart_to_use, column_with_labels_or_subgroups = column_with_labels_or_subgroups, consider_skewed_dist_when_estimating_with_std = consider_skewed_dist_when_estimating_with_std, rare_event_indication = rare_event_indication, rare_event_timedelta_unit = rare_event_timedelta_unit)
            # Apply the method
            plot = plot.create_grouped_df()
                
            # Now, dataframe is ready for the calculation of control limits.
            # Let's select the appropriate chart:
        
            if (chart_to_use == '3s_as_natural_variation'):

                plot = plot.chart_3s()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
            
            elif (chart_to_use == 'std_error'):

                plot = plot.chart_std_error()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
                
            elif (chart_to_use == 'xbar_s'):
                
                plot = plot.chart_x_bar_s()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
            
            elif (chart_to_use == 'p'):
                
                plot = plot.chart_p()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)

            elif (chart_to_use == 'np'):
            
                plot = plot.chart_np()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)

            elif (chart_to_use == 'c'):
                
                plot = plot.chart_c()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)
                
            elif (chart_to_use == 'u'):
                
                plot = plot.chart_u()
                # Append the updated dictionary to the support_list (access the dictionary attribute):
                support_list.append(plot.dictionary)

            else:

                print(f"Select a valid control chart: {['3s_as_natural_variation', 'std_error', 'i_mr', 'xbar_s', 'np', 'p', 'u', 'c', 'g', 't']}.\n")
                return "error", "error" # Two, since two dataframes are returned
            
            # Go to the next element (dictionary) from the list list_of_dictionaries_with_dfs
        
        # Now that we finished looping through dictionaries, make list_of_dictionaries_with_dfs
        # the support_list itself:
        list_of_dictionaries_with_dfs = support_list
        
        # Now we finished looping, we can print the warnings
        if (chart_to_use == '3s_as_natural_variation'):
    
            print("Using 3s (3 times the standard deviation) as estimator of the natural variation (control limits). Remember that we are taking the standard deviation from the subgroup (label) aggregates.\n")
        
        elif (chart_to_use == 'xbar_s'):
            
            print("WARNING: the X-bar-S control limits are based on the strong hypothesis that the mean values from the subgroups follow a normal distribution. If it is not the case, do not use this chart.")
            print("If you are not confident about the statistical distribution, select chart_to_use = \'3s_as_natural_variation\' to use 3 times the standard deviation as estimator for the natural variation (the control limits).\n")
            print("Use this chart for analyzing mean values from multiple data collected together in groups (or specific labels), usually in close moments.\n")
        
        elif (chart_to_use == 'np'):
            print("WARNING: the U control limits are based on the strong hypothesis that the counting of values from the subgroups follow a Poisson distribution (Poisson is a case from the Gamma distribution). If it is not the case, do not use this chart.")
            
        
        elif (chart_to_use == 'p'):
            print("WARNING: the U control limits are based on the strong hypothesis that the counting of values from the subgroups follow a Poisson distribution (Poisson is a case from the Gamma distribution). If it is not the case, do not use this chart.")
            
        
        elif (chart_to_use == 'u'):
            
            print("WARNING: the U control limits are based on the strong hypothesis that the counting of values from the subgroups follow a Poisson distribution (Poisson is a case from the Gamma distribution). If it is not the case, do not use this chart.")
            print("If you are not confident about the statistical distribution, select chart_to_use = \'3s_as_natural_variation\' to use 3 times the standard deviation as estimator for the natural variation (the control limits).\n")
        
        else:
            # chart_to_use == 'c'
            print("WARNING: the C control limits are based on the strong hypothesis that the counting of values from the subgroups follow a Poisson distribution (Poisson is a case from the Gamma distribution). If it is not the case, do not use this chart.")
            print("If you are not confident about the statistical distribution, select chart_to_use = \'3s_as_natural_variation\' to use 3 times the standard deviation as estimator for the natural variation (the control limits).\n")
        
        
    # Now, we have all the control limits calculated, and data aggregated when it is the case.
    # Let's merge (append - SQL UNION) all the dataframes stored in the dictionaries (elements)
    # from the list list_of_dictionaries_with_dfs. Pick the first dictionary, i.e., element of
    # index 0 from the list:
    
    dictionary = list_of_dictionaries_with_dfs[0]
    # access the dataframe:
    df = dictionary['df']
    
    # Also, start a list for storing the timestamps where the event frames start:
    time_of_event_frame_start = []
    # Let's pick the last element from df[timestamp_tag_column]: it is the time
    # when the time window (event frame) is changing. We can convert it to a list
    # and slice the list from its last element (index -1; the immediately before is -2,
    # etc).
    timestamp_tag_value = list(df[timestamp_tag_column])[-1:]
    # Now concatenate this 1-element list with time_of_event_frame_start:
    time_of_event_frame_start = time_of_event_frame_start + timestamp_tag_value
    # When concatenating lists, the elements from the right list are sequentially added to the
    # first one: list1 = ['a', 'b'], list2 = ['c', 'd'], list1 + list2 = ['a', 'b', 'c', 'd'] 
    
    # Now, let's loop through each one of the other dictionaries from the list:
    
    for i in range (1, len(list_of_dictionaries_with_dfs)):
        
        # start from i = 1, index of the second element (we already picked the first one),
        # and goes to the index of the last dictionary, len(list_of_dictionaries_with_dfs) - 1:
        
        # access element (dictionary) i from the list:
        dictionary_i = list_of_dictionaries_with_dfs[i]
        # access the dataframe i:
        df_i = dictionary_i['df']
        
        # Again, pick the last timestamp from this window:
        timestamp_tag_value = list(df[timestamp_tag_column])[-1:]
        # Now concatenate this 1-element list with time_of_event_frame_start:
        time_of_event_frame_start = time_of_event_frame_start + timestamp_tag_value
        
        # Append df_i to df (SQL UNION - concatenate or append rows):
        # Save in df itself:
        df = pd.concat([df, df_i], axis = 0, ignore_index = True, sort = True, join = 'inner')
        # axis = 0 to append rows; 'inner' join removes missing values
    
    # Now, reset the index of the final concatenated dataframe, containing all the event frames
    # separately processed, and containing the control limits and mean values for each event:
    df = df.reset_index(drop = True)
    
    # Now, add a column to inform if each point is in or out of the control limits.
    # apply a filter to select each situation:
    # (syntax: dataset.loc[dataset['column_filtered'] <= 0.87, 'labelled_column'] = 1)
    # Start the new column as 'in_control_lim' (default case):
    df['control_limits_check'] = 'in_control_limits'
    
    # Get the control limits:
    lower_cl = df['lower_cl']
    upper_cl = df['upper_cl']
    
    # Now modify only points which are out of the control ranges:
    df.loc[(df[column_with_variable_to_be_analyzed] < lower_cl), 'control_limits_check'] = 'below_lower_control_limit'
    df.loc[(df[column_with_variable_to_be_analyzed] > upper_cl), 'control_limits_check'] = 'above_upper_control_limit'
                
    # Let's also create the 'red_df' containing only the values outside of the control limits.
    # This dataframe will be used to highlight the values outside the control limits
    
    # copy the dataframe:
    red_df = df.copy(deep = True)
    # Filter red_df to contain only the values outside the control limits:
    boolean_filter = ((red_df[column_with_variable_to_be_analyzed] < lower_cl) | (red_df[column_with_variable_to_be_analyzed] > upper_cl))
    red_df = red_df[boolean_filter]
    # Reset the index of this dataframe:
    red_df = red_df.reset_index(drop = True)
    
    if (len(red_df) > 0):
        
        # There is at least one row outside the control limits.
        print("Attention! Point outside of natural variation (control limits).")
        print("Check the red_df dataframe returned for details on values outside the control limits.")
        print("They occur at the following time values:\n")
        
        try:
            # only works in Jupyter Notebook:
            from IPython.display import display
            display(red_df)

        except: # regular mode
            print(list(red_df[timestamp_tag_column]))
    
        print("\n")
    
    # specification_limits = {'lower_spec_lim': value1, 'upper_spec_lim': value2}
    
    # Check if there is a lower specification limit:
    if (specification_limits['lower_spec_lim'] is not None):
        
        lower_spec_lim = specification_limits['lower_spec_lim']
        
        # Now, add a column to inform if each point is in or out of the specification limits.
        # apply a filter to select each situation:

        # Start the new column as 'in_spec_lim' (default case):
        df['spec_limits_check'] = 'in_spec_lim'
        # Now modify only points which are below the specification limit:
        df.loc[(df[column_with_variable_to_be_analyzed] < lower_spec_lim), 'spec_limits_check'] = 'below_lower_spec_lim'
    
        if (len(df[(df[column_with_variable_to_be_analyzed] < lower_spec_lim)]) > 0):
            
            print("Attention! Point below lower specification limit.")
            print("Check the returned dataframe df to obtain more details.\n")
        
        # Check if there is an upper specification limit too:
        if (specification_limits['upper_spec_lim'] is not None):

            upper_spec_lim = specification_limits['upper_spec_lim']

            # Now modify only points which are above the specification limit:
            df.loc[(df[column_with_variable_to_be_analyzed] > upper_spec_lim), 'spec_limits_check'] = 'above_upper_spec_lim'

            if (len(df[(df[column_with_variable_to_be_analyzed] > upper_spec_lim)]) > 0):

                print("Attention! Point above upper specification limit.")
                print("Check the returned dataframe df to obtain more details.\n")
    
    # Check the case where there is no lower, but there is a specification limit:
    elif (specification_limits['upper_spec_lim'] is not None):
        
        upper_spec_lim = specification_limits['upper_spec_lim']
        
        # Start the new column as 'in_spec_lim' (default case):
        df['spec_limits_check'] = 'in_spec_lim'
        
        # Now modify only points which are above the specification limit:
        df.loc[(df[column_with_variable_to_be_analyzed] > upper_spec_lim), 'spec_limits_check'] = 'above_upper_spec_lim'

        if (len(df[(df[column_with_variable_to_be_analyzed] > upper_spec_lim)]) > 0):

            print("Attention! Point above upper specification limit.")
            print("Check the returned dataframe df to obtain more details.\n")
    
    
    # Now, we have the final dataframe containing analysis regarding the control and specification
    # limits, and the red_df, that will be used to plot red points for values outside the control
    # range. Then, we can plot the graph.
    
    # Now, we can plot the figure.
    # we set alpha = 0.95 (opacity) to give a degree of transparency (5%), 
    # so that one series do not completely block the visualization of the other.
        
    # Matplotlib linestyle:
    # https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html?msclkid=68737f24d16011eca9e9c4b41313f1ad
        
    if (plot_title is None):
        
        if (chart_to_use == 'g'):
            plot_title = f"{chart_to_use}_for_count_of_events_between_rare_occurence"
        
        elif (chart_to_use == 't'):
            plot_title = f"{chart_to_use}_for_timedelta_between_rare_occurence"
        
        else:
            plot_title = f"{chart_to_use}_for_{column_with_variable_to_be_analyzed}"
    
    if ((column_with_labels_or_subgroups is None) | (len(unique_labels) <= 1)):
        
        # 'i_mr' or, 'std_error', or '3s_as_natural_variation' (individual measurements)
        
        LABEL = column_with_variable_to_be_analyzed
        
        if (chart_to_use == 'i_mr'):
            LABEL_MEAN = 'mean'
            
        elif ((chart_to_use == '3s_as_natural_variation')|(chart_to_use == 'std_error')):
            if(consider_skewed_dist_when_estimating_with_std):
                LABEL_MEAN = 'median'

            else:
                LABEL_MEAN = 'mean'
        
        else:
            
            if (chart_to_use == 'g'):
                LABEL = 'count_between\nrare_events'
                LABEL_MEAN = 'median'
            
            elif (chart_to_use == 't'):
                LABEL = 'timedelta_between\nrare_events'
                LABEL_MEAN = 'central_line'
                
    else:
        if ((chart_to_use == '3s_as_natural_variation')|(chart_to_use == 'std_error')):
            
            LABEL = "mean_value\nby_label"
            
            if(consider_skewed_dist_when_estimating_with_std):
                LABEL_MEAN = 'median'
            else:
                LABEL_MEAN = 'mean'
        
        elif (chart_to_use == 'xbar_s'):
            
            LABEL = "mean_value\nby_label"
            LABEL_MEAN = 'mean'
            
        elif (chart_to_use == 'np'):
            
            LABEL = "total_occurences\nby_label"
            LABEL_MEAN = 'sum_of_ocurrences'
            
        elif (chart_to_use == 'p'):
            
            LABEL = "mean_value\ngrouped_by_label"
            LABEL_MEAN = 'mean'
        
        elif (chart_to_use == 'u'):
            
            LABEL = "mean_value\ngrouped_by_label"
            LABEL_MEAN = 'mean'
            
        else:
            # chart_to_use == 'c'
            LABEL = "total_occurences\nby_label"
            LABEL_MEAN = 'average_sum\nof_ocurrences'
    
    x = df[timestamp_tag_column]
    
    y = df[column_with_variable_to_be_analyzed]  
    upper_control_lim = df['upper_cl']
    lower_control_lim = df['lower_cl']
    mean_line = df['center']
    
    if (specification_limits['lower_spec_lim'] is not None):
        
        lower_spec_lim = specification_limits['lower_spec_lim']
    
    else:
        lower_spec_lim = None
    
    if (specification_limits['upper_spec_lim'] is not None):
        
        upper_spec_lim = specification_limits['upper_spec_lim']
    
    else:
        upper_spec_lim = None
    
    if (len(red_df) > 0):
        
        red_x = red_df[timestamp_tag_column]
        red_y = red_df[column_with_variable_to_be_analyzed]
        
    # Let's put a small degree of transparency (1 - OPACITY) = 0.05 = 5%
    # so that the bars do not completely block other views.
    OPACITY = 0.95
        
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    fig = plt.figure(figsize = (12, 8))
    ax = fig.add_subplot()
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 0 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    # Set graphic title
    ax.set_title(plot_title) 

    if not (horizontal_axis_title is None):
        # Set horizontal axis title
        ax.set_xlabel(horizontal_axis_title)

    if not (vertical_axis_title is None):
        # Set vertical axis title
        ax.set_ylabel(vertical_axis_title)

    # Scatter plot of time series:
    ax.plot(x, y, linestyle = "-", marker = '', color = 'darkblue', alpha = OPACITY, label = LABEL)
    # Axes.plot documentation:
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.plot.html?msclkid=42bc92c1d13511eca8634a2c93ab89b5
            
    # x and y are positional arguments: they are specified by their position in function
    # call, not by an argument name like 'marker'.
            
    # Matplotlib markers:
    # https://matplotlib.org/stable/api/markers_api.html?msclkid=36c5eec5d16011ec9583a5777dc39d1f
    
    # Plot the mean line as a step function (values connected by straight splines, forming steps):
    ax.step(x, y = mean_line, color = 'fuchsia', linestyle = 'dashed', label = LABEL_MEAN, alpha = OPACITY)
    
    # Plot the control limits as step functions too:
    ax.step(x, y = upper_control_lim, color = 'crimson', linestyle = 'dashed', alpha = OPACITY, label = 'control\nlimit')
    ax.step(x, y = lower_control_lim, color = 'crimson', linestyle = 'dashed', alpha = OPACITY)
    
    # If there are specifications or reference values, plot as horizontal constant lines (axhlines):
    if (lower_spec_lim is not None):
        
        ax.axhline(lower_spec_lim, color = 'black', linestyle = 'dashed', label = 'specification\nlimit', alpha = OPACITY)
    
    if (upper_spec_lim is not None):
        
        ax.axhline(upper_spec_lim, color = 'black', linestyle = 'dashed', label = 'specification\nlimit', alpha = OPACITY)
    
    if (reference_value is not None):
        
        ax.axhline(reference_value, color = 'darkgreen', linestyle = 'dashed', label = 'reference\nvalue', alpha = OPACITY)   
    
    # If there are red points outside of control limits to highlight, plot them above the graph
    # (plot as scatter plot, with no spline, and 100% opacity = 1.0):
    if (len(red_df) > 0):
        
        ax.plot(red_x, red_y, linestyle = '', marker = 'o', color = 'firebrick', alpha = 1.0)
    
    # If the length of list time_of_event_frame_start is higher than zero,
    # loop through each element on the list and add a vertical constant line for the timestamp
    # correspondent to the beginning of an event frame:
    
    if (len(time_of_event_frame_start) > 0):
        
        for timestamp in time_of_event_frame_start:
            # add timestamp as a vertical line (axvline):
            ax.axvline(timestamp, color = 'aqua', linestyle = 'dashed', label = 'event_frame\nchange', alpha = OPACITY)
            
    # Now we finished plotting all of the series, we can set the general configuration:
    ax.grid(grid) # show grid or not
    ax.legend(loc = "lower left")

    if (export_png == True):
        # Image will be exported
        import os

        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = ""

        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = f"control_chart_{chart_to_use}"

        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 330 dpi
            png_resolution_dpi = 330

        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)

        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")

    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    #plt.figure(figsize = (12, 8))
    #fig.tight_layout()

    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    return df, red_df

# **Evaluating the Process Capability (in relation to specifications)**

In [ ]:
class capability_analysis:
            
    # Initialize instance attributes.
    # define the Class constructor, i.e., how are its objects:
    def __init__ (self, df, column_with_variable_to_be_analyzed, specification_limits, total_of_bins = 10, alpha = 0.10):
                
        import numpy as np
        import pandas as pd
        
        # If the user passes the argument, use them. Otherwise, use the standard values.
        # Set the class objects' attributes.
        # Suppose the object is named plot. We can access the attribute as:
        # plot.dictionary, for instance.
        # So, we can save the variables as objects' attributes.
        self.df = df
        self.column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed
        self.specification_limits = specification_limits
        self.sample_size = df[column_with_variable_to_be_analyzed].count()
        self.mu = (df[column_with_variable_to_be_analyzed]).mean() 
        self.median = (df[column_with_variable_to_be_analyzed]).median()
        self.sigma = (df[column_with_variable_to_be_analyzed]).std()
        self.lowest = (df[column_with_variable_to_be_analyzed]).min()
        self.highest = (df[column_with_variable_to_be_analyzed]).max()
        self.total_of_bins = total_of_bins
        self.alpha = alpha
        
        # Start a dictionary of constants
        self.dict_of_constants = {}
        # Get parameters to update later:
        self.histogram_dict = {}
        self.capability_dict = {}
        self.normality_dict = {}
        
        print("WARNING: this capability analysis is based on the strong hypothesis that data follows the normal (Gaussian) distribution.\n")
        
    # Define the class methods.
    # All methods must take an object from the class (self) as one of the parameters
   
    # Define a dictionary of constants.
    # Each key in the dictionary corresponds to a number of samples in a subgroup.
    # sample_size - This variable represents the total of labels or subgroups n. 
    # If there are multiple labels, this variable will be updated later.
    
    def check_data_normality (self):
        
        import numpy as np
        import pandas as pd
        from scipy import stats
        from statsmodels.stats import diagnostic
        
        alpha = self.alpha
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        sample_size = self.sample_size
        mu = self.mu 
        median = self.median
        sigma = self.sigma
        lowest = self.lowest
        highest = self.highest
        normality_dict = self.normality_dict # empty dictionary 
        
        print("WARNING: The statistical tests require at least 20 samples.\n")
        print("Interpretation:")
        print("p-value: probability that data is described by the normal distribution.")
        print("Criterion: the series is not described by normal if p < alpha = %.3f." %(alpha))
        
        if (sample_size < 20):
            
            print(f"Unable to test series normality: at least 20 samples are needed, but found only {sample_size} entries for this series.\n")
            normality_dict['WARNING'] = "Series without the minimum number of elements (20) required to test the normality."
            
        else:
            # Let's test the series.
            y = df[column_with_variable_to_be_analyzed]
            
            # Scipy.stats’ normality test
            # It is based on D’Agostino and Pearson’s test that combines 
            # skew and kurtosis to produce an omnibus test of normality.
            _, scipystats_test_pval = stats.normaltest(y)
            # The underscore indicates an output to be ignored, which is s^2 + k^2, 
            # where s is the z-score returned by skewtest and k is the z-score returned by kurtosistest.
            # https://docs.scipy.org/doc/scipy-1.8.0/html-scipyorg/reference/generated/scipy.stats.normaltest.html
            
            print("\n")
            print("D\'Agostino and Pearson\'s normality test (scipy.stats normality test):")
            print(f"p-value = {scipystats_test_pval:e} = {scipystats_test_pval*100:.2f}% of probability of being normal.")
            # :e indicates the scientific notation; .2f: float with 2 decimal cases
            
            if (scipystats_test_pval < alpha):
                
                print("p = %.3f < %.3f" %(scipystats_test_pval, alpha))
                print(f"According to this test, data is not described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            else:
                
                print("p = %.3f >= %.3f" %(scipystats_test_pval, alpha))
                print(f"According to this test, data is described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            # add this test result to the dictionary:
            normality_dict['dagostino_pearson_p_val'] = scipystats_test_pval
            normality_dict['dagostino_pearson_p_in_pct'] = scipystats_test_pval*100
            
            # Scipy.stats’ Shapiro-Wilk test
            # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html
            shapiro_test = stats.shapiro(y)
            # returns ShapiroResult(statistic=0.9813305735588074, pvalue=0.16855233907699585)
             
            print("\n")
            print("Shapiro-Wilk normality test:")
            print(f"p-value = {shapiro_test[1]:e} = {(shapiro_test[1])*100:.2f}% of probability of being normal.")
            
            if (shapiro_test[1] < alpha):
                
                print("p = %.3f < %.3f" %(shapiro_test[1], alpha))
                print(f"According to this test, data is not described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            else:
                
                print("p = %.3f >= %.3f" %(shapiro_test[1], alpha))
                print(f"According to this test, data is described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            # add this test result to the dictionary:
            normality_dict['shapiro_wilk_p_val'] = shapiro_test[1]
            normality_dict['shapiro_wilk_p_in_pct'] = (shapiro_test[1])*100
            
            # Lilliefors’ normality test
            lilliefors_test = diagnostic.kstest_normal(y, dist = 'norm', pvalmethod = 'table')
            # Returns a tuple: index 0: ksstat: float
            # Kolmogorov-Smirnov test statistic with estimated mean and variance.
            # index 1: p-value:float
            # If the pvalue is lower than some threshold, e.g. 0.10, then we can reject the Null hypothesis that the sample comes from a normal distribution.
            
            print("\n")
            print("Lilliefors\'s normality test:")
            print(f"p-value = {lilliefors_test[1]:e} = {(lilliefors_test[1])*100:.2f}% of probability of being normal.")
            
            if (lilliefors_test[1] < alpha):
                
                print("p = %.3f < %.3f" %(lilliefors_test[1], alpha))
                print(f"According to this test, data is not described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            else:
                
                print("p = %.3f >= %.3f" %(lilliefors_test[1], alpha))
                print(f"According to this test, data is described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            # add this test result to the dictionary:
            normality_dict['lilliefors_p_val'] = lilliefors_test[1]
            normality_dict['lilliefors_p_in_pct'] = (lilliefors_test[1])*100

            # Anderson-Darling normality test
            ad_test = diagnostic.normal_ad(y, axis = 0)
            # Returns a tuple: index 0 - ad2: float
            # Anderson Darling test statistic.
            # index 1 - p-val: float
            # The p-value for hypothesis that the data comes from a normal distribution with unknown mean and variance.
            
            print("\n")
            print("Anderson-Darling (AD) normality test:")
            print(f"p-value = {ad_test[1]:e} = {(ad_test[1])*100:.2f}% of probability of being normal.")
            
            if (ad_test[1] < alpha):
                
                print("p = %.3f < %.3f" %(ad_test[1], alpha))
                print(f"According to this test, data is not described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            else:
                
                print("p = %.3f >= %.3f" %(ad_test[1], alpha))
                print(f"According to this test, data is described by the normal distribution, for the {alpha*100}% confidence level defined.")
            
            # add this test result to the dictionary:
            normality_dict['anderson_darling_p_val'] = ad_test[1]
            normality_dict['anderson_darling_p_in_pct'] = (ad_test[1])*100
            
            # Update the attribute:
            self.normality_dict = normality_dict
            
            return self
    
    def get_constants (self):
        
        if (self.sample_size < 2):
            
            self.sample_size = 2
            
        if (self.sample_size <= 25):
            
            dict_of_constants = {
                
                2: {'A':2.121, 'A2':1.880, 'A3':2.659, 'c4':0.7979, '1/c4':1.2533, 'B3':0, 'B4':3.267, 'B5':0, 'B6':2.606, 'd2':1.128, '1/d2':0.8865, 'd3':0.853, 'D1':0, 'D2':3.686, 'D3':0, 'D4':3.267},
                3: {'A':1.732, 'A2':1.023, 'A3':1.954, 'c4':0.8862, '1/c4':1.1284, 'B3':0, 'B4':2.568, 'B5':0, 'B6':2.276, 'd2':1.693, '1/d2':0.5907, 'd3':0.888, 'D1':0, 'D2':4.358, 'D3':0, 'D4':2.574},
                4: {'A':1.500, 'A2':0.729, 'A3':1.628, 'c4':0.9213, '1/c4':1.0854, 'B3':0, 'B4':2.266, 'B5':0, 'B6':2.088, 'd2':2.059, '1/d2':0.4857, 'd3':0.880, 'D1':0, 'D2':4.698, 'D3':0, 'D4':2.282},
                5: {'A':1.342, 'A2':0.577, 'A3':1.427, 'c4':0.9400, '1/c4':1.0638, 'B3':0, 'B4':2.089, 'B5':0, 'B6':1.964, 'd2':2.326, '1/d2':0.4299, 'd3':0.864, 'D1':0, 'D2':4.918, 'D3':0, 'D4':2.114},
                6: {'A':1.225, 'A2':0.483, 'A3':1.287, 'c4':0.9515, '1/c4':1.0510, 'B3':0.030, 'B4':1.970, 'B5':0.029, 'B6':1.874, 'd2':2.534, '1/d2':0.3946, 'd3':0.848, 'D1':0, 'D2':5.078, 'D3':0, 'D4':2.004},
                7: {'A':1.134, 'A2':0.419, 'A3':1.182, 'c4':0.9594, '1/c4':1.0423, 'B3':0.118, 'B4':1.882, 'B5':0.113, 'B6':1.806, 'd2':2.704, '1/d2':0.3698, 'd3':0.833, 'D1':0.204, 'D2':5.204, 'D3':0.076, 'D4':1.924},
                8: {'A':1.061, 'A2':0.373, 'A3':1.099, 'c4':0.9650, '1/c4':1.0363, 'B3':0.185, 'B4':1.815, 'B5':0.179, 'B6':1.751, 'd2':2.847, '1/d2':0.3512, 'd3':0.820, 'D1':0.388, 'D2':5.306, 'D3':0.136, 'D4':1.864},
                9: {'A':1.000, 'A2':0.337, 'A3':1.032, 'c4':0.9693, '1/c4':1.0317, 'B3':0.239, 'B4':1.761, 'B5':0.232, 'B6':1.707, 'd2':2.970, '1/d2':0.3367, 'd3':0.808, 'D1':0.547, 'D2':5.393, 'D3':0.184, 'D4':1.816},
                10: {'A':0.949, 'A2':0.308, 'A3':0.975, 'c4':0.9727, '1/c4':1.0281, 'B3':0.284, 'B4':1.716, 'B5':0.276, 'B6':1.669, 'd2':3.078, '1/d2':0.3249, 'd3':0.797, 'D1':0.687, 'D2':5.469, 'D3':0.223, 'D4':1.777},
                11: {'A':0.905, 'A2':0.285, 'A3':0.927, 'c4':0.9754, '1/c4':1.0252, 'B3':0.321, 'B4':1.679, 'B5':0.313, 'B6':1.637, 'd2':3.173, '1/d2':0.3152, 'd3':0.787, 'D1':0.811, 'D2':5.535, 'D3':0.256, 'D4':1.744},
                12: {'A':0.866, 'A2':0.266, 'A3':0.886, 'c4':0.9776, '1/c4':1.0229, 'B3':0.354, 'B4':1.646, 'B5':0.346, 'B6':1.610, 'd2':3.258, '1/d2':0.3069, 'd3':0.778, 'D1':0.922, 'D2':5.594, 'D3':0.283, 'D4':1.717},
                13: {'A':0.832, 'A2':0.249, 'A3':0.850, 'c4':0.9794, '1/c4':1.0210, 'B3':0.382, 'B4':1.618, 'B5':0.374, 'B6':1.585, 'd2':3.336, '1/d2':0.2998, 'd3':0.770, 'D1':1.025, 'D2':5.647, 'D3':0.307, 'D4':1.693},
                14: {'A':0.802, 'A2':0.235, 'A3':0.817, 'c4':0.9810, '1/c4':1.0194, 'B3':0.406, 'B4':1.594, 'B5':0.399, 'B6':1.563, 'd2':3.407, '1/d2':0.2935, 'd3':0.763, 'D1':1.118, 'D2':5.696, 'D3':0.328, 'D4':1.672},
                15: {'A':0.775, 'A2':0.223, 'A3':0.789, 'c4':0.9823, '1/c4':1.0180, 'B3':0.428, 'B4':1.572, 'B5':0.421, 'B6':1.544, 'd2':3.472, '1/d2':0.2880, 'd3':0.756, 'D1':1.203, 'D2':5.741, 'D3':0.347, 'D4':1.653},
                16: {'A':0.750, 'A2':0.212, 'A3':0.763, 'c4':0.9835, '1/c4':1.0168, 'B3':0.448, 'B4':1.552, 'B5':0.440, 'B6':1.526, 'd2':3.532, '1/d2':0.2831, 'd3':0.750, 'D1':1.282, 'D2':5.782, 'D3':0.363, 'D4':1.637},
                17: {'A':0.728, 'A2':0.203, 'A3':0.739, 'c4':0.9845, '1/c4':1.0157, 'B3':0.466, 'B4':1.534, 'B5':0.458, 'B6':1.511, 'd2':3.588, '1/d2':0.2787, 'd3':0.744, 'D1':1.356, 'D2':5.820, 'D3':0.378, 'D4':1.622},
                18: {'A':0.707, 'A2':0.194, 'A3':0.718, 'c4':0.9854, '1/c4':1.0148, 'B3':0.482, 'B4':1.518, 'B5':0.475, 'B6':1.496, 'd2':3.640, '1/d2':0.2747, 'd3':0.739, 'D1':1.424, 'D2':5.856, 'D3':0.391, 'D4':1.608},
                19: {'A':0.688, 'A2':0.187, 'A3':0.698, 'c4':0.9862, '1/c4':1.0140, 'B3':0.497, 'B4':1.503, 'B5':0.490, 'B6':1.483, 'd2':3.689, '1/d2':0.2711, 'd3':0.734, 'D1':1.487, 'D2':5.891, 'D3':0.403, 'D4':1.597},
                20: {'A':0.671, 'A2':0.180, 'A3':0.680, 'c4':0.9869, '1/c4':1.0133, 'B3':0.510, 'B4':1.490, 'B5':0.504, 'B6':1.470, 'd2':3.735, '1/d2':0.2677, 'd3':0.729, 'D1':1.549, 'D2':5.921, 'D3':0.415, 'D4':1.585},
                21: {'A':0.655, 'A2':0.173, 'A3':0.663, 'c4':0.9876, '1/c4':1.0126, 'B3':0.523, 'B4':1.477, 'B5':0.516, 'B6':1.459, 'd2':3.778, '1/d2':0.2647, 'd3':0.724, 'D1':1.605, 'D2':5.951, 'D3':0.425, 'D4':1.575},
                22: {'A':0.640, 'A2':0.167, 'A3':0.647, 'c4':0.9882, '1/c4':1.0119, 'B3':0.534, 'B4':1.466, 'B5':0.528, 'B6':1.448, 'd2':3.819, '1/d2':0.2618, 'd3':0.720, 'D1':1.659, 'D2':5.979, 'D3':0.434, 'D4':1.566},
                23: {'A':0.626, 'A2':0.162, 'A3':0.633, 'c4':0.9887, '1/c4':1.0114, 'B3':0.545, 'B4':1.455, 'B5':0.539, 'B6':1.438, 'd2':3.858, '1/d2':0.2592, 'd3':0.716, 'D1':1.710, 'D2':6.006, 'D3':0.443, 'D4':1.557},
                24: {'A':0.612, 'A2':0.157, 'A3':0.619, 'c4':0.9892, '1/c4':1.0109, 'B3':0.555, 'B4':1.445, 'B5':0.549, 'B6':1.429, 'd2':3.895, '1/d2':0.2567, 'd3':0.712, 'D1':1.759, 'D2':6.031, 'D3':0.451, 'D4':1.548},
                25: {'A':0.600, 'A2':0.153, 'A3':0.606, 'c4':0.9896, '1/c4':1.0105, 'B3':0.565, 'B4':1.435, 'B5':0.559, 'B6':1.420, 'd2':3.931, '1/d2':0.2544, 'd3':0.708, 'D1':1.806, 'D2':6.056, 'D3':0.459, 'D4':1.541},
            }
            
            # Access the key:
            dict_of_constants = dict_of_constants[self.sample_size]
            
        else: #>= 26
            
            dict_of_constants = {'A':(3/(self.sample_size**(0.5))), 'A2':0.153, 
                                 'A3':3/((4*(self.sample_size-1)/(4*self.sample_size-3))*(self.sample_size**(0.5))), 
                                 'c4':(4*(self.sample_size-1)/(4*self.sample_size-3)), 
                                 '1/c4':1/((4*(self.sample_size-1)/(4*self.sample_size-3))), 
                                 'B3':(1-3/(((4*(self.sample_size-1)/(4*self.sample_size-3)))*((2*(self.sample_size-1))**(0.5)))), 
                                 'B4':(1+3/(((4*(self.sample_size-1)/(4*self.sample_size-3)))*((2*(self.sample_size-1))**(0.5)))),
                                 'B5':(((4*(self.sample_size-1)/(4*self.sample_size-3)))-3/((2*(self.sample_size-1))**(0.5))), 
                                 'B6':(((4*(self.sample_size-1)/(4*self.sample_size-3)))+3/((2*(self.sample_size-1))**(0.5))), 
                                 'd2':3.931, '1/d2':0.2544, 'd3':0.708, 'D1':1.806, 'D2':6.056, 'D3':0.459, 'D4':1.541}
        
        # Update the attribute
        self.dict_of_constants = dict_of_constants
        
        return self
    
    def get_histogram_array (self):
        
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        y_hist = df[column_with_variable_to_be_analyzed]
        lowest = self.lowest
        highest = self.highest
        sample_size = self.sample_size
        
        # Number of bins set by the user:
        total_of_bins = self.total_of_bins
        
        # Firstly, get the ideal bin-size according to the Montgomery's method:
        # Douglas C. Montgomery (2009). Introduction to Statistical Process Control, 
        # Sixth Edition, John Wiley & Sons.
        # Sort by the column to analyze (ascending order) and reset the index:
        y_hist = y_hist.sort_values(ascending = True)
        y_hist = y_hist.reset_index(drop = True)
        #Calculo do bin size - largura do histograma:
        #1: Encontrar o menor (lowest) e o maior (highest) valor dentro da tabela de dados)
        #2: Calcular rangehist = highest - lowest
        #3: Calcular quantidade de dados (samplesize) de entrada fornecidos
        #4: Calcular a quantidade de celulas da tabela de frequencias (ncells)
        #ncells = numero inteiro mais proximo da (raiz quadrada de samplesize)
        #5: Calcular binsize = (df[column_to_analyze])rangehist/(ncells)
        #ATENCAO: Nao se esquecer de converter range, ncells, samplesize e binsize para valores absolutos (modulos)
        #isso porque a largura do histograma tem que ser um numero positivo 

        # bin-size
        range_hist = abs(highest - lowest)
        n_cells = int(np.rint((sample_size)**(0.5)))
        # We must use the int function to guarantee that the ncells will store an
        # integer number of cells (we cannot have a fraction of a sentence).
        # The int function guarantees that the variable will be stored as an integer.
        # The numpy.rint(a) function rounds elements of the array to the nearest integer.
        # https://numpy.org/doc/stable/reference/generated/numpy.rint.html
        # For values exactly halfway between rounded decimal values, 
        # NumPy rounds to the nearest even value. 
        # Thus 1.5 and 2.5 round to 2.0; -0.5 and 0.5 round to 0.0; etc.
        if (n_cells > 3):
            
            print(f"Ideal number of histogram bins calculated through Montgomery's method = {n_cells} bins.\n")
        
        # Retrieve the histogram array hist_array
        fig, ax = plt.subplots() # (0,0) not to show the plot now:
        
        # Get a 10-bins histogram:
        hist_array = plt.hist(y_hist, bins = total_of_bins)
        plt.delaxes(ax) # this will delete ax, so that it will not be plotted.
        plt.show()
        print("") # use this print not to mix with the final plot

        # hist_array is an array of arrays:
        # hist_array = (array([count_1, count_2, ..., cont_n]), array([bin_center_1,...,
        # bin_center_n])), where n = total_of_bins
        # hist_array[0] is the array of countings for each bin, whereas hist_array[1] is
        # the array of the bin center, i.e., the central value of the analyzed variable for
        # that bin.

        # It is possible that the hist_array[0] contains more elements than hist_array[1].
        # This happens when the last bins created by the division contain zero elements.
        # In this case, we have to pad the sequence of hist_array[0], completing it with zeros.

        MAX_LENGTH = max(len(hist_array[0]), len(hist_array[1])) # Get the length of the longest sequence
        SEQUENCES = [list(hist_array[0]), list(hist_array[1])] # get a list of sequences to pad.
        # Notice that we applied the list attribute to create a list of lists

        # We cannot pad with the function pad_sequences from tensorflow because it converts all values
        # to integers. Then, we have to pad the sequences by looping through the elements from SEQUENCES:

        # Start a support_list
        support_list = []

        # loop through each sequence in SEQUENCES:
        for sequence in SEQUENCES:
            # add a zero at the end of the sequence until its length reaches MAX_LENGTH
            while (len(sequence) < MAX_LENGTH):

                sequence.append(0)

            # append the sequence to support_list:
            support_list.append(sequence)

        # Tuples and arrays are immutable. It means they do not support assignment, i.e., we cannot
        # do tuple[0] = variable. Since arrays support vectorial (element-wise) operations, we can
        # modify the whole array making it equals to support_list at once by using function np.array:
        hist_array = np.array(support_list)

        # Get the bin_size as the average difference between successive elements from support_list[1]:

        diff_lists = []

        for i in range (1, len(support_list[1])):

            diff_lists.append(support_list[1][i] - support_list[1][(i-1)])

        # Now, get the mean value as the bin_size:
        bin_size = np.amax(np.array(diff_lists))

        # Let's get the frequency table, which will be saved on DATASET (to get the code
        # equivalent to the code for the function 'histogram'):

        DATASET = pd.DataFrame(data = {'bin_center': hist_array[1], 'count': hist_array[0]})

        # Get a lists of bin_center and column_to_analyze:
        list_of_bins = list(hist_array[1])
        list_of_counts = list(hist_array[0])

        # get the maximum count:
        max_count = DATASET['count'].max()
        # Get the index of the max count:
        max_count_index = list_of_counts.index(max_count)

        # Get the value bin_center correspondent to the max count (maximum probability):
        bin_of_max_proba = list_of_bins[max_count_index]
        bin_after_the_max_proba = list_of_bins[(max_count_index + 1)] # the next bin
        number_of_bins = len(DATASET) # Total of elements on the frequency table
        
        # Obtain a list of differences between bins
        bins_diffs = [(list_of_bins[i] - list_of_bins[(i-1)]) for i in range (1, len(list_of_bins))]
        # Convert it to Pandas series and use the mean method to retrieve the average bin size:
        bin_size = pd.Series(bins_diffs).mean()
        
        self.histogram_dict = {'df': DATASET, 'list_of_bins': list_of_bins, 'list_of_counts': list_of_counts,
                              'max_count': max_count, 'max_count_index': max_count_index,
                              'bin_of_max_proba': bin_of_max_proba, 'bin_after_the_max_proba': bin_after_the_max_proba,
                              'number_of_bins': number_of_bins, 'bin_size': bin_size}
        
        return self
    
    def get_desired_normal (self):
        
        import numpy as np
        import pandas as pd
        
        # Get a normal completely (6s) in the specifications, and centered
        # within these limits
        
        mu = self.mu
        sigma = self.sigma
        histogram_dict = self.histogram_dict
        max_count = histogram_dict['max_count']
        
        specification_limits = self.specification_limits
        
        lower_spec = specification_limits['lower_spec_lim']
        upper_spec = specification_limits['upper_spec_lim']
        
        if (lower_spec is None):
            
            # There is no lower specification: everything below it is in the specifications.
            # Make it mean - 6sigma (virtually infinite).
            lower_spec = mu - 6*(sigma)
            # Update the dictionary:
            specification_limits['lower_spec_lim'] = lower_spec
        
        if (upper_spec is None):
            
            # There is no upper specification: everything above it is in the specifications.
            # Make it mean + 6sigma (virtually infinite).
            upper_spec = mu + 6*(sigma)
            # Update the dictionary:
            specification_limits['upper_spec_lim'] = upper_spec
        
        # Desired normal mu: center of the specification limits.
        desired_mu = (lower_spec + upper_spec)/2
        
        # Desired sigma: 6 times the variation within the specific limits
        desired_sigma = (upper_spec - lower_spec)/6
        
        if (desired_sigma == 0):
            print("Impossible to obtain a normal curve overlayed, because the standard deviation is zero.\n")
            print("The analyzed variable is constant throughout the whole sample space.\n")
            
            # Get a dictionary of empty lists for this case
            desired_normal = {'x': [], 'y':[]}
            
        else:
            # create lists to store the normal curve. Center the normal curve in the bin
            # of maximum bar (max probability, which will not be the mean if the curve
            # is skewed). For normal distributions, this value will be the mean and the median.

            # set the lowest value x used for obtaining the normal curve as center_of_bin_of_max_proba - 4*sigma
            # the highest x will be center_of_bin_of_max_proba - 4*sigma
            # each value will be created by incrementing (0.10)*sigma

            # The arrays created by the plt.hist method present the value of the extreme left 
            # (the beginning) of the histogram bars, not the bin center. So, let's add half of the bin size
            # to the bin_of_max_proba, so that the adjusted normal will be positioned on the center of the
            # bar of maximum probability. We can do it by taking the average between bin_of_max_proba
            # and the following bin, bin_after_the_max_proba:
            
            # Let's create a normal around the desired mean value. Firstly, create the range X - 4s to
            # X + 4s. The probabilities will be calculated for each value in this range:

            x = (desired_mu - (4 * desired_sigma))
            x_of_normal = [x]

            while (x < (desired_mu + (4 * desired_sigma))):

                x = x + (0.10)*(desired_sigma)
                x_of_normal.append(x)

            # Convert the list to a NumPy array, so that it is possible to perform element-wise
            # (vectorial) operations:
            x_of_normal = np.array(x_of_normal)

            # Create an array of the normal curve y, applying the normal curve equation:
            # normal curve = 1/(sigma* ((2*pi)**(0.5))) * exp(-((x-mu)**2)/(2*(sigma**2)))
            # where pi = 3,14...., and exp is the exponential function (base e)
            # Let's center the normal curve on desired_mu:
            y_normal = (1 / (desired_sigma* (np.sqrt(2 * (np.pi))))) * (np.exp(-0.5 * (((1 / desired_sigma) * (x_of_normal - desired_mu)) ** 2)))
            y_normal = np.array(y_normal)

            # Pick the maximum value obtained for y_normal:
            # https://numpy.org/doc/stable/reference/generated/numpy.amax.html#numpy.amax
            y_normal_max = np.amax(y_normal)

            # Let's get a correction factor, comparing the maximum of the histogram counting, max_count,
            # with y_normal_max:
            correction_factor = max_count/(y_normal_max)

            # Now, multiply each value of the array y_normal by the correction factor, to adjust the height:
            y_normal = y_normal * correction_factor
            # Now the probability density function (values originally from 0 to 1) has the same 
            # height as the histogram.
            
            desired_normal = {'x': x_of_normal, 'y': y_normal}
        
        # Nest the desired_normal dictionary into specification_limits dictionary:
        specification_limits['desired_normal'] = desired_normal
        # Update the attribute:
        self.specification_limits = specification_limits
        
        return self
    
    def get_fitted_normal (self):
        
        import numpy as np
        import pandas as pd
        
        # Get a normal completely (6s) in the specifications, and centered
        # within these limits
        
        mu = self.mu
        sigma = self.sigma
        histogram_dict = self.histogram_dict
        max_count = histogram_dict['max_count']
        bin_of_max_proba = histogram_dict['bin_of_max_proba']
        specification_limits = self.specification_limits
        
        if (sigma == 0):
            print("Impossible to obtain a normal curve overlayed, because the standard deviation is zero.\n")
            print("The analyzed variable is constant throughout the whole sample space.\n")
            
            # Get a dictionary of empty lists for this case
            fitted_normal = {'x': [], 'y':[]}
            
        else:
            # create lists to store the normal curve. Center the normal curve in the bin
            # of maximum bar (max probability, which will not be the mean if the curve
            # is skewed). For normal distributions, this value will be the mean and the median.

            # set the lowest value x used for obtaining the normal curve as bin_of_max_proba - 4*sigma
            # the highest x will be bin_of_max_proba - 4*sigma
            # each value will be created by incrementing (0.10)*sigma

            x = (bin_of_max_proba - (4 * sigma))
            x_of_normal = [x]

            while (x < (bin_of_max_proba + (4 * sigma))):

                x = x + (0.10)*(sigma)
                x_of_normal.append(x)

            # Convert the list to a NumPy array, so that it is possible to perform element-wise
            # (vectorial) operations:
            x_of_normal = np.array(x_of_normal)

            # Create an array of the normal curve y, applying the normal curve equation:
            # normal curve = 1/(sigma* ((2*pi)**(0.5))) * exp(-((x-mu)**2)/(2*(sigma**2)))
            # where pi = 3,14...., and exp is the exponential function (base e)
            # Let's center the normal curve on bin_of_max_proba
            y_normal = (1 / (sigma* (np.sqrt(2 * (np.pi))))) * (np.exp(-0.5 * (((1 / sigma) * (x_of_normal - bin_of_max_proba)) ** 2)))
            y_normal = np.array(y_normal)

            # Pick the maximum value obtained for y_normal:
            # https://numpy.org/doc/stable/reference/generated/numpy.amax.html#numpy.amax
            y_normal_max = np.amax(y_normal)

            # Let's get a correction factor, comparing the maximum of the histogram counting, max_count,
            # with y_normal_max:
            correction_factor = max_count/(y_normal_max)

            # Now, multiply each value of the array y_normal by the correction factor, to adjust the height:
            y_normal = y_normal * correction_factor
            
            fitted_normal = {'x': x_of_normal, 'y': y_normal}
        
        # Nest the fitted_normal dictionary into specification_limits dictionary:
        specification_limits['fitted_normal'] = fitted_normal
        # Update the attribute:
        self.specification_limits = specification_limits
        
        return self
    
    def get_actual_pdf (self):
        
        # PDF: probability density function.
        # KDE: Kernel density estimation: estimation of the actual probability density
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html#scipy.stats.gaussian_kde
        
        import numpy as np
        import pandas as pd
        from scipy import stats
        
        df = self.df
        column_with_variable_to_be_analyzed = self.column_with_variable_to_be_analyzed
        array_to_analyze = np.array(df[column_with_variable_to_be_analyzed])
        
        mu = self.mu
        sigma = self.sigma
        lowest = self.lowest
        highest = self.highest
        sample_size = self.sample_size
        
        histogram_dict = self.histogram_dict
        max_count = histogram_dict['max_count']
        specification_limits = self.specification_limits 
        
        # Get the KDE object
        kde = stats.gaussian_kde(array_to_analyze)
        
        # Here, kde may represent a distribution with high skewness and kurtosis. So, let's check
        # if the intervals mu - 6s and mu + 6s are represented by the array:
        inf_kde_lim = mu - 6*sigma
        sup_kde_lim = mu + 6*sigma
        
        if (inf_kde_lim > min(list(array_to_analyze))):
            # make the inferior limit the minimum value from the array:
            inf_kde_lim = min(list(array_to_analyze))
        
        if (sup_kde_lim < max(list(array_to_analyze))):
            # make the superior limit the minimum value from the array:
            sup_kde_lim = max(list(array_to_analyze))
        
        # Let's obtain a X array, consisting with all values from which we will calculate the PDF:
        new_x = inf_kde_lim
        new_x_list = [new_x]
        
        while ((new_x) < sup_kde_lim):
            # There is already the first element, so go to the next one.
            new_x = new_x + (0.10)*sigma
            new_x_list.append(new_x)
        
        # Convert the new_x_list to NumPy array, making it the array_to_analyze:
        array_to_analyze = np.array(new_x_list)
        
        # Apply the pdf method to convert the array_to_analyze into the array of probabilities:
        # i.e., calculate the probability for each one of the values in array_to_analyze:
        # PDF: Probability density function
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.pdf.html#scipy.stats.gaussian_kde.pdf
        array_of_probs = kde.pdf(array_to_analyze)
        
        # Pick the maximum value obtained for array_of_probs:
        # https://numpy.org/doc/stable/reference/generated/numpy.amax.html#numpy.amax
        array_of_probs_max = np.amax(array_of_probs)

        # Let's get a correction factor, comparing the maximum of the histogram counting, max_count,
        # with array_of_probs_max:
        correction_factor = max_count/(array_of_probs_max)

        # Now, multiply each value of the array y_normal by the correction factor, to adjust the height:
        array_of_probs = array_of_probs * correction_factor
        # Now the probability density function (values originally from 0 to 1) has the same 
        # height as the histogram.
        
        # Define a dictionary
        # X of the probability density plot: values from the series being analyzed.
        # Y of the probability density plot: probabilities calculated for each X.
        actual_pdf = {'x': array_to_analyze, 'y': array_of_probs}
        
        # Nest the desired_normal dictionary into specification_limits dictionary:
        specification_limits['actual_pdf'] = actual_pdf
        # Update the attribute:
        self.specification_limits = specification_limits
        
        return self
    
    def get_capability_indicators (self):
        
        import numpy as np
        import pandas as pd
        
        # Get a normal completely (6s) in the specifications, and centered
        # within these limits
        
        mu = self.mu
        sigma = self.sigma
        histogram_dict = self.histogram_dict
        bin_of_max_proba = histogram_dict['bin_of_max_proba']
        bin_after_the_max_proba = histogram_dict['bin_after_the_max_proba']
        max_count = histogram_dict['max_count']
        
        specification_limits = self.specification_limits
        lower_spec = specification_limits['lower_spec_lim']
        upper_spec = specification_limits['upper_spec_lim']
        desired_mu = (lower_spec + upper_spec)/2 
        # center of the specification limits: we want the mean to be in the center of the
        # specification limits
        
        range_spec = abs(upper_spec - lower_spec)
        
        # Get the constant:
        self = self.get_constants()
        dict_of_constants = self.dict_of_constants
        constant = dict_of_constants['1/c4']
        
        # Calculate corrected sigma:
        sigma_corrected = sigma*constant
        
        # Calculate the capability indicators, adding them to the
        # capability_dict
        cp = (range_spec)/(6*sigma_corrected)
        cr = 100*(6*sigma_corrected)/(range_spec)
        cm = (range_spec)/(8*sigma_corrected)
        zu = (upper_spec - mu)/(sigma_corrected)
        zl = (mu - lower_spec)/(sigma_corrected)
        
        z_min = min(zu, zl)
        cpk = (z_min)/3

        cpm_factor = 1 + ((mu - desired_mu)/sigma_corrected)**2
        cpm_factor = cpm_factor**(0.5) # square root
        cpm = (cp)/(cpm_factor)
        
        capability_dict = {'indicator': ['cp', 'cr', 'cm', 'zu', 'zl', 'z_min', 'cpk', 'cpm'], 
                            'value': [cp, cr, cm, zu, zl, z_min, cpk, cpm]}
        # Already in format for pd.DataFrame constructor
        
        # Update the attribute:
        self.capability_dict = capability_dict
        
        return self
    
    def capability_interpretation (self):
       
        print("Capable process: a process which attends its specifications.")
        print("Naturally, we want processes capable of attending the specifications.\n")
        
        print("Specification range:")
        print("Absolute value of the difference between the upper and the lower limits of specification.\n")
        
        print("6s interval:")
        print("Consider mean value = mu; standard deviation = s")
        print("For a normal distribution, 99.7% of the values range from its (mu - 3s) to (mu + 3s).")
        print("So, if the process follows the normal distribution, we can consider that virtually all of the data is in this range with 6s width.\n")
        
        print ("Cp:")
        print ("Relation between specification range and 6s.\n")
        
        print("Cr:")
        print("Usually, 6s > specification range.")
        print("So, the inverse of Cp is the fraction of 6s correspondent to the specification range.")
        print("Example: if 1/Cp = 0.2, then the specification range corresponds to 0.20 (20%) of the 6s interval.")
        print("Cr = 100 x (1/Cp) - the percent of 6s correspondent to the specification range.")
        print("Again, if 1/Cp = 0.2, then Cr = 20: the specification range corresponds to 20% of the 6s interval.\n")
        
        print("Cm:")
        print("It is a more generalized version of Cp.")
        print("Cm is the relation between specification range and 8s.")
        print("Then, even highly distant values from long-tailed curves are analyzed by this indicator.\n")
        
        print("Zu:")
        print("Represents how far is the mean of the values from the upper specification limit.")
        print("Zu = ([upper specification limit] - mu)/s")
        print("A higher Zu indicates a mean value lower than (and more distant from) the upper specification.")
        print("A negative Zu, in turns, indicates that the mean value is greater than the upper specification (i.e.: in average, specification is not attended).\n")
        
        print("Zl:")
        print("Represents how far is the mean of the values from the lower specification limit.")
        print("Zl = (mu - [lower specification limit])/s\n")
        print("A higher Zl indicates a mean value higher than  (and more distant from) the lower specification.")
        print("A negative Zl, in turns, indicates that the mean value is inferior than the lower specification (i.e.: in average, specification is not attended).\n")
        
        print("Zmin:")
        print("It is the minimum value between Zu and Zl.")
        print("So, Zmin indicates which specification is more difficult for the process to attend: the upper or the lower one.")
        print("Example: if Zmin = Zl, the mean of the process is closer to the lower specification than it is from the upper specification.")
        print("If Zmin, Zu, and Zl are equal, than the process is equally distant from the two specifications.")
        print("Again, if Zmin is negative, at least one of the specifications is not attended.\n")
        
        print("Cpk:")
        print("This is the most fundamental capability indicator.")
        print("Consider again that 99.7% of the normally distributed data are within [(mu - 3s), (mu + 3s)].")
        print("Cpk = Zmin/3")
        print("Cpk = min((([upper specification limit] - mu)/3s), ((mu - [lower specification limit])/3s))")
        print("\n")
        print("Cpk simultaneously assess the process centrality, and if the process is capable of attending its specifications.")
        print("Here, the process centrality is verified as results which are well and simetrically distributed throughout the mean of the specification limits.")
        print("Basically, a perfectly-centralized process has its mean equally distant from both specifications")
        print("i.e., the mean is in the center of the specification interval.")
        print("Cpk = + 1 is usually considered the minimum value acceptable for a process.")
        print("Many quality programs define reaching Cpk = + 1.33 as their goal.")
        print("A 6-sigma process, in turns, is defined as a process with Cpk = + 2.")
        print("\n")
        print("High values of Cpk indicate that the process is not only centralized, but that the differences")
        print("([upper specification limit] - mu) and (mu - [lower specification limit]) are greater than 3s.")
        print("Since mu +- 3s is the range for 99.7% of data, it indicates that most of the values generated fall in a range")
        print("that is only a fraction of the specification range.")
        print("So, it is easier for the process to attend the specifications.")
        print("\n")
        print("Cpk values inferior than 1 indicate that at least one of the intervals ([upper specification limit] - mu) and (mu - [lower specification limit])")
        print("is lower than 3s, i.e., the process naturally generates values beyond at least one of the specifications.")
        print("Low values of Cpk (in particular the negative ones) indicate not-centralized processes and processes not capable of attending their specifications.")
        print("So, lower (and, specially, more negative) Cpk: process' outputs more distant from the specifications.\n")
        
        print("Cpm:")
        print("This indicator is a more generalized version of the Cpk.")
        print("It basically consists on a standard normalization of the Cpk.")
        print("For that, a normalization factor is defined as:")
        print("factor = square root(1 + ((mu - target)/s)**2)")
        print("where target is the center of the specification limits, and **2 represents the second power (square)")
        print("Cpm = Cpk/(factor)")

In [126]:
def process_capability (df, column_with_variable_to_be_analyzed, specification_limits = {'lower_spec_lim': None, 'upper_spec_lim': None}, reference_value = None, x_axis_rotation = 0, y_axis_rotation = 0, grid = True, horizontal_axis_title = None, vertical_axis_title = None, plot_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 330):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from scipy import stats
    
    # COLUMN_WITH_VARIABLE_TO_BE_ANALYZED: name (header) of the column containing the variable
    # which stability will be analyzed by the control chart. The column name may be a string or a number.
    # Example: COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'analyzed_column' will analyze a column named
    # "analyzed_column", whereas COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'col1' will evaluate column 'col1'.

    # SPECIFICATION_LIMITS = {'lower_spec_lim': None, 'upper_spec_lim': None}
    # If there are specification limits, input them in this dictionary. Do not modify the keys,
    # simply substitute None by the lower and/or the upper specification.
    # e.g. Suppose you have a tank that cannot have more than 10 L. So:
    # SPECIFICATION_LIMITS = {'lower_spec_lim': None, 'upper_spec_lim': 10}, there is only
    # an upper specification equals to 10 (do not add units);
    # Suppose a temperature cannot be lower than 10 ºC, but there is no upper specification. So,
    # SPECIFICATION_LIMITS = {'lower_spec_lim': 10, 'upper_spec_lim': None}. Finally, suppose
    # a liquid which pH must be between 6.8 and 7.2:
    # SPECIFICATION_LIMITS = {'lower_spec_lim': 6.8, 'upper_spec_lim': 7.2}

    # REFERENCE_VALUE: keep it as None or add a float value.
    # This reference value will be shown as a red constant line to be compared
    # with the plots. e.g. REFERENCE_VALUE = 1.0 will plot a line passing through
    # VARIABLE_TO_ANALYZE = 1.0
    
    # Set a local copy of the dataframe to manipulate:
    DATASET = df.copy(deep = True)
    
    # Sort by the column to analyze (ascending order) and reset the index:
    DATASET = DATASET.sort_values(by = column_with_variable_to_be_analyzed, ascending = True)
    
    DATASET = DATASET.reset_index(drop = True)
    
    # Create an instance (object) from class capability_analysis:
    capability_obj = capability_analysis(df = DATASET, column_with_variable_to_be_analyzed = column_with_variable_to_be_analyzed, specification_limits = specification_limits)
    
    # Check data normality:
    capability_obj = capability_obj.check_data_normality()
    # Attribute .normality_dict: dictionary with results from normality tests
    
    # Get histogram array:
    capability_obj = capability_obj.get_histogram_array()
    # Attribute .histogram_dict: dictionary with keys 'list_of_bins' and 'list_of_counts'.
    
    # Get desired normal:
    capability_obj = capability_obj.get_desired_normal()
    # Now the .specification_limits attribute contains the nested dict desired_normal = {'x': x_of_normal, 'y': y_normal}
    # in key 'desired_normal'.
    
    # Get the actual probability density function (PDF):
    capability_obj = capability_obj.get_actual_pdf()
    # Now the dictionary in the attribute .specification_limits has the nested dict actual_pdf = {'x': array_to_analyze, 'y': array_of_probs}
    # in key 'actual_pdf'.
    
    # Get capability indicators:
    capability_obj = capability_obj.get_capability_indicators()
    # Now the attribute capability_dict has a dictionary with the indicators, already in the format
    # for pandas DataFrame constructor.
    
    # Retrieve general statistics:
    stats_dict = {
        
        'sample_size': capability_obj.sample_size,
        'mu': capability_obj.mu,
        'median': capability_obj.median,
        'sigma': capability_obj.sigma,
        'lowest': capability_obj.lowest,
        'highest': capability_obj.highest
    }
    
    # Retrieve the normality summary:
    normality_dict = capability_obj.normality_dict
    # Nest this dictionary in stats_dict:
    stats_dict['normality_dict'] = normality_dict
    
    # Retrieve the histogram dict:
    histogram_dict = capability_obj.histogram_dict
    # Nest this dictionary in stats_dict:
    stats_dict['histogram_dict'] = histogram_dict
    
    # Retrieve the specification limits dictionary updated:
    specification_limits = capability_obj.specification_limits
    # Retrieve the desired normal and actual PDFs dictionaries:
    desired_normal = specification_limits['desired_normal']
    # Nest this dictionary in stats_dict:
    stats_dict['desired_normal'] = desired_normal
    
    actual_pdf = specification_limits['actual_pdf']
    # Nest this dictionary in stats_dict:
    stats_dict['actual_pdf'] = actual_pdf
    
    # Retrieve the capability dictionary:
    capability_dict = capability_obj.capability_dict
    # Create a capability dataframe:
    capability_df = pd.DataFrame(data = capability_dict)
    # Set column 'indicator' as index:
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
    capability_df.set_index('indicator', inplace = True)
    # Nest this dataframe in stats_dict:
    stats_dict['capability_df'] = capability_df
    
    print("\n")
    print("Check the capability summary dataframe:\n")
    
    try:
        # only works in Jupyter Notebook:
        from IPython.display import display
        display(capability_df)
            
    except: # regular mode
        print(capability_df)
    
    # Print the indicators' interpretation:
    capability_obj.capability_interpretation()
    
    string_for_title = " - $\mu = %.2f$, $\sigma = %.2f$" %(stats_dict['mu'], stats_dict['sigma'])
    
    if (plot_title is not None):
        plot_title = plot_title + string_for_title
        
    else:
        # Set graphic title
        plot_title = f"Process Capability" + string_for_title

    if (horizontal_axis_title is None):
        # Set horizontal axis title
        horizontal_axis_title = column_with_variable_to_be_analyzed

    if (vertical_axis_title is None):
        # Set vertical axis title
        vertical_axis_title = "Counting/Frequency"
        
    y_hist = DATASET[column_with_variable_to_be_analyzed]
    number_of_bins = histogram_dict['number_of_bins']
    
    upper_spec = specification_limits['upper_spec_lim']
    lower_spec = specification_limits['lower_spec_lim']
    target = (upper_spec + lower_spec)/2 # center of the specification range
    
    # Let's put a small degree of transparency (1 - OPACITY) = 0.05 = 5%
    # so that the bars do not completely block other views.
    OPACITY = 0.95
    
    # Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    fig = plt.figure(figsize = (12, 8))
    ax = fig.add_subplot()
    
    #STANDARD MATPLOTLIB METHOD:
    #bins = number of bins (intervals) of the histogram. Adjust it manually
    #increasing bins will increase the histogram's resolution, but height of bars
    
    ax.hist(y_hist, bins = number_of_bins, alpha = OPACITY, label = f'counting_of\n{column_with_variable_to_be_analyzed}', color = 'darkblue')
    
    # desired_normal = {'x': x_of_normal, 'y': y_normal}
    # actual_pdf = {'x': array_to_analyze, 'y': array_of_probs}
    
    # Plot the probability density function for the data:
    pdf_x = actual_pdf['x']
    pdf_y = actual_pdf['y']
    
    ax.plot(pdf_x, pdf_y, color = 'darkgreen', linestyle = '-', alpha = OPACITY, label = 'probability\ndensity')
    
    # Check if a normal curve was obtained:
    x_of_normal = desired_normal['x']
    y_normal = desired_normal['y']
    
    if (len(x_of_normal) > 0):
        # Non-empty list, add the normal curve:
        ax.plot(x_of_normal, y_normal, color = 'crimson', linestyle = 'dashed', alpha = OPACITY, label = 'expected\nnormal_curve')
    
    # Add the specification limits and target vertical lines (axvline):
    ax.axvline(upper_spec, color = 'black', linestyle = 'dashed', label = 'specification\nlimit', alpha = OPACITY)
    ax.axvline(lower_spec, color = 'black', linestyle = 'dashed', alpha = OPACITY)
    ax.axvline(target, color = 'aqua', linestyle = 'dashed', label = 'target\nvalue', alpha = OPACITY)

    # If there is a reference value, plot it as a vertical line:
    if (reference_value is not None):
        ax.axvline(reference_value, color = 'fuchsia', linestyle = 'dashed', label = 'reference\nvalue', alpha = OPACITY)
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 0 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)

    ax.set_title(plot_title)
    ax.set_xlabel(horizontal_axis_title)
    ax.set_ylabel(vertical_axis_title)

    ax.grid(grid) # show grid or not
    ax.legend(loc = 'upper right')
    # position options: 'upper right'; 'upper left'; 'lower left'; 'lower right';
    # 'right', 'center left'; 'center right'; 'lower center'; 'upper center', 'center'
    # https://www.statology.org/matplotlib-legend-position/

    if (export_png == True):
        # Image will be exported
        import os

        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = ""

        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "capability_plot"

        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 330 dpi
            png_resolution_dpi = 330

        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)

        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")

    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    #plt.figure(figsize = (12, 8))
    #fig.tight_layout()

    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()

    return stats_dict

# **Function for exporting the dataframe as CSV File (to notebook's workspace)**

In [ ]:
def export_pd_dataframe_as_csv (dataframe_obj_to_be_exported, new_file_name_without_extension, file_directory_path = None):
    
    import os
    import pandas as pd
    
    ## WARNING: all files exported from this function are .csv (comma separated values)
    
    # dataframe_obj_to_be_exported: dataframe object that is going to be exported from the
    # function. Since it is an object (not a string), it should not be declared in quotes.
    # example: dataframe_obj_to_be_exported = dataset will export the dataset object.
    # ATTENTION: The dataframe object must be a Pandas dataframe.
    
    # FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
    # (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
    # or FILE_DIRECTORY_PATH = "/folder"
    # If you want to export the file to AWS S3, this parameter will have no effect.
    # In this case, you can set FILE_DIRECTORY_PATH = None

    # new_file_name_without_extension - (string, in quotes): input the name of the 
    # file without the extension. e.g. new_file_name_without_extension = "my_file" 
    # will export a file 'my_file.csv' to notebook's workspace.
    
    # Create the complete file path:
    file_path = os.path.join(file_directory_path, new_file_name_without_extension)
    # Concatenate the extension ".csv":
    file_path = file_path + ".csv"

    dataframe_obj_to_be_exported.to_csv(file_path, index = False)

    print(f"Dataframe {new_file_name_without_extension} exported as CSV file to notebook\'s workspace as \'{file_path}\'.")
    print("Warning: if there was a file in this file path, it was replaced by the exported dataframe.")

# **Function for importing or exporting models and dictionaries**

In [ ]:
def import_export_model_list_dict (action = 'import', objects_manipulated = 'model_only', model_file_name = None, dictionary_or_list_file_name = None, directory_path = '', model_type = 'keras', dict_or_list_to_export = None, model_to_export = None, use_colab_memory = False):
    
    import os
    import pickle
    import dill
    import tarfile
    import tensorflow as tf
    from zipfile import ZipFile
    # https://docs.python.org/3/library/tarfile.html#tar-examples
    # https://docs.python.org/3/library/zipfile.html#zipfile-objects
    # pickle and dill save the file in binary (bits) serialized mode. So, we must use
    # open 'rb' or 'wb' when calling the context manager. The 'b' stands for 'binary',
    # informing the context manager (with statement) that a bit-file will be processed
    from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
    from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
    from sklearn.neural_network import MLPRegressor, MLPClassifier
    from xgboost import XGBRegressor, XGBClassifier
    
    # action = 'import' for importing a model and/or a dictionary;
    # action = 'export' for exporting a model and/or a dictionary.
    
    # objects_manipulated = 'model_only' if only a model will be manipulated.
    # objects_manipulated = 'dict_or_list_only' if only a dictionary or list will be manipulated.
    # objects_manipulated = 'model_and_dict' if both a model and a dictionary will be
    # manipulated.
    
    # model_file_name: string with the name of the file containing the model (for 'import');
    # or of the name that the exported file will have (for 'export')
    # e.g. model_file_name = 'model'
    # WARNING: Do not add the file extension.
    # Keep it in quotes. Keep model_file_name = None if no model will be manipulated.
    
    # dictionary_or_list_file_name: string with the name of the file containing the dictionary 
    # (for 'import');
    # or of the name that the exported file will have (for 'export')
    # e.g. dictionary_or_list_file_name = 'history_dict'
    # WARNING: Do not add the file extension.
    # Keep it in quotes. Keep dictionary_or_list_file_name = None if no 
    # dictionary or list will be manipulated.
    
    # DIRECTORY_PATH: path of the directory where the model will be saved,
    # or from which the model will be retrieved. If no value is provided,
    # the DIRECTORY_PATH will be the root: "/"
    # Notice that the model and the dictionary must be stored in the same path.
    # If a model and a dictionary will be exported, they will be stored in the same
    # DIRECTORY_PATH.
    
    # model_type: This parameter has effect only when a model will be manipulated.
    # model_type = 'keras' for deep learning keras/ tensorflow models with extension .h5
    # model_type = 'tensorflow_general' for generic deep learning tensorflow models containing 
    # custom layers, losses and architectures. Such models are compressed as tar.gz, tar, or zip.
    # model_type = 'sklearn' for models from scikit-learn (non-deep learning)
    # model_type = 'xgb_regressor' for XGBoost regression models (non-deep learning)
    # model_type = 'xgb_classifier' for XGBoost classification models (non-deep learning)
    # model_type = 'arima' for ARIMA model (Statsmodels)
    
    # dict_or_list_to_export and model_to_export: 
    # These two parameters have effect only when ACTION == 'export'. In this case, they
    # must be declared. If ACTION == 'export', keep:
    # dict_or_list_to_export = None, 
    # model_to_export = None
    # If one of these objects will be exported, substitute None by the name of the object
    # e.g. if your model is stored in the global memory as 'keras_model' declare:
    # model_to_export = keras_model. Notice that it must be declared without quotes, since
    # it is not a string, but an object.
    # For exporting a dictionary named as 'dict':
    # dict_or_list_to_export = dict
    
    # use_colab_memory: this parameter has only effect when using Google Colab (or it will
    # raise an error). Set as use_colab_memory = True if you want to use the instant memory
    # from Google Colaboratory: you will update or download the file and it will be available
    # only during the time when the kernel is running. It will be excluded when the kernel
    # dies, for instance, when you close the notebook.
    
    # If action == 'export' and use_colab_memory == True, then the file will be downloaded
    # to your computer (running the cell will start the download).
    
    # Check the directory path
    if (directory_path is None):
        # set as the root (empty string):
        directory_path = ""
        
        
    bool_check1 = (objects_manipulated != 'model_only')
    # bool_check1 == True if a dictionary will be manipulated
    
    bool_check2 = (objects_manipulated != 'dict_or_list_only')
    # bool_check1 == True if a dictionary will be manipulated
    
    if (bool_check1 == True):
        #manipulate a dictionary
        
        if (dictionary_or_list_file_name is None):
            print("Please, enter a name for the dictionary or list.")
            return "error1"
        
        else:
            # Create the file path for the dictionary:
            dict_path = os.path.join(directory_path, dictionary_or_list_file_name)
            # Extract the file extension
            dict_extension = 'pkl'
            #concatenate:
            dict_path = dict_path + "." + dict_extension
            
    
    if (bool_check2 == True):
        #manipulate a model
        
        if (model_file_name is None):
            print("Please, enter a name for the model.")
            return "error1"
        
        else:
            # Create the file path for the dictionary:
            model_path = os.path.join(directory_path, model_file_name)
            # Extract the file extension
            
            #check model_type:
            if (model_type == 'keras'):
                model_extension = 'h5'
            
            elif (model_type == 'sklearn'):
                model_extension = 'dill'
                #it could be 'pkl', though
            
            elif (model_type == 'xgb_regressor'):
                model_extension = 'json'
                #it could be 'ubj', though
            
            elif (model_type == 'xgb_classifier'):
                model_extension = 'json'
                #it could be 'ubj', though
            
            elif (model_type == 'arima'):
                model_extension = 'pkl'
            
            # Finally, check if it is not the only one which can have several extensions:
            elif (model_type != 'tensorflow_general'):
                print("Enter a valid model_type: keras, tensorflow_general, sklearn, xgb_regressor, xgb_classifier, or arima.")
                return "error2"
            
            # If there is an extension, add it:
            if (model_type != 'tensorflow_general'):
                #concatenate:
                model_path = model_path +  "." + model_extension
            
    # Now we have the full paths for the dictionary and for the model.
    
    if (action == 'import'):
        
        if (use_colab_memory == True):
             
            from google.colab import files
            # google.colab library must be imported only in case 
            # it is going to be used, for avoiding 
            # AWS compatibility issues.
            
            print("Click on the button for file selection and select the files from your machine that will be uploaded in the Colab environment.")
            print("Warning: the files will be removed from Colab memory after the Kernel dies or after the notebook is closed.")
            # this functionality requires the previous declaration:
            ## from google.colab import files
            colab_files_dict = files.upload()
            # The files are stored into a dictionary called colab_files_dict where the keys
            # are the names of the files and the values are the files themselves.
            ## e.g. if you upload a single file named "dictionary.pkl", the dictionary will be
            ## colab_files_dict = {'dictionary.pkl': file}, where file is actually a big string
            ## representing the contents of the file. The length of this value is the size of the
            ## uploaded file, in bytes.
            ## To access the file is like accessing a value from a dictionary: 
            ## d = {'key1': 'val1'}, d['key1'] == 'val1'
            ## we simply declare the key inside brackets and quotes, the same way we would do for
            ## accessing the column of a dataframe.
            ## In this example, colab_files_dict['dictionary.pkl'] access the content of the 
            ## .pkl file, and len(colab_files_dict['dictionary.pkl']) is the size of the .pkl
            ## file in bytes.
            ## To check the dictionary keys, apply the method .keys() to the dictionary (with empty
            ## parentheses): colab_files_dict.keys()
            
            for key in colab_files_dict.keys():
                #loop through each element of the list of keys of the dictionary
                # (list colab_files_dict.keys()). Each element is named 'key'
                print(f"User uploaded file {key} with length {len(colab_files_dict[key])} bytes.")
                # The key is the name of the file, and the length of the value
                ## correspondent to the key is the file's size in bytes.
                ## Notice that the content of the uploaded object must be passed 
                ## as argument for a proper function to be interpreted. 
                ## For instance, the content of a xlsx file should be passed as
                ## argument for Pandas .read_excel function; the pkl file must be passed as
                ## argument for pickle.
                ## e.g., if you uploaded 'table.xlsx' and stored it into colab_files_dict you should
                ## declare df = pd.read_excel(colab_files_dict['table.xlsx']) to obtain a dataframe
                ## df from the uploaded table. Notice that is the value, not the key, that is the
                ## argument.
        
        if (bool_check1 == True):
            #manipulate a dictionary
            if (use_colab_memory == True):
                key = dictionary_file_name + "." + dict_extension
                #Use the key to access the file content, and pass the file content
                # to pickle:
                with open(colab_files_dict[key], 'rb') as opened_file:
            
                    imported_dict = pickle.load(opened_file)
                    # The structure imported_dict = pkl.load(open(colab_files_dict[key], 'rb')) relies 
                    # on the GC to close the file. That's not a good idea: If someone doesn't use 
                    # CPython the garbage collector might not be using refcounting (which collects 
                    # unreferenced objects immediately) but e.g. collect garbage only after some time.
                    # Since file handles are closed when the associated object is garbage collected or 
                    # closed explicitly (.close() or .__exit__() from a context manager) the file 
                    # will remain open until the GC kicks in.
                    # Using 'with' ensures the file is closed as soon as the block is left - even if 
                    # an exception happens inside that block, so it should always be preferred for any 
                    # real application.
                    # source: https://stackoverflow.com/questions/39447362/equivalent-ways-to-json-load-a-file-in-python

                print(f"Dictionary or list {key} successfully imported to Colab environment.")
            
            else:
                #standard method
                with open(dict_path, 'rb') as opened_file:
            
                    imported_dict = pickle.load(opened_file)
                
                # 'rb' stands for read binary (read mode). For writing mode, 'wb', 'write binary'
                print(f"Dictionary or list successfully imported from {dict_path}.")
                
        if (bool_check2 == True):
            #manipulate a model
            # select the proper model
        
            if (model_type == 'keras'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = tf.keras.models.load_model(colab_files_dict[key])
                    print(f"Keras/TensorFlow model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    # We previously declared:
                    # from keras.models import load_model
                    model = tf.keras.models.load_model(model_path)
                    print(f"Keras/TensorFlow model successfully imported from {model_path}.")
            
            elif (model_type == 'tensorflow_general'):
                
                print("Warning, save the model in a directory called 'saved_model' (before compressing.)\n")
                # Create a temporary folder in case it does not exist:
                # https://www.geeksforgeeks.org/python-os-makedirs-method/
                # Set exist_ok = True
                os.makedirs("tmp/", exist_ok = True)
                
                if (use_colab_memory == True):
                    
                    key = model_file_name
                    
                    try:
                        model_extension = ".tar"
                        key = key + model_extension
                        model_path = colab_files_dict[key]
                        # Open the context manager
                        with tarfile.open (model_path, 'r:') as compressed_model:
                            #extract all to the tmp directory:
                            compressed_model.extractall("tmp/")
                        
                        # if you were not using the context manager, it would be necessary to apply
                        # close method: tar = tarfile.open(fname, "r:gz"); tar.extractall(); tar.close()
                    
                    except:
                        
                        try:
                            # try tar.gz extension
                            model_extension = ".tar.gz"
                            key = key + model_extension
                            model_path = colab_files_dict[key]
                            
                            # Open the context manager
                            with tarfile.open (model_path, 'r:gz') as compressed_model:
                                #extract all to the tmp directory:
                                compressed_model.extractall("tmp/")
                        
                        except:
                            # try .zip extension
                            try:
                                model_extension = ".zip"
                                key = key + model_extension
                                model_path = colab_files_dict[key]
                                
                                # Open the context manager
                                with ZipFile (model_path, 'r') as compressed_model:
                                    #extract all to the tmp directory:
                                    compressed_model.extractall("tmp/")
                            
                            except:
                                print("Failed to load the model. Compress it as zip, tar or tar.gz file.\n")
                    
                    
                    # Compress the directory using tar
                    # https://www.gnu.org/software/tar/manual/tar.html
                    #    ! tar --extract --file=model_path --verbose --verbose tmp/
                    
                    try:
                        model = tf.keras.models.load_model("tmp/saved_model")
                        print(f"TensorFlow model: {model_path} successfully imported to Colab environment.")
                    
                    except:
                        print("Failed to load the model. Save it in a directory named 'saved_model' before compressing.\n")
                    
                else:
                    #standard method
                    
                    # Try simply accessing the directory:
                    try:
                        model = tf.keras.models.load_model("tmp/saved_model")
                    
                    except:
                        
                        try:
                            model = tf.keras.models.load_model(model_file_name)
                        
                        except:
                            
                            # It is compressed
                            try:
                                model_extension = ".tar"
                                model_path = model_file_name

                                # Open the context manager
                                with tarfile.open (model_path, 'r:') as compressed_model:
                                    #extract all to the tmp directory:
                                    compressed_model.extractall("tmp/")

                                # if you were not using the context manager, it would be necessary to apply
                                # close method: tar = tarfile.open(fname, "r:gz"); tar.extractall(); tar.close()

                            except:

                                try:
                                    # try tar.gz extension
                                    model_extension = ".tar.gz"
                                    model_path = model_file_name

                                    # Open the context manager
                                    with tarfile.open (model_path, 'r:gz') as compressed_model:
                                        #extract all to the tmp directory:
                                        compressed_model.extractall("tmp/")

                                except:
                                    # try .zip extension
                                    try:
                                        model_extension = ".zip"
                                        model_path = model_file_name

                                        # Open the context manager
                                        with ZipFile (model_path, 'r') as compressed_model:
                                            #extract all to the tmp directory:
                                            compressed_model.extractall("tmp/")

                                    except:
                                        print("Failed to load the model. Compress it as zip, tar or tar.gz file.\n")

                    
                    try:
                        model = tf.keras.models.load_model("tmp/saved_model")
                        print(f"TensorFlow model: {model_path} successfully imported to Colab environment.")
                    
                    except:
                        print("Failed to load the model. Save it in a directory named 'saved_model' before compressing.\n")
                    
                    
            elif (model_type == 'sklearn'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    
                    with open(colab_files_dict[key], 'rb') as opened_file:
            
                        model = dill.load(opened_file)
                    
                    print(f"Scikit-learn model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    with open(model_path, 'rb') as opened_file:
            
                        model = dill.load(opened_file)
                
                    print(f"Scikit-learn model successfully imported from {model_path}.")
                    # For loading a pickle model:
                    ## model = pkl.load(open(model_path, 'rb'))
                    # 'rb' stands for read binary (read mode). For writing mode, 'wb', 'write binary'

            elif (model_type == 'xgb_regressor'):
                
                # Create an instance (object) from the class XGBRegressor:
                
                model = XGBRegressor()
                # Now we can apply the load_model method from this class:
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = model.load_model(colab_files_dict[key])
                    print(f"XGBoost regression model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    model = model.load_model(model_path)
                    print(f"XGBoost regression model successfully imported from {model_path}.")
                    # model.load_model("model.json") or model.load_model("model.ubj")
                    # .load_model is a method from xgboost object
            
            elif (model_type == 'xgb_classifier'):

                # Create an instance (object) from the class XGBClassifier:

                model = XGBClassifier()
                # Now we can apply the load_model method from this class:
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = model.load_model(colab_files_dict[key])
                    print(f"XGBoost classification model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    model = model.load_model(model_path)
                    print(f"XGBoost classification model successfully imported from {model_path}.")
                    # model.load_model("model.json") or model.load_model("model.ubj")
                    # .load_model is a method from xgboost object

            elif (model_type == 'arima'):
                
                if (use_colab_memory == True):
                    key = model_file_name + "." + model_extension
                    model = ARIMAResults.load(colab_files_dict[key])
                    print(f"ARIMA model: {key} successfully imported to Colab environment.")
            
                else:
                    #standard method
                    # We previously declared:
                    # from statsmodels.tsa.arima.model import ARIMAResults
                    model = ARIMAResults.load(model_path)
                    print(f"ARIMA model successfully imported from {model_path}.")
            
            if (objects_manipulated == 'model_only'):
                # only the model should be returned
                return model
            
            elif (objects_manipulated == 'dict_only'):
                # only the dictionary should be returned:
                return imported_dict
            
            else:
                # Both objects are returned:
                return model, imported_dict

    
    elif (action == 'export'):
        
        #Let's export the models or dictionary:
        if (use_colab_memory == True):
            
            from google.colab import files
            # google.colab library must be imported only in case 
            # it is going to be used, for avoiding 
            # AWS compatibility issues.
            
            print("The files will be downloaded to your computer.")
        
        if (bool_check1 == True):
            #manipulate a dictionary
            if (use_colab_memory == True):
                ## Download the dictionary
                key = dictionary_or_list_file_name + "." + dict_extension
                
                with open(key, 'wb') as opened_file:
            
                    pickle.dump(dict_or_list_to_export, opened_file)
                
                # this functionality requires the previous declaration:
                ## from google.colab import files
                files.download(key)
                
                print(f"Dictionary or list {key} successfully downloaded from Colab environment.")
            
            else:
                #standard method 
                with open(dict_path, 'wb') as opened_file:
            
                    pickle.dump(dict_or_list_to_export, opened_file)
                
                #to save the file, the mode must be set as 'wb' (write binary)
                print(f"Dictionary or list successfully exported as {dict_path}.")
                
        if (bool_check2 == True):
            #manipulate a model
            # select the proper model
        
            if (model_type == 'keras'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save(key)
                    files.download(key)
                    print(f"Keras/TensorFlow model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save(model_path)
                    print(f"Keras/TensorFlow model successfully exported as {model_path}.")
            
            elif (model_type == 'tensorflow_general'):
                
                # Save your model in the SavedModel format
                # Save as a directory named 'saved_model'
                model_to_export.save('saved_model')
                model_path = 'saved_model'
            
                try:
                    model_path = model_path + ".tar.gz"
                    
                    # Open the context manager
                    with tarfile.open (model_path, 'w:gz') as compressed_model:
                        #Add the folder:
                        compressed_model.add('saved_model/')    
                        # if you were not using the context manager, it would be necessary to apply
                        # close method: tar = tarfile.open(fname, "r:gz"); tar.extractall(); tar.close()
                
                except:
                    # try compressing as tar:
                    try:
                        model_path = model_path + ".tar"
                        # Open the context manager
                        with tarfile.open (model_path, 'w:') as compressed_model:
                            #Add the folder:
                            compressed_model.add('saved_model/') 
                    
                    except:
                        # compress as zip:
                        model_path = model_path + ".zip"
                        with ZipFile (model_path, 'w') as compressed_model:
                            compressed_model.write('saved_model/')
                
                if (use_colab_memory == True):
                    
                    key = model_path
                    files.download(key)
                    print(f"TensorFlow model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    print(f"TensorFlow model successfully exported as {model_path}.")

            elif (model_type == 'sklearn'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    
                    with open(key, 'wb') as opened_file:

                        dill.dump(model_to_export, opened_file)
                    
                    #to save the file, the mode must be set as 'wb' (write binary)
                    files.download(key)
                    print(f"Scikit-learn model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    with open(model_path, 'wb') as opened_file:

                        dill.dump(model_to_export, opened_file)
                    
                    print(f"Scikit-learn model successfully exported as {model_path}.")
                    # For exporting a pickle model:
                    ## pkl.dump(model_to_export, open(model_path, 'wb'))
            
            elif ((model_type == 'xgb_regressor')|(model_type == 'xgb_classifier')):
                # In both cases, the XGBoost object is already loaded in global
                # context memory. So there is already the object for using the
                # save_model method, available for both classes (XGBRegressor and
                # XGBClassifier).
                # We can simply check if it is one type OR the other, since the
                # method is the same:
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save_model(key)
                    files.download(key)
                    print(f"XGBoost model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save_model(model_path)
                    print(f"XGBoost model successfully exported as {model_path}.")
                    # For exporting a pickle model:
                    ## pkl.dump(model_to_export, open(model_path, 'wb'))
            
            elif (model_type == 'arima'):
                
                if (use_colab_memory == True):
                    ## Download the model
                    key = model_file_name + "." + model_extension
                    model_to_export.save(key)
                    files.download(key)
                    print(f"ARIMA model: {key} successfully downloaded from Colab environment.")
            
                else:
                    #standard method
                    model_to_export.save(model_path)
                    print(f"ARIMA model successfully exported as {model_path}.")
        
        print("Export of files completed.")
    
    else:
        print("Enter a valid action, import or export.")

# **Function for downloading a file from Google Colab to the local machine; or uploading a file from the machine to Colab's instant memory**

In [ ]:
def upload_to_or_download_file_from_colab (action = 'download', file_to_download_from_colab = None):
    
    # action = 'download' to download the file to the local machine
    # action = 'upload' to upload a file from local machine to
    # Google Colab's instant memory
    
    # file_to_download_from_colab = None. This parameter is obbligatory when
    # action = 'download'. 
    # Declare as file_to_download_from_colab the file that you want to download, with
    # the correspondent extension.
    # It should not be declared in quotes.
    # e.g. to download a dictionary named dict, object_to_download_from_colab = 'dict.pkl'
    # To download a dataframe named df, declare object_to_download_from_colab = 'df.csv'
    # To export a model named keras_model, declare object_to_download_from_colab = 'keras_model.h5'
 
    from google.colab import files
    # google.colab library must be imported only in case 
    # it is going to be used, for avoiding 
    # AWS compatibility issues.
        
    if (action == 'upload'):
            
        print("Click on the button for file selection and select the files from your machine that will be uploaded in the Colab environment.")
        print("Warning: the files will be removed from Colab memory after the Kernel dies or after the notebook is closed.")
        # this functionality requires the previous declaration:
        ## from google.colab import files
            
        colab_files_dict = files.upload()
            
        # The files are stored into a dictionary called colab_files_dict where the keys
        # are the names of the files and the values are the files themselves.
        ## e.g. if you upload a single file named "dictionary.pkl", the dictionary will be
        ## colab_files_dict = {'dictionary.pkl': file}, where file is actually a big string
        ## representing the contents of the file. The length of this value is the size of the
        ## uploaded file, in bytes.
        ## To access the file is like accessing a value from a dictionary: 
        ## d = {'key1': 'val1'}, d['key1'] == 'val1'
        ## we simply declare the key inside brackets and quotes, the same way we would do for
        ## accessing the column of a dataframe.
        ## In this example, colab_files_dict['dictionary.pkl'] access the content of the 
        ## .pkl file, and len(colab_files_dict['dictionary.pkl']) is the size of the .pkl
        ## file in bytes.
        ## To check the dictionary keys, apply the method .keys() to the dictionary (with empty
        ## parentheses): colab_files_dict.keys()
            
        for key in colab_files_dict.keys():
            #loop through each element of the list of keys of the dictionary
            # (list colab_files_dict.keys()). Each element is named 'key'
            print(f"User uploaded file {key} with length {len(colab_files_dict[key])} bytes.")
            # The key is the name of the file, and the length of the value
            ## correspondent to the key is the file's size in bytes.
            ## Notice that the content of the uploaded object must be passed 
            ## as argument for a proper function to be interpreted. 
            ## For instance, the content of a xlsx file should be passed as
            ## argument for Pandas .read_excel function; the pkl file must be passed as
            ## argument for pickle.
            ## e.g., if you uploaded 'table.xlsx' and stored it into colab_files_dict you should
            ## declare df = pd.read_excel(colab_files_dict['table.xlsx']) to obtain a dataframe
            ## df from the uploaded table. Notice that is the value, not the key, that is the
            ## argument.
                
            print("The uploaded files are stored into a dictionary object named as colab_files_dict.")
            print("Each key from this dictionary is the name of an uploaded file. The value correspondent to that key is the file itself.")
            print("The structure of a general Python dictionary is dict = {\'key1\': value1}. To access value1, declare file = dict[\'key1\'], as if you were accessing a column from a dataframe.")
            print("Then, if you uploaded a file named \'table.xlsx\', you can access this file as:")
            print("uploaded_file = colab_files_dict[\'table.xlsx\']")
            print("Notice, though, that the object uploaded_file is the whole file content, not a Python object already converted. To convert to a Python object, pass this element as argument for a proper function or method.")
            print("In this example, to convert the object uploaded_file to a dataframe, Pandas pd.read_excel function could be used. In the following line, a df dataframe object is obtained from the uploaded file:")
            print("df = pd.read_excel(uploaded_file)")
            print("Also, the uploaded file itself will be available in the Colaboratory Notebook\'s workspace.")
            
            return colab_files_dict
        
    elif (action == 'download'):
            
        if (file_to_download_from_colab is None):
                
            #No object was declared
            print("Please, inform a file to download from the notebook\'s workspace. It should be declared in quotes and with the extension: e.g. \'table.csv\'.")
            
        else:
                
            print("The file will be downloaded to your computer.")

            files.download(file_to_download_from_colab)

            print(f"File {file_to_download_from_colab} successfully downloaded from Colab environment.")

    else:
            
            print("Please, select a valid action, \'download\' or \'upload\'.")

# **Function for exporting a list of files from notebook's workspace to AWS Simple Storage Service (S3)**

In [262]:
def export_files_to_s3 (list_of_file_names_with_extensions, directory_of_notebook_workspace_storing_files_to_export = None, s3_bucket_name = None, s3_obj_prefix = None):
    
    import os
    import boto3
    # boto3 is AWS S3 Python SDK
    # sagemaker and boto3 libraries must be imported only in case 
    # they are going to be used, for avoiding 
    # Google Colab compatibility issues.
    from getpass import getpass
    
    # list_of_file_names_with_extensions: list containing all the files to export to S3.
    # Declare it as a list even if only a single file will be exported.
    # It must be a list of strings containing the file names followed by the extensions.
    # Example, to a export a single file my_file.ext, where my_file is the name and ext is the
    # extension:
    # list_of_file_names_with_extensions = ['my_file.ext']
    # To export 3 files, file1.ext1, file2.ext2, and file3.ext3:
    # list_of_file_names_with_extensions = ['file1.ext1', 'file2.ext2', 'file3.ext3']
    # Other examples:
    # list_of_file_names_with_extensions = ['Screen_Shot.png', 'dataset.csv']
    # list_of_file_names_with_extensions = ["dictionary.pkl", "model.h5"]
    # list_of_file_names_with_extensions = ['doc.pdf', 'model.dill']
    
    # directory_of_notebook_workspace_storing_files_to_export: directory from notebook's workspace
    # from which the files will be exported to S3. Keep it None, or
    # directory_of_notebook_workspace_storing_files_to_export = "/"; or
    # directory_of_notebook_workspace_storing_files_to_export = '' (empty string) to export from
    # the root (main) directory.
    # Alternatively, set as a string containing only the directories and folders, not the file names.
    # Examples: directory_of_notebook_workspace_storing_files_to_export = 'folder1';
    # directory_of_notebook_workspace_storing_files_to_export = 'folder1/folder2/'
    
    # For this function, all exported files must be located in the same directory.
    
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # s3_obj_prefix = None. Keep it None or as an empty string (s3_obj_key_prefix = '')
    # to import the whole bucket content, instead of a single object from it.
    # Alternatively, set it as a string containing the subfolder from the bucket to import:
    # Suppose that your bucket (admin-created) has four objects with the following object 
    # keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
    # s3-dg.pdf. The s3-dg.pdf key does not have a prefix, so its object appears directly 
    # at the root level of the bucket. If you open the Development/ folder, you see 
    # the Projects.xlsx object in it.
    # Check Amazon documentation:
    # https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-keys.html
    
    # In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
    # where 'bucket' is the bucket's name, key_prefix = 'my_path/.../', without the
    # 'file.csv' (file name with extension) last part.
    
    # So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
    # a given folder (directory) of the bucket.
    # DO NOT PUT A SLASH before (to the right of) the prefix;
    # DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

    # Alternatively, provide the full path of a given file if you want to import only it:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
    # where my_file is the file's name, and ext is its extension.


    # Attention: after running this function for connecting with AWS Simple Storage System (S3), 
    # your 'AWS Access key ID' and your 'Secret access key' will be requested.
    # The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
    # other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
    # and the prefix. All of these are sensitive information from the organization.
    # Therefore, after importing the information, always remember of cleaning the output of this cell
    # and of removing such information from the strings.
    # Remember that these data may contain privilege for accessing the information, so it should not
    # be used for non-authorized people.

    # Also, remember of deleting the exported from the workspace after finishing the analysis.
    # The costs for storing the files in S3 is quite inferior than those for storing directly in the
    # workspace. Also, files stored in S3 may be accessed for other users than those with access to
    # the notebook's workspace.
    
    
    # Check if directory_of_notebook_workspace_storing_files_to_export is None. 
    # If it is, make it the root directory:
    if ((directory_of_notebook_workspace_storing_files_to_export is None)|(str(directory_of_notebook_workspace_storing_files_to_export) == "/")):
            
            # For the S3 buckets, the path should not start with slash. Assign the empty
            # string instead:
            directory_of_notebook_workspace_storing_files_to_export = ""
            print("The files will be exported from the notebook\'s root directory to S3.")
    
    elif (str(directory_of_notebook_workspace_storing_files_to_export) == ""):
        
            # Guarantee that the path is the empty string.
            # Avoid accessing the else condition, what would raise an error
            # since the empty string has no character of index 0
            directory_of_notebook_workspace_storing_files_to_export = str(directory_of_notebook_workspace_storing_files_to_export)
            print("The files will be exported from the notebook\'s root directory to S3.")
          
    else:
        # Use the str attribute to guarantee that the path was read as a string:
        directory_of_notebook_workspace_storing_files_to_export = str(directory_of_notebook_workspace_storing_files_to_export)
            
        if(directory_of_notebook_workspace_storing_files_to_export[0] == "/"):
            # the first character is the slash. Let's remove it

            # In AWS, neither the prefix nor the path to which the file will be imported
            # (file from S3 to workspace) or from which the file will be exported to S3
            # (the path in the notebook's workspace) may start with slash, or the operation
            # will not be concluded. Then, we have to remove this character if it is present.

            # The slash is character 0. Then, we want all characters from character 1 (the
            # second) to character len(str(path_to_store_imported_s3_bucket)) - 1, the index
            # of the last character. So, we can slice the string from position 1 to position
            # the slicing syntax is: string[1:] - all string characters from character 1
            # string[:10] - all string characters from character 10-1 = 9 (including 9); or
            # string[1:10] - characters from 1 to 9
            # So, slice the whole string, starting from character 1:
            directory_of_notebook_workspace_storing_files_to_export = directory_of_notebook_workspace_storing_files_to_export[1:]
            # attention: even though strings may be seem as list of characters, that can be
            # sliced, we cannot neither simply assign a character to a given position nor delete
            # a character from a position.

    # Ask the user to provide the credentials:
    ACCESS_KEY = input("Enter your AWS Access Key ID here (in the right). It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
    print("\n") # line break
    SECRET_KEY = getpass("Enter your password (Secret key) here (in the right). It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
        
    # The use of 'getpass' instead of 'input' hide the password behind dots.
    # So, the password is not visible by other users and cannot be copied.
        
    print("\n")
    print("WARNING: The bucket\'s name, the prefix, the AWS access key ID, and the AWS Secret access key are all sensitive information, which may grant access to protected information from the organization.\n")
    print("After finish exporting data to S3, remember of removing these information from the notebook, specially if it is going to be shared. Also, remember of removing the files from the workspace.\n")
    print("The cost for storing files in Simple Storage Service is quite inferior than the one for storing directly in SageMaker workspace. Also, files stored in S3 may be accessed for other users than those with access the notebook\'s workspace.\n")

    # Check if the user actually provided the mandatory inputs, instead
    # of putting None or empty string:
    if ((ACCESS_KEY is None) | (ACCESS_KEY == '')):
        print("AWS Access Key ID is missing. It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
        return "error"
    elif ((SECRET_KEY is None) | (SECRET_KEY == '')):
        print("AWS Secret Access Key is missing. It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
        return "error"
    elif ((s3_bucket_name is None) | (s3_bucket_name == '')):
        print ("Please, enter a valid S3 Bucket\'s name. Do not add sub-directories or folders (prefixes), only the name of the bucket itself.")
        return "error"
    
    else:
        # Use the str attribute to guarantee that all AWS parameters were properly read as strings, and not as
        # other variables (like integers or floats):
        ACCESS_KEY = str(ACCESS_KEY)
        SECRET_KEY = str(SECRET_KEY)
        s3_bucket_name = str(s3_bucket_name)

    if(s3_bucket_name[0] == "/"):
        # the first character is the slash. Let's remove it

        # In AWS, neither the prefix nor the path to which the file will be imported
        # (file from S3 to workspace) or from which the file will be exported to S3
        # (the path in the notebook's workspace) may start with slash, or the operation
        # will not be concluded. Then, we have to remove this character if it is present.

        # So, slice the whole string, starting from character 1 (as did for 
        # path_to_store_imported_s3_bucket):
        s3_bucket_name = s3_bucket_name[1:]

    # Remove any possible trailing (white and tab spaces) spaces
    # That may be present in the string. Use the Python string
    # rstrip method, which is the equivalent to the Trim function:
    # When no arguments are provided, the whitespaces and tabulations
    # are the removed characters
    # https://www.w3schools.com/python/ref_string_rstrip.asp?msclkid=ee2d05c3c56811ecb1d2189d9f803f65
    s3_bucket_name = s3_bucket_name.rstrip()
    ACCESS_KEY = ACCESS_KEY.rstrip()
    SECRET_KEY = SECRET_KEY.rstrip()
    # Since the user manually inputs the parameters ACCESS and SECRET_KEY,
    # it is easy to input whitespaces without noticing that.

    # Now process the non-obbligatory parameter.
    # Check if a prefix was passed as input parameter. If so, we must select only the names that start with
    # The prefix.
    # Example: in the bucket 'my_bucket' we have a directory 'dir1'.
    # In the main (root) directory, we have a file 'file1.json' like: '/file1.json'
    # If we pass the prefix 'dir1', we want only the files that start as '/dir1/'
    # such as: 'dir1/file2.json', excluding the file in the main (root) directory and excluding the files in other
    # directories. Also, we want to eliminate the file names with no extensions, like 'dir1/' or 'dir1/dir2',
    # since these object names represent folders or directories, not files.	

    if (s3_obj_prefix is None):
        print ("No prefix, specific object, or subdirectory provided.") 
        print (f"Then, exporting to \'{s3_bucket_name}\' root (main) directory.\n")
        # s3_path: path that the file should have in S3:
        s3_path = "" # empty string for the root directory
    elif ((s3_obj_prefix == "/") | (s3_obj_prefix == '')):
        # The root directory in the bucket must not be specified starting with the slash
        # If the root "/" or the empty string '' is provided, make
        # it equivalent to None (no directory)
        print ("No prefix, specific object, or subdirectory provided.") 
        print (f"Then, exporting to \'{s3_bucket_name}\' root (main) directory.\n")
        # s3_path: path that the file should have in S3:
        s3_path = "" # empty string for the root directory
    
    else:
        # Since there is a prefix, use the str attribute to guarantee that the path was read as a string:
        s3_obj_prefix = str(s3_obj_prefix)
            
        if(s3_obj_prefix[0] == "/"):
            # the first character is the slash. Let's remove it

            # In AWS, neither the prefix nor the path to which the file will be imported
            # (file from S3 to workspace) or from which the file will be exported to S3
            # (the path in the notebook's workspace) may start with slash, or the operation
            # will not be concluded. Then, we have to remove this character if it is present.

            # So, slice the whole string, starting from character 1 (as did for 
            # path_to_store_imported_s3_bucket):
            s3_obj_prefix = s3_obj_prefix[1:]

        # Remove any possible trailing (white and tab spaces) spaces
        # That may be present in the string. Use the Python string
        # rstrip method, which is the equivalent to the Trim function:
        s3_obj_prefix = s3_obj_prefix.rstrip()
            
        # s3_path: path that the file should have in S3:
        # Make the path the prefix itself, since there is a prefix:
        s3_path = s3_obj_prefix
            
        print("AWS Access Credentials, and bucket\'s prefix, object or subdirectory provided.\n")	

            
        print ("Starting connection with the S3 bucket.\n")
        
        try:
            # Start S3 client as the object 's3_client'
            s3_client = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = SECRET_KEY)
        
            print(f"Credentials accepted by AWS. S3 client successfully started.\n")
            # An object 'data_table.xlsx' in the main (root) directory of the s3_bucket is stored in Python environment as:
            # s3.ObjectSummary(bucket_name='bucket_name', key='data_table.xlsx')
            # The name of each object is stored as the attribute 'key' of the object.
        
        except:
            
            print("Failed to connect to AWS Simple Storage Service (S3). Review if your credentials are correct.")
            print("The variable \'access_key\' must be set as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("The variable \'secret_key\' must be set as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
        
        
        try:
            # Connect to the bucket specified as 'bucket_name'.
            # The bucket is started as the object 's3_bucket':
            s3_bucket = s3_client.Bucket(s3_bucket_name)
            print(f"Connection with bucket \'{s3_bucket_name}\' stablished.\n")
            
        except:
            
            print("Failed to connect with the bucket, which usually happens when declaring a wrong bucket\'s name.") 
            print("Check the spelling of your bucket_name string and remember that it must be all in lower-case.\n")
                
        # Now, let's obtain the lists of all file paths in the notebook's workspace and
        # of the paths that the files should have in S3, after being exported.
        
        try:
            
            # start the lists:
            workspace_full_paths = []
            s3_full_paths = []
            
            # Get the total of files in list_of_file_names_with_extensions:
            total_of_files = len(list_of_file_names_with_extensions)
            
            # And Loop through all elements, named 'my_file' from the list
            for my_file in list_of_file_names_with_extensions:
                
                # Get the full path in the notebook's workspace:
                workspace_file_full_path = os.path.join(directory_of_notebook_workspace_storing_files_to_export, my_file)
                # Get the full path that the file will have in S3:
                s3_file_full_path = os.path.join(s3_path, my_file)
                
                # Append these paths to the correspondent lists:
                workspace_full_paths.append(workspace_file_full_path)
                s3_full_paths.append(s3_file_full_path)
                
            # Now, both lists have the same number of elements. For an element (file) i,
            # workspace_full_paths has the full file path in notebook's workspace, and
            # s3_full_paths has the path that the new file should have in S3 bucket.
        
        except:
            
            print("The function returned an error when trying to access the list of files. Declare it as a list of strings, even if there is a single element in the list.")
            print("Example: list_of_file_names_with_extensions = [\'my_file.ext\']\n")
            return "error"
        
        
        # Now, loop through all elements i from the lists.
        # The first elements of the lists have index 0; the last elements have index
        # total_of_files - 1, since there are 'total_of_files' elements:
        
        # Then, export the correspondent element to S3:
        
        try:
            
            for i in range(total_of_files):
                # goes from i = 0 to i = total_of_files - 1

                # get the element from list workspace_file_full_path 
                # (original path of file i, from which it will be exported):
                PATH_IN_WORKSPACE = workspace_full_paths[i]

                # get the correspondent element of list s3_full_paths
                # (path that the file i should have in S3, after being exported):
                S3_FILE_PATH = s3_full_paths[i]

                # Start the new object in the bucket previously started as 's3_bucket'.
                # Start it with the specified prefix, in S3_FILE_PATH:
                new_s3_object = s3_bucket.Object(S3_FILE_PATH)
                
                # Finally, upload the file in PATH_IN_WORKSPACE.
                # Make new_s3_object the exported file:
            
                # Upload the selected object from the workspace path PATH_IN_WORKSPACE
                # to the S3 path specified as S3_FILE_PATH.
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" exports a xlsx file named 'my_table' to the notebook's main (root)
                # directory
                new_s3_object.upload_file(Filename = PATH_IN_WORKSPACE)

                print(f"The file \'{list_of_file_names_with_extensions[i]}\' was successfully exported from notebook\'s workspace to AWS Simple Storage Service (S3).\n")

                
            print("Finished exporting the files from the the notebook\'s workspace to S3 bucket. It may take a couple of minutes untill they be shown in S3 environment.\n") 
            print("Do not forget to delete these copies after finishing the analysis. They will remain stored in the bucket.\n")


        except:

            # Run this code for any other exception that may happen (no exception error
            # specified, so any exception runs the following code).
            # Check: https://pythonbasics.org/try-except/?msclkid=4f6b4540c5d011ecb1fe8a4566f632a6
            # for seeing how to handle successive exceptions

            print("Attention! The function raised an exception error, which is probably due to the AWS Simple Storage Service (S3) permissions.")
            print("Before running again this function, check this quick guide for configuring the permission roles in AWS.\n")
            print("It is necessary to create an user with full access permissions to interact with S3 from SageMaker. To configure the User, go to the upper ribbon of AWS, click on Services, and select IAM – Identity and Access Management.")
            print("1. In IAM\'s lateral panel, search for \'Users\' in the group of Access Management.")
            print("2. Click on the \'Add users\' button.")
            print("3. Set an user name in the text box \'User name\'.")
            print("Attention: users and S3 buckets cannot be written in upper case. Also, selecting a name already used by an Amazon user or bucket will raise an error message.\n")
            print("4. In the field \'Select type of Access to AWS\'-\'Select type of AWS credentials\' select the option \'Access key - Programmatic access\'. After that, click on the button \'Next: Permissions\'.")
            print("5. In the field \'Set Permissions\', keep the \'Add user to a group\' button marked.")
            print("6. In the field \'Add user to a group\', click on \'Create group\' (alternatively, you can be added to a group already configured or copy the permissions of another user.")
            print("7. In the text box \'Group\'s name\', set a name for the new group of permissions.")
            print("8. In the search bar below (\'Filter politics\'), search for a politics that fill your needs, and check the option button on the left of this politic. The politics \'AmazonS3FullAccess\' grants full access to the S3 content.")
            print("9. Finally, click on \'Create a group\'.")
            print("10. After the group is created, it will appear with a check box marked, over the previous groups. Keep it marked and click on the button \'Next: Tags\'.")
            print("11. Create and note down the Access key ID and Secret access key. You can also download a comma separated values (CSV) file containing the credentials for future use.")
            print("ATTENTION: These parameters are required for accessing the bucket\'s content from any application, including AWS SageMaker.")
            print("12. Click on \'Next: Review\' and review the user credentials information and permissions.")
            print("13. Click on \'Create user\' and click on the download button to download the CSV file containing the user credentials information.")
            print("The headers of the CSV file (the stored fields) is: \'User name, Password, Access key ID, Secret access key, Console login link\'.")
            print("You need both the values indicated as \'Access key ID\' and as \'Secret access key\' to fetch the S3 bucket.")
            print("\n") # line break
            print("After acquiring the necessary user privileges, use the boto3 library to export the file from the notebook’s workspace to the bucket (i.e., to upload a file to the bucket).")
            print("For exporting the file as a new bucket\'s file use the following code:\n")
            print("1. Set a variable \'access_key\' as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("2. Set a variable \'secret_key\' as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
            print("3. Set a variable \'bucket_name\' as a string containing only the name of the bucket. Do not add subdirectories, folders (prefixes), or file names.")
            print("Example: if your bucket is named \'my_bucket\' and its main directory contains folders like \'folder1\', \'folder2\', etc, do not declare bucket_name = \'my_bucket/folder1\', even if you only want files from folder1.")
            print("ALWAYS declare only the bucket\'s name: bucket_name = \'my_bucket\'.")
            print("4. Set a variable \'file_path_in_workspace\' containing the path of the file in notebook’s workspace. The file will be exported from “file_path_in_workspace” to the S3 bucket.")
            print("If the file is stored in the notebook\'s root (main) directory: file_path = \"my_file.ext\".")
            print("If the path of the file in the notebook workspace is: \'dir1/…/dirN/my_file.ext\', where dirN is the N-th subdirectory, and dir1 is a folder or directory of the main (root) bucket\'s directory: file_path = \"dir1/…/dirN/my_file.ext\".")
            print("5. Set a variable named \'file_path_in_s3\' containing the path from the bucket’s subdirectories to the file you want to fetch. Include the file name and its extension.")
            print("6. Finally, declare the following code, which refers to the defined variables:\n")

            # Let's use triple quotes to declare a formated string
            example_code = """
                import boto3
                # Start S3 client as the object 's3_client'
                s3_client = boto3.resource('s3', aws_access_key_id = access_key, aws_secret_access_key = secret_key)
                # Connect to the bucket specified as 'bucket_name'.
                # The bucket is started as the object 's3_bucket':
                s3_bucket = s3_client.Bucket(bucket_name)
                # Start the new object in the bucket previously started as 's3_bucket'.
                # Start it with the specified prefix, in file_path_in_s3:
                new_s3_object = s3_bucket.Object(file_path_in_s3)
                # Finally, upload the file in file_path_in_workspace.
                # Make new_s3_object the exported file:
                # Upload the selected object from the workspace path file_path_in_workspace
                # to the S3 path specified as file_path_in_s3.
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" exports a xlsx file named 'my_table' to 
                # the notebook's main (root) directory.
                new_s3_object.upload_file(Filename = file_path_in_workspace)
                """

            print(example_code)

            print("An object \'my_file.ext\' in the main (root) directory of the s3_bucket is stored in Python environment as:")
            print("""s3.ObjectSummary(bucket_name='bucket_name', key='my_file.ext'""") 
            # triple quotes to keep the internal quotes without using too much backslashes "\" (the ignore next character)
            print("Then, the name of each object is stored as the attribute \'key\' of the object. To view all objects, we can loop through their \'key\' attributes:\n")
            example_code = """
                # Loop through all objects of the bucket:
                for stored_obj in s3_bucket.objects.all():		
                    # Loop through all elements 'stored_obj' from s3_bucket.objects.all()
                    # Which stores the ObjectSummary for all objects in the bucket s3_bucket:
                    # Print the object’s names:
                    print(stored_obj.key)
                    """

            print(example_code)

## **Call the functions**

### **Mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [ ]:
SOURCE = 'aws'
# SOURCE = 'google' for mounting the google drive;
# SOURCE = 'aws' for accessing an AWS S3 bucket

## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN SOURCE == 'aws':

PATH_TO_STORE_IMPORTED_S3_BUCKET = ''
# PATH_TO_STORE_IMPORTED_S3_BUCKET: path of the Python environment to which the
# S3 bucket contents will be imported. If it is None; or if it is an empty string; or if 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = '/', bucket will be imported to the root path. 
# Alternatively, input the path as a string (in quotes). e.g. 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = 'copied_s3_bucket'

S3_BUCKET_NAME = 'my_bucket'
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

S3_OBJECT_FOLDER_PREFIX = ""
# S3_OBJECT_FOLDER_PREFIX = None. Keep it None; or as an empty string 
# (S3_OBJECT_FOLDER_PREFIX = ''); or as the root "/" to import the 
# whole bucket content, instead of a single object from it.
# Alternatively, set it as a string containing the subfolder from the bucket to import:
# Suppose that your bucket (admin-created) has four objects with the following object 
# keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
# s3-dg.pdf. 
# The s3-dg.pdf key does not have a prefix, so its object appears directly 
# at the root level of the bucket. If you open the Development/ folder, you see 
# the Projects.xlsx object in it.
# In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
# where 'bucket' is the bucket's name, prefix = 'my_path/.../', without the
# 'file.csv' (file name with extension) last part.

# So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
# a given folder (directory) of the bucket.
# DO NOT PUT A SLASH before (to the right of) the prefix;
# DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

# Alternatively, provide the full path of a given file if you want to import only it:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
# where my_file is the file's name, and ext is its extension.


# Attention: after running this function for fetching AWS Simple Storage System (S3), 
# your 'AWS Access key ID' and your 'Secret access key' will be requested.
# The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
# other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
# and the prefix. All of these are sensitive information from the organization.
# Therefore, after importing the information, always remember of cleaning the output of this cell
# and of removing such information from the strings.
# Remember that these data may contain privilege for accessing protected information, 
# so it should not be used for non-authorized people.

# Also, remember of deleting the imported files from the workspace after finishing the analysis.
# The costs for storing the files in S3 is quite inferior than those for storing directly in the
# workspace. Also, files stored in S3 may be accessed for other users than those with access to
# the notebook's workspace.
mount_storage_system (source = SOURCE, path_to_store_imported_s3_bucket = PATH_TO_STORE_IMPORTED_S3_BUCKET, s3_bucket_name = S3_BUCKET_NAME, s3_obj_prefix = S3_OBJECT_FOLDER_PREFIX)

### **Importing the dataset**

In [ ]:
## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, xlsm, xlsb, odf, ods and odt), 
## JSON, txt, or CSV (comma separated values) files.

FILE_DIRECTORY_PATH = ""
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "" 
# or FILE_DIRECTORY_PATH = "folder"

FILE_NAME_WITH_EXTENSION = "dataset.csv"
# FILE_NAME_WITH_EXTENSION - (string, in quotes): input the name of the file with the 
# extension. e.g. FILE_NAME_WITH_EXTENSION = "file.xlsx", or, 
# FILE_NAME_WITH_EXTENSION = "file.csv", "file.txt", or "file.json"
# Again, the extensions may be: xls, xlsx, xlsm, xlsb, odf, ods, odt, json, txt or csv.

LOAD_TXT_FILE_WITH_JSON_FORMAT = False
# LOAD_TXT_FILE_WITH_JSON_FORMAT = False. Set LOAD_TXT_FILE_WITH_JSON_FORMAT = True 
# if you want to read a file with txt extension containing a text formatted as JSON 
# (but not saved as JSON).
# WARNING: if LOAD_TXT_FILE_WITH_JSON_FORMAT = True, all the JSON file parameters of the 
# function (below) must be set. If not, an error message will be raised.

HOW_MISSING_VALUES_ARE_REGISTERED = None
# HOW_MISSING_VALUES_ARE_REGISTERED = None: keep it None if missing values are registered as None,
# empty or np.nan. Pandas automatically converts None to NumPy np.nan objects (floats).
# This parameter manipulates the argument na_values (default: None) from Pandas functions.
# By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, 
#‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, 
# ‘n/a’, ‘nan’, ‘null’.

# If a different denomination is used, indicate it as a string. e.g.
# HOW_MISSING_VALUES_ARE_REGISTERED = '.' will convert all strings '.' to missing values;
# HOW_MISSING_VALUES_ARE_REGISTERED = 0 will convert zeros to missing values.

# If dict passed, specific per-column NA values. For example, if zero is the missing value
# only in column 'numeric_col', you can specify the following dictionary:
# how_missing_values_are_registered = {'numeric-col': 0}

    
HAS_HEADER = True
# HAS_HEADER = True if the the imported table has headers (row with columns names).
# Alternatively, HAS_HEADER = False if the dataframe does not have header.

DECIMAL_SEPARATOR = '.'
# DECIMAL_SEPARATOR = '.' - String. Keep it '.' or None to use the period ('.') as
# the decimal separator. Alternatively, specify here the separator.
# e.g. DECIMAL_SEPARATOR = ',' will set the comma as the separator.
# It manipulates the argument 'decimal' from Pandas functions.

TXT_CSV_COL_SEP = "comma"
# txt_csv_col_sep = "comma" - This parameter has effect only when the file is a 'txt'
# or 'csv'. It informs how the different columns are separated.
# Alternatively, txt_csv_col_sep = "comma", or txt_csv_col_sep = "," 
# for columns separated by comma;
# txt_csv_col_sep = "whitespace", or txt_csv_col_sep = " " 
# for columns separated by simple spaces.
# You can also set a specific separator as string. For example:
# txt_csv_col_sep = '\s+'; or txt_csv_col_sep = '\t' (in this last example, the tabulation
# is used as separator for the columns - '\t' represents the tab character).

## Parameters for loading Excel files:

LOAD_ALL_SHEETS_AT_ONCE = False
# LOAD_ALL_SHEETS_AT_ONCE = False - This parameter has effect only when for Excel files.
# If LOAD_ALL_SHEETS_AT_ONCE = True, the function will return a list of dictionaries, each
# dictionary containing 2 key-value pairs: the first key will be 'sheet', and its
# value will be the name (or number) of the table (sheet). The second key will be 'df',
# and its value will be the pandas dataframe object obtained from that sheet.
# This argument has preference over SHEET_TO_LOAD. If it is True, all sheets will be loaded.
    
SHEET_TO_LOAD = None
# SHEET_TO_LOAD - This parameter has effect only when for Excel files.
# keep SHEET_TO_LOAD = None not to specify a sheet of the file, so that the first sheet
# will be loaded.
# SHEET_TO_LOAD may be an integer or an string (inside quotes). SHEET_TO_LOAD = 0
# loads the first sheet (sheet with index 0); SHEET_TO_LOAD = 1 loads the second sheet
# of the file (index 1); SHEET_TO_LOAD = "Sheet1" loads a sheet named as "Sheet1".
# Declare a number to load the sheet with that index, starting from 0; or declare a
# name to load the sheet with that name.

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFIX_LIST = None
# JSON_METADATA_PREFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFIX_LIST = ['name', 'last']


# The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = load_pandas_dataframe (file_directory_path = FILE_DIRECTORY_PATH, file_name_with_extension = FILE_NAME_WITH_EXTENSION, load_txt_file_with_json_format = LOAD_TXT_FILE_WITH_JSON_FORMAT, how_missing_values_are_registered = HOW_MISSING_VALUES_ARE_REGISTERED, has_header = HAS_HEADER, decimal_separator = DECIMAL_SEPARATOR, txt_csv_col_sep = TXT_CSV_COL_SEP, load_all_sheets_at_once = LOAD_ALL_SHEETS_AT_ONCE, sheet_to_load = SHEET_TO_LOAD, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_prefix_list = JSON_METADATA_PREFIX_LIST)

# OBS: If an Excel file is loaded and LOAD_ALL_SHEETS_AT_ONCE = True, then the object
# dataset will be a list of dictionaries, with 'sheet' as key containing the sheet name; and 'df'
# as key correspondent to the Pandas dataframe. So, to access the 3rd dataframe (index 2, since
# indexing starts from zero): df = dataframe[2]['df'], where dataframe is the list returned.

### **Converting JSON object to dataframe**

In [ ]:
# JSON object in terms of Python structure: list of dictionaries, where each value of a
# dictionary may be a dictionary or a list of dictionaries (nested structures).
# example of highly nested structure saved as a list 'json_formatted_list'. Note that the same
# structure could be declared and stored into a string variable. For instance, if you have a txt
# file containing JSON, you could read the txt and save its content as a string.
# json_formatted_list = [{'field1': val1, 'field2': {'dict_val': dict_val}, 'field3': [{
# 'nest1': nest_val1}, {'nest2': nestval2}]}, {'field1': val1, 'field2': {'dict_val': dict_val}, 
# 'field3': [{'nest1': nest_val1}, {'nest2': nestval2}]}]

JSON_OBJ_TO_CONVERT = json_object #Alternatively: object containing the JSON to be converted

# JSON_OBJ_TO_CONVERT: object containing JSON, or string with JSON content to parse.
# Objects may be: string with JSON formatted text;
# list with nested dictionaries (JSON formatted);
# dictionaries, possibly with nested dictionaries (JSON formatted).

JSON_OBJ_TYPE = 'list'
# JSON_OBJ_TYPE = 'list', in case the object was saved as a list of dictionaries (JSON format)
# JSON_OBJ_TYPE = 'string', in case it was saved as a string (text) containing JSON.

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFIX_LIST = None
# JSON_METADATA_PREFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: [{'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]}]
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFIX_LIST = ['name', 'last']


# The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = json_obj_to_pandas_dataframe (json_obj_to_convert = JSON_OBJ_TO_CONVERT, json_obj_type = JSON_OBJ_TYPE, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_prefix_list = JSON_METADATA_PREFIX_LIST)

### **Obtaining Statistical Process Control (SPC) charts**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

TIMESTAMP_TAG_COLUMN = None
# TIMESTAMP_TAG_COLUMN: name (header) of the column containing the timestamps or the numeric scale
# used to represent time (column with floats or integers). The column name may be a string or a number.
# e.g. TIMESTAMP_TAG_COLUMN = 'date' will use the values from column 'date'
# Keep TIMESTAMP_TAG_COLUMN = None if the dataframe do not contain timestamps, so the index will be
# used.

COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'analyzed_column'
# COLUMN_WITH_VARIABLE_TO_BE_ANALYZED: name (header) of the column containing the variable
# which stability will be analyzed by the control chart. The column name may be a string or a number.
# Example: COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'analyzed_column' will analyze a column named
# "analyzed_column", whereas COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'col1' will evaluate column 'col1'.

COLUMN_WITH_LABELS_OR_SUBGROUPS = None
# COLUMN_WITH_LABELS_OR_SUBGROUPS: name (header) of the column containing the variable
# indicating the subgroups or label indication which will be used for grouping the samples. 
# Example: Suppose you want to analyze the means for 4 different subgroups: 1, 2, 3, 4. For each subgroup,
# 4 or 5 samples of data (the values in COLUMN_WITH_VARIABLE_TO_BE_ANALYZED) are collected, and you
# want the average and standard deviation within the subgroups. Them, you create a column named
# 'label' and store the values: 1 for samples correspondent to subgroup 1; 2 for samples from
# subgroup 2,... etc. In this case, COLUMN_WITH_LABELS_OR_SUBGROUPS = 'label'
# Notice that the samples do not need to be collected in order. The function will automatically separate
# the entries according to the subgroups. So, the entries in the dataset may be in an arbitrary order
# like: 1, 1, 2, 1, 4, 3, etc.
# The values in the COLUMN_WITH_LABELS_OR_SUBGROUPS may be strings (text) or numeric values (like
# integers), but different values will be interpreted as different subgroups.
# As an example of text, you could have a column named 'col1' with group identifications as: 
# 'A', 'B', 'C', 'D', and COLUMN_WITH_LABELS_OR_SUBGROUPS = 'col1'.
# Notice the difference between COLUMN_WITH_LABELS_OR_SUBGROUPS and COLUMN_WITH_VARIABLE_TO_BE_ANALYZED:
# COLUMN_WITH_VARIABLE_TO_BE_ANALYZED accepts only numeric values, so the binary variables must be
# converted to integers 0 and 1 before the analysis. The COLUMN_WITH_LABELS_OR_SUBGROUPS, in turns,
# accept both numeric and text (string) values.

COLUMN_WITH_EVENT_FRAME_INDICATION = None
# COLUMN_WITH_EVENT_FRAME_INDICATION: name (header) of the column containing the variable
# indicating the stages, time windows, or event frames. The central line and the limits of natural
# variation will be independently calculated for each event frame. The indication of an event frame
# may be textual (string) or numeric. 
# For example: suppose you have a column named 'event_frames'. For half of the entries, event_frame = 
# 'A'; and for the second half, event_frame = 'B'. If COLUMN_WITH_EVENT_FRAME_INDICATION = 'event_frame',
# the dataframe will be divided into two subsets: 'A', and 'B'. For each subset, the central lines
# and the limits of natural variation will be calculated independently. So, you can check if there is
# modification of the average value and of the dispersion when the time window is modified. It could
# reflect, for example, the use of different operational parameters on each event frame.
# Other possibilities of event indications: 0, 1, 2, 3, ... (sequence of integers); 'A', 'B', 'C', etc;
# 'stage1', 'stage2', ..., 'treatment1', 'treatment2',....; 'frame0', 'frame1', 'frame2', etc.
# ATTENTION: Do not identify different frames with the same value. For example, if
# COLUMN_WITH_EVENT_FRAME_INDICATION has missing values for the first values, then a sequence of rows
# is identified as number 0; followed by a sequence of missing values. In this case, the two windows
# with missing values would be merged as a single window, and the mean and variation would be
# obtained for this merged subset. Then, always specify different windows with different values.
# Other example: COLUMN_WITH_EVENT_FRAME_INDICATION = 'col1' will search for event frame indications
# in column 'col1'.

SPECIFICATION_LIMITS = {'lower_spec_lim': None, 
                        'upper_spec_lim': None}

# SPECIFICATION_LIMITS = {'lower_spec_lim': None, 'upper_spec_lim': None}
# If there are specification limits, input them in this dictionary. Do not modify the keys,
# simply substitute None by the lower and/or the upper specification.
# e.g. Suppose you have a tank that cannot have more than 10 L. So:
# SPECIFICATION_LIMITS = {'lower_spec_lim': None, 'upper_spec_lim': 10}, there is only
# an upper specification equals to 10 (do not add units);
# Suppose a temperature cannot be lower than 10 ºC, but there is no upper specification. So,
# SPECIFICATION_LIMITS = {'lower_spec_lim': 10, 'upper_spec_lim': None}. Finally, suppose
# a liquid which pH must be between 6.8 and 7.2:
# SPECIFICATION_LIMITS = {'lower_spec_lim': 6.8, 'upper_spec_lim': 7.2}

REFERENCE_VALUE = None
# REFERENCE_VALUE: keep it as None or add a float value.
# This reference value will be shown as a red constant line to be compared
# with the plots. e.g. REFERENCE_VALUE = 1.0 will plot a line passing through
# VARIABLE_TO_ANALYZE = 1.0

USE_SPC_CHART_ASSISTANT = False
# USE_SPC_CHART_ASSISTANT = False. Set USE_SPC_CHART_ASSISTANT = True to open the 
# visual flow chart assistant that will help you select the appropriate parameters; 
# as well as passing the data in the correct format. If the assistant is open, many of the 
# arguments of the function will be filled when using it.

CHART_TO_USE = 'std_error'
# The type of chart that will be obtained, as well as the methodology used for estimating the
# natural variation of the process. Notice that it may be strongly dependent on the statistical
# distribution. So, if you are not sure about the distribution, or simply want to apply a more
# general (less restrictive) methodology, set:

# CHART_TO_USE = '3s_as_natural_variation' to estimate the natural variation as 3 times the
# standard deviation (s); or
# CHART_TO_USE = 'std_error' for estimating it as 3 times the standard error, where 
# standard error = s/(n**0.5) = s/sqrt(n), n = number of samples (that may be the number of 
# individual data samples collected, or the number of subgroups or labels); sqrt is the square root.

# CHART_TO_USE = '3s_as_natural_variation' and CHART_TO_USE = 'std_error' are the only ones available
# for both individual and grouped data.

## ATTENTION: Do not group the variables before using the function. It will perform the automatic
# grouping in accordance to the values specified as COLUMN_WITH_LABELS_OR_SUBGROUPS.

# Other values allowed for CHART_TO_USE:

# CHART_TO_USE = 'i_mr', for individual values of a numeric (continuous) variable which follows the
# NORMAL distribution.

# CHART_TO_USE = 'xbar_s', for grouped values (by mean) of a numeric variable, where the mean values
# of labels or subgroups follow a NORMAL distribution.

# CHART_TO_USE = 'np', for grouped binary variables (allowed values: 0 or 1). This is the
# control chart for proportion of defectives. - Original data must follow the BINOMIAL distribution.

# CHART_TO_USE = 'p', for grouped binary variables (allowed values: 0 or 1). This is the
# control chart for count of defectives. - Original data must follow the BINOMIAL distribution.

# Attention: an error will be raised if CHART_TO_USE = 'np' or 'p' and the variable was not converted
# to a numeric binary, with values 0 or 1. This function do not perform the automatic ordinal or
# One-Hot encoding of the categorical features.

# CHART_TO_USE = 'u', for counts of occurrences per unit. - Original data must follow the POISSON
# distribution (special case of the gamma distribution).

# CHART_TO_USE = 'c', for average occurrence per unit. - Original data must follow the POISSON
# distribution (special case of the gamma distribution).

# CHARTS FOR ANALYZING RARE EVENTS
# CHART_TO_USE = 'g', for analyzing count of events between successive rare events occurrences 
# (data follow the GEOMETRIC distribution).
# CHART_TO_USE = 't', for analyzing time interval between successive rare events occurrences.

CONSIDER_SKEWED_DISTRIBUTION_WHEN_ESTIMATING_STD = False
# Whether the distribution of data to be analyzed present high skewness or kurtosis.
# If CONSIDER_SKEWED_DISTRIBUTION_WHEN_ESTIMATING_STD = False, the central lines will be estimated
# as the mean values of the analyzed variable. 
# If CONSIDER_SKEWED_DISTRIBUTION_WHEN_ESTIMATING_STD = True, the central lines will be estimated 
# as the median of the analyzed variable, which is a better alternative for skewed data such as the 
# ones that follow geometric or lognormal distributions (median = mean × 0.693).
# Notice that this argument has effect only when CHART_TO_USE = '3s_as_natural_variation' or 
# CHART_TO_USE = 'std_error'.

RARE_EVENT_INDICATION = None
# RARE_EVENT_INDICATION = None. String (in quotes), float or integer. If you want to analyze a
# rare event through 'g' or 't' control charts, this parameter is obbligatory. Also, notice that:
# COLUMN_WITH_VARIABLE_TO_BE_ANALYZED must be the column which contains an indication of the rare
# event occurrence, and the RARE_EVENT_INDICATION is the value of the column COLUMN_WITH_VARIABLE_TO_BE_ANALYZED
# when a rare event takes place.
# For instance, suppose RARE_EVENT_INDICATION = 'shutdown'. It means that column COLUMN_WITH_VARIABLE_TO_BE_ANALYZED
# has the value 'shutdown' when the rare event occurs, i.e., for timestamps when the system
# stopped. Other possibilities are RARE_EVENT_INDICATION = 0, or RARE_EVENT_INDICATION = -1,
# indicating that when COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 0 (or -1), we know that
# a rare event occurred. The most important thing here is that the value given to the rare event
# should be assigned ONLY to the rare events.

# You do not need to assign values for the other timestamps when no rare event took place. But it is
# important to keep all timestamps in the dataframe. That is because the rare events charts will
# compare the rare event occurrence against all other events and timestamps.
# If you are not analyzing rare events with 'g' or 't' charts, keep RARE_EVENT_INDICATION = None.

RARE_EVENT_TIMEDELTA_UNIT = 'day'
# RARE_EVENT_TIMEDELTA_UNIT: 'day', 'second', 'nanosecond', 'minute', 'hour',
# 'month', 'year' - This is the unit of time that will be used to plot the time interval
# (timedelta) between each successive rare events. If None or invalid value used, timedelta
# will be given in days.
# Notice that this parameter is referrent only to the rare events analysis with 'g' or 't' charts.
# Also, it is valid only when the timetag column effectively stores a timestamp. If the timestamp
# column stores a float or an integer (numeric) value, then the final dataframe and plot will be
# obtained in the same numeric scale of the original data, not in the unit indicated as
# RARE_EVENT_TIMEDELTA_UNIT.

X_AXIS_ROTATION = 70
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.
HORIZONTAL_AXIS_TITLE = None #Alternatively: string inside quotes for horizontal title
VERTICAL_AXIS_TITLE = None #Alternatively: string inside quotes for vertical title
PLOT_TITLE = None #Alternatively: string inside quotes for graphic title
# e.g. HORIZONTAL_AXIS_TITLE = "X", VERTICAL_AXIS_TITLE = "Y", PLOT_TITLE = "YxX"

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.
DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "" 
# or DIRECTORY_TO_SAVE = "folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = ""
FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# '{PLOT_TYPE}_plot.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.
PNG_RESOLUTION_DPI = 330
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 330.


# Processed dataframe and dataframe with values out of limits
# returned as dataset and red_df, respectively.
# Simply modify these objects on the left of equality:
dataset, red_df = statistical_process_control_chart (df = DATASET, column_with_variable_to_be_analyzed = COLUMN_WITH_VARIABLE_TO_BE_ANALYZED, timestamp_tag_column = TIMESTAMP_TAG_COLUMN, column_with_labels_or_subgroups = COLUMN_WITH_LABELS_OR_SUBGROUPS, column_with_event_frame_indication = COLUMN_WITH_EVENT_FRAME_INDICATION, specification_limits = SPECIFICATION_LIMITS, reference_value = REFERENCE_VALUE, use_spc_chart_assistant = USE_SPC_CHART_ASSISTANT, chart_to_use = CHART_TO_USE, consider_skewed_dist_when_estimating_with_std = CONSIDER_SKEWED_DISTRIBUTION_WHEN_ESTIMATING_STD, rare_event_indication = RARE_EVENT_INDICATION, rare_event_timedelta_unit = RARE_EVENT_TIMEDELTA_UNIT, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Evaluating the Process Capability (in relation to specifications)**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'analyzed_column'
# COLUMN_WITH_VARIABLE_TO_BE_ANALYZED: name (header) of the column containing the variable
# which stability will be analyzed by the control chart. The column name may be a string or a number.
# Example: COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'analyzed_column' will analyze a column named
# "analyzed_column", whereas COLUMN_WITH_VARIABLE_TO_BE_ANALYZED = 'col1' will evaluate column 'col1'.

SPECIFICATION_LIMITS = {'lower_spec_lim': None, 
                        'upper_spec_lim': None}

# SPECIFICATION_LIMITS = {'lower_spec_lim': None, 'upper_spec_lim': None}
# If there are specification limits, input them in this dictionary. Do not modify the keys,
# simply substitute None by the lower and/or the upper specification.
# e.g. Suppose you have a tank that cannot have more than 10 L. So:
# SPECIFICATION_LIMITS = {'lower_spec_lim': None, 'upper_spec_lim': 10}, there is only
# an upper specification equals to 10 (do not add units);
# Suppose a temperature cannot be lower than 10 ºC, but there is no upper specification. So,
# SPECIFICATION_LIMITS = {'lower_spec_lim': 10, 'upper_spec_lim': None}. Finally, suppose
# a liquid which pH must be between 6.8 and 7.2:
# SPECIFICATION_LIMITS = {'lower_spec_lim': 6.8, 'upper_spec_lim': 7.2}

REFERENCE_VALUE = None
# REFERENCE_VALUE: keep it as None or add a float value.
# This reference value will be shown as a red constant line to be compared
# with the plots. e.g. REFERENCE_VALUE = 1.0 will plot a line passing through
# VARIABLE_TO_ANALYZE = 1.0


X_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.
HORIZONTAL_AXIS_TITLE = None #Alternatively: string inside quotes for horizontal title
VERTICAL_AXIS_TITLE = None #Alternatively: string inside quotes for vertical title
PLOT_TITLE = None #Alternatively: string inside quotes for graphic title
# e.g. HORIZONTAL_AXIS_TITLE = "X", VERTICAL_AXIS_TITLE = "Y", PLOT_TITLE = "YxX"

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.
DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "" 
# or DIRECTORY_TO_SAVE = "folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = ""
FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'capability_plot.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.
PNG_RESOLUTION_DPI = 330
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 330.


# Summary dictionary returned as stats_dict.
# Simply modify this object on the left of equality:
stats_dict = process_capability (df = DATASET, column_with_variable_to_be_analyzed = COLUMN_WITH_VARIABLE_TO_BE_ANALYZED, specification_limits = SPECIFICATION_LIMITS, reference_value = REFERENCE_VALUE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Importing or exporting models and dictionaries (or lists)**

#### Case 1: import only a model

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_or_list_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_OR_LIST_FILE_NAME = None
# DICTIONARY_OR_LIST_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_OR_LIST_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_OR_LIST_FILE_NAME = None if no dictionary 
# or list will be manipulated.

DIRECTORY_PATH = ''
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: ""
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'arima'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE: 'tensorflow_general' for generic deep learning tensorflow models containing 
# custom layers, losses and architectures. Such models are compressed as tar.gz, tar, or zip.
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_OR_LIST_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_OR_LIST_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_OR_LIST_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

# Model object saved as model.
# Simply modify this object on the left of equality:
model = import_export_model_list_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_or_list_file_name = DICTIONARY_OR_LIST_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_or_list_to_export = DICT_OR_LIST_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY) 

#### Case 2: import only a dictionary or a list

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'dict_or_list_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_or_list_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_OR_LIST_FILE_NAME = None
# DICTIONARY_OR_LIST_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_OR_LIST_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_OR_LIST_FILE_NAME = None if no dictionary 
# or list will be manipulated.

DIRECTORY_PATH = ''
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: ""
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'arima'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE: 'tensorflow_general' for generic deep learning tensorflow models containing 
# custom layers, losses and architectures. Such models are compressed as tar.gz, tar, or zip.
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_OR_LIST_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_OR_LIST_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_OR_LIST_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

# Dictionary or list saved as imported_dict_or_list.
# Simply modify this object on the left of equality:
imported_dict_or_list = import_export_model_list_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_or_list_file_name = DICTIONARY_OR_LIST_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_or_list_to_export = DICT_OR_LIST_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY) 

#### Case 3: import a model and a dictionary (or a list)

In [ ]:
ACTION = 'import'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_and_dict'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_or_list_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_OR_LIST_FILE_NAME = None
# DICTIONARY_OR_LIST_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_OR_LIST_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_OR_LIST_FILE_NAME = None if no dictionary 
# or list will be manipulated.

DIRECTORY_PATH = ''
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: ""
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'arima'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE: 'tensorflow_general' for generic deep learning tensorflow models containing 
# custom layers, losses and architectures. Such models are compressed as tar.gz, tar, or zip.
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_OR_LIST_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_OR_LIST_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_OR_LIST_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

# Model object saved as model. Dictionary or list saved as imported_dict_or_list.
# Simply modify these objects on the left of equality:
model, imported_dict_or_list = import_export_model_list_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_or_list_file_name = DICTIONARY_OR_LIST_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_or_list_to_export = DICT_OR_LIST_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY) 

#### Case 4: export a model and/or a dictionary (or a list)

In [ ]:
ACTION = 'export'
# ACTION = 'import' for importing a model and/or a dictionary;
# ACTION = 'export' for exporting a model and/or a dictionary.

OBJECTS_MANIPULATED = 'model_only'
# OBJECTS_MANIPULATED = 'model_only' if only a model will be manipulated.
# OBJECTS_MANIPULATED = 'dict_or_list_only' if only a dictionary will be manipulated.
# OBJECTS_MANIPULATED = 'model_and_dict' if both a model and a dictionary will 
#  be manipulated.

MODEL_FILE_NAME = None
# MODEL_FILE_NAME: string with the name of the file containing the model (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. MODEL_FILE_NAME = 'model'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep MODEL_FILE_NAME = None if no model will be manipulated.

DICTIONARY_OR_LIST_FILE_NAME = None
# DICTIONARY_OR_LIST_FILE_NAME: string with the name of the file containing the dictionary 
# (for 'import');
# or of the name that the exported file will have (for 'export')
# e.g. DICTIONARY_OR_LIST_FILE_NAME = 'history_dict'
# WARNING: Do not add the file extension.
# Keep it in quotes. Keep DICTIONARY_OR_LIST_FILE_NAME = None if no dictionary 
# or list will be manipulated.

DIRECTORY_PATH = ''
# DIRECTORY_PATH: path of the directory where the model will be saved,
# or from which the model will be retrieved. If no value is provided,
# the DIRECTORY_PATH will be the root: ""
# Notice that the model and the dictionary must be stored in the same path.
# If a model and a dictionary will be exported, they will be stored in the same
# DIRECTORY_PATH.
    
MODEL_TYPE = 'arima'
# This parameter has effect only when a model will be manipulated.
# MODEL_TYPE: 'keras' for deep learning Keras/ TensorFlow models with extension .h5
# MODEL_TYPE: 'tensorflow_general' for generic deep learning tensorflow models containing 
# custom layers, losses and architectures. Such models are compressed as tar.gz, tar, or zip.
# MODEL_TYPE = 'sklearn' for models from Scikit-learn (non-deep learning)
# MODEL_TYPE = 'xgb_regressor' for XGBoost regression models (non-deep learning)
# MODEL_TYPE = 'xgb_classifier' for XGBoost classification models (non-deep learning)
# MODEL_TYPE = 'arima' for ARIMA model (Statsmodels)

DICT_OR_LIST_TO_EXPORT = None
MODEL_TO_EXPORT = None 
# These two parameters have effect only when ACTION == 'export'. In this case, they
# must be declared. If ACTION == 'export', keep:
# DICT_OR_LIST_TO_EXPORT = None, 
# MODEL_TO_EXPORT = None
# If one of these objects will be exported, substitute None by the name of the object
# e.g. if your model is stored in the global memory as 'keras_model' declare:
# MODEL_TO_EXPORT = keras_model. Notice that it must be declared without quotes, since
# it is not a string, but an object.
# For exporting a dictionary named as 'dict':
# DICT_OR_LIST_TO_EXPORT = dict

USE_COLAB_MEMORY = False
# USE_COLAB_MEMORY: this parameter has only effect when using Google Colab (or it will
# raise an error). Set as USE_COLAB_MEMORY = True if you want to use the instant memory
# from Google Colaboratory: you will update or download the file and it will be available
# only during the time when the kernel is running. It will be excluded when the kernel
# dies, for instance, when you close the notebook.
    
# If ACTION == 'export' and USE_COLAB_MEMORY == True, then the file will be downloaded
# to your computer (running the cell will start the download).

import_export_model_list_dict (action = ACTION, objects_manipulated = OBJECTS_MANIPULATED, model_file_name = MODEL_FILE_NAME, dictionary_or_list_file_name = DICTIONARY_OR_LIST_FILE_NAME, directory_path = DIRECTORY_PATH, model_type = MODEL_TYPE, dict_or_list_to_export = DICT_OR_LIST_TO_EXPORT, model_to_export = MODEL_TO_EXPORT, use_colab_memory = USE_COLAB_MEMORY) 

## **Exporting the dataframe as CSV file (to notebook's workspace)**

In [ ]:
## WARNING: all files exported from this function are .csv (comma separated values)

DATAFRAME_OBJ_TO_BE_EXPORTED = dataset
# Alternatively: object containing the dataset to be exported.
# DATAFRAME_OBJ_TO_BE_EXPORTED: dataframe object that is going to be exported from the
# function. Since it is an object (not a string), it should not be declared in quotes.
# example: DATAFRAME_OBJ_TO_BE_EXPORTED = dataset will export the dataset object.
# ATTENTION: The dataframe object must be a Pandas dataframe.

FILE_DIRECTORY_PATH = ""
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "" 
# or FILE_DIRECTORY_PATH = "folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None

NEW_FILE_NAME_WITHOUT_EXTENSION = "dataset"
# NEW_FILE_NAME_WITHOUT_EXTENSION - (string, in quotes): input the name of the 
# file without the extension. e.g. set NEW_FILE_NAME_WITHOUT_EXTENSION = "my_file" 
# to export the CSV file 'my_file.csv' to notebook's workspace.

export_pd_dataframe_as_csv (dataframe_obj_to_be_exported = DATAFRAME_OBJ_TO_BE_EXPORTED, new_file_name_without_extension = NEW_FILE_NAME_WITHOUT_EXTENSION, file_directory_path = FILE_DIRECTORY_PATH)

## **Downloading a file from Google Colab to the local machine; or uploading a file from the machine to Colab's instant memory**

#### Case 1: upload a file to Colab's workspace

In [ ]:
ACTION = 'upload'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

FILE_TO_DOWNLOAD_FROM_COLAB = None
# FILE_TO_DOWNLOAD_FROM_COLAB = None. This parameter is obbligatory when
# action = 'download'. 
# Declare as FILE_TO_DOWNLOAD_FROM_COLAB the file that you want to download, with
# the correspondent extension.
# It should not be declared in quotes.
# e.g. to download a dictionary named dict, FILE_TO_DOWNLOAD_FROM_COLAB = 'dict.pkl'
# To download a dataframe named df, declare FILE_TO_DOWNLOAD_FROM_COLAB = 'df.csv'
# To export a model named keras_model, declare FILE_TO_DOWNLOAD_FROM_COLAB = 'keras_model.h5'

# Dictionary storing the uploaded files returned as colab_files_dict.
# Simply modify this object on the left of the equality:
colab_files_dict = upload_to_or_download_file_from_colab (action = ACTION, file_to_download_from_colab = FILE_TO_DOWNLOAD_FROM_COLAB)

#### Case 2: download a file from Colab's workspace

In [ ]:
ACTION = 'download'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

FILE_TO_DOWNLOAD_FROM_COLAB = None
# FILE_TO_DOWNLOAD_FROM_COLAB = None. This parameter is obbligatory when
# action = 'download'. 
# Declare as FILE_TO_DOWNLOAD_FROM_COLAB the file that you want to download, with
# the correspondent extension.
# It should not be declared in quotes.
# e.g. to download a dictionary named dict, FILE_TO_DOWNLOAD_FROM_COLAB = 'dict.pkl'
# To download a dataframe named df, declare FILE_TO_DOWNLOAD_FROM_COLAB = 'df.csv'
# To export a model nameACTION = 'upload'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

upload_to_or_download_file_from_colab (action = ACTION, file_to_download_from_colab = FILE_TO_DOWNLOAD_FROM_COLAB)

## **Exporting a list of files from notebook's workspace to AWS Simple Storage Service (S3)**

In [ ]:
LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['s3_file1.txt', 's3_file2.txt']
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS: list containing all the files to export to S3.
# Declare it as a list even if only a single file will be exported.
# It must be a list of strings containing the file names followed by the extensions.
# Example, to a export a single file my_file.ext, where my_file is the name and ext is the
# extension:
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['my_file.ext']
# To export 3 files, file1.ext1, file2.ext2, and file3.ext3:
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['file1.ext1', 'file2.ext2', 'file3.ext3']
# Other examples:
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['Screen_Shot.png', 'dataset.csv']
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ["dictionary.pkl", "model.h5"]
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['doc.pdf', 'model.dill']

DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = ''
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT: directory from notebook's workspace
# from which the files will be exported to S3. Keep it None, or
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = "/"; or
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = '' (empty string) to export from
# the root (main) directory.
# Alternatively, set as a string containing only the directories and folders, not the file names.
# Examples: DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = 'folder1';
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = 'folder1/folder2/'
    
# For this function, all exported files must be located in the same directory.

S3_BUCKET_NAME = 'my_bucket'
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

S3_OBJECT_FOLDER_PREFIX = ""
# S3_OBJECT_FOLDER_PREFIX = None. Keep it None; or as an empty string 
# (S3_OBJECT_FOLDER_PREFIX = ''); or as the root "/" to import the 
# whole bucket content, instead of a single object from it.
# Alternatively, set it as a string containing the subfolder from the bucket to import:
# Suppose that your bucket (admin-created) has four objects with the following object 
# keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
# s3-dg.pdf. 
# The s3-dg.pdf key does not have a prefix, so its object appears directly 
# at the root level of the bucket. If you open the Development/ folder, you see 
# the Projects.xlsx object in it.
# In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
# where 'bucket' is the bucket's name, prefix = 'my_path/.../', without the
# 'file.csv' (file name with extension) last part.

# So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
# a given folder (directory) of the bucket.
# DO NOT PUT A SLASH before (to the right of) the prefix;
# DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

# Alternatively, provide the full path of a given file if you want to import only it:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
# where my_file is the file's name, and ext is its extension.


# Attention: after running this function for connecting with AWS Simple Storage System (S3), 
# your 'AWS Access key ID' and your 'Secret access key' will be requested.
# The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
# other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
# and the prefix. All of these are sensitive information from the organization.
# Therefore, after importing the information, always remember of cleaning the output of this cell
# and of removing such information from the strings.
# Remember that these data may contain privilege for accessing protected information, 
# so it should not be used for non-authorized people.

# Also, remember of deleting the imported files from the workspace after finishing the analysis.
# The costs for storing the files in S3 is quite inferior than those for storing directly in the
# workspace. Also, files stored in S3 may be accessed for other users than those with access to
# the notebook's workspace.
export_files_to_s3 (list_of_file_names_with_extensions = LIST_OF_FILE_NAMES_WITH_EXTENSIONS, directory_of_notebook_workspace_storing_files_to_export = DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT, s3_bucket_name = S3_BUCKET_NAME, s3_obj_prefix = S3_OBJECT_FOLDER_PREFIX)

****